In [1]:
import numpy as np
import pandas as pd
import xlwings as xw
import math
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time 

from collections import Counter
from numpy import nan as NA
from dateutil.parser import parse
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
# module_dir = os.path.join(current_dir, 'Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Data_Clean, Risk_Data
pd.set_option('display.max_columns', None)
# 实例化All_Model类
all_models = All_Model()
clean = Data_Clean()
risk = Risk_Data()

In [2]:
def query(sql,
          host="rm-wz930e5269fur1ht1mo.mysql.rds.aliyuncs.com",
          user="huangxq",
          password="f_epz6hDmZ0Hzi0OzI0N",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 1.样本数据整理

## 1_11.获取_租后数据

In [3]:
sql = '''-- 租后应收监控  
SELECT  tprm.* 
,om.`status` as 订单状态值
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as  订单状态
,om.order_finish_date as 订单完成时间,om.order_method
,tmu.true_name ,tmu.id_card_num,tmu.mobile 
,tod.product_name, om.order_type, cc.channel_type_id
from db_digua_business.t_postlease_receivables_monitoring   tprm 
left join db_digua_business.t_order om on tprm.order_id = om.id
left join  db_digua_business.t_member_user tmu on om.user_id = tmu.id
left join  db_digua_business.t_order_details tod on tprm.order_id = tod.order_id
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- where om.create_time >= date_add(current_date, interval -3 month)
;
'''

df_zhys = query(sql)
# (27787, 62)
df_zhys.shape

(48784, 65)

## 1_12. 剔除商家数据

In [4]:
# df_zhys["年"] = df_zhys["下单月份"].str[:4]
# pd.crosstab(dfzh1["商家名称"],dfzh1["年"],margins=True).loc[["北京海鸟窝科技有限公司","深圳优优大数据科技有限公司","租着用电脑数码","苏州蚁诺宝"],:]
# 剔除商家数据
def drop_sj(df_zhys,col1,col2):
    df_zhys.drop(df_zhys[df_zhys[col1]=="深圳优优大数据科技有限公司"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="优优2店"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="小豚租（代收）"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="苏州蚁诺宝"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="租着用电脑数码"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="北京海鸟窝科技有限公司"].index,inplace=True)

    df_zhys.drop(df_zhys[df_zhys[col1]=="汇客好租"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="澄心优租"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="CPS渠道合作"].index,inplace=True)
    
    df_zhys.drop(df_zhys[df_zhys[col1]=="趣智数码"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="格木木二奢名品"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="广州康基贸易有限公司"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="线下小店"].index,inplace=True)  # 计划
    df_zhys.drop(df_zhys[df_zhys[col1]=="小蚂蚁租机"].index,inplace=True)  #拒量订单
    df_zhys.drop(df_zhys[df_zhys[col1]=="兴鑫兴通讯"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="人人享租"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="崇胜数码"].index,inplace=True)


    df_zhys.drop(df_zhys[df_zhys[col2].str.contains(pat='探路者',regex=False)==True].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys['merchant_name'] == "乙辉数码"].index, inplace=True)
    return df_zhys

df_zhys = drop_sj(df_zhys,'merchant_name','model_number')
df_zhys.shape

(45898, 65)

## 1_13.历史订单渠道名称补充

In [5]:
# 1148 条历史订单渠道名称补充
qudao_name_df = pd.read_excel('F:/myfile/租后数据/历史订单渠道名称补充.xlsx')
qudao_name_df.drop(columns=['渠道id','下单时间'],inplace = True)

# dfzh1 = df_zh.merge(qudao_name_df,on = '订单号',how = 'left')
dfzh1 =  df_zhys.merge(qudao_name_df,left_on = 'order_number',right_on = '订单号',how = 'left')
# dfzh1.columns

# qudao_name_df.shape
# qudao_name_df[:2]
# qudao_name_df['渠道名称'].value_counts()

# 渠道名称缺失补充
# dfzh1["渠道名称"] = dfzh1["渠道名称_x"].fillna(dfzh1.渠道名称_y)
dfzh1["channel_name"] = np.where(dfzh1.渠道名称.notnull(),dfzh1.渠道名称,dfzh1["channel_name"])
dfzh1["channel_name"].isnull().sum()

# dfzh1[['order_number','channel_name','渠道名称']][:10]
# df_zhys[df_zhys['order_number'] == 'A202207131230485'][['order_number','channel_name']]

np.int64(0)

## 1_14.渠道归属

In [6]:
# df.loc[:,"归属渠道"]=df.apply(lambda x:qudao_type(x["来源渠道"],x["activity_name"],x["order_method"]),axis=1)
dfzh1["activity_source"] = dfzh1["activity_source"].fillna('未知活动') 
dfzh1["channel_name"] = dfzh1["channel_name"].fillna('未知渠道') 

dfzh1.loc[:,"归属渠道"]=dfzh1.apply(lambda x: clean.qudao_type(x["channel_name"],x["activity_source"],x["order_method"], x['channel_type_id'], x['order_type']),axis=1)

# pd.crosstab(dfzh1["下单月份"],dfzh1["归属渠道"],margins=True)

## 1_15.判断颜色 内存

In [7]:
dfzh1["内存"]=dfzh1["specification"].str.split("内存：").str[1].str.split("颜色：").str[0].str.split(" ").str[0]
# dfzh1["内存"].value_counts()[14:]

dfzh1["颜色"]=dfzh1["specification"].str.split("内存：").str[1].str.split("颜色：").str[1].str.split(" ").str[0]
# dfzh1["颜色"].value_counts()[1:10]

## 1_16.判断二手、带锁

In [8]:
# dfzh1['是否二手'] = np.where(dfzh1['model_number'].str.contains('95新')|dfzh1['model_number'].str.contains('99新')|dfzh1['model_number'].str.contains('准新'),"二手","全新")
# note 20241124 修改口径，用product_name判断是否二手
dfzh1['是否二手'] = np.where(dfzh1['product_name'].str.contains('95新')|dfzh1['product_name'].str.contains('99新')|dfzh1['product_name'].str.contains('准新'),"二手","全新")
dfzh1['是否二手'].value_counts()

是否二手
全新    44692
二手     1206
Name: count, dtype: int64

In [9]:
#读取导入最新出库台账数据,2023.9及以后的数据
# （1）512 带锁
# （2）非512的多头带锁
# （3）抖音带锁
# note 20241204
# （1）512 带锁，但每天豁免一台 512P 不带锁，目前还没量
# （2）非512的命中多头提示带锁，但出库是否带锁由人工控制
# （3）抖音带锁（苹果11-15）
# 维客壹佰2023&2024年台账1204.xlsz 为截止到20241203的数据
# f_path_ck = "维客壹佰2023&2024年台账1204.xlsx"
f_path_ck = "维客壹佰2023&2024年台账.xlsx"
path = 'F:/myfile/p站数据/台账数据/'
dfck = pd.read_excel(path + f_path_ck,sheet_name="2023")   #,skiprows=0 header 

# dfck['是否带锁']  = np.where((dfck['型号'].str.contains(r'带锁|有锁'))|(dfck['内存']=='512G')|(dfck['内存']=='512GB'), 1, 0)
dfck['是否带锁']  = np.where((dfck['型号'].str.contains(r'带锁|有锁'))|(dfck['内存'].str.contains(r'512')), '带锁', '未带锁')
dfck = dfck[[u'订单号',u'是否带锁']]
dfck.shape
dfck['是否带锁'].value_counts()

dfck.drop_duplicates(subset=["订单号"],inplace=True)

# 合并到主表中去
# dfck 的带锁不一定出库，dfzh1 是出库的
dfck = dfck.rename(columns= {'订单号': 'order_number'})
dfzh1 = dfzh1.merge(dfck, on = 'order_number', how = 'left')
dfzh1['是否带锁'] = dfzh1['是否带锁'].fillna('未带锁') 
dfzh1.shape

pd.pivot_table(dfzh1,index='是否二手',columns='是否带锁',values='order_id',aggfunc='count')

(40549, 2)

是否带锁
未带锁    37477
带锁      3072
Name: count, dtype: int64

(45898, 72)

是否带锁,带锁,未带锁
是否二手,,
二手,511,695
全新,2212,42480


## 1_17.日期处理

In [10]:
dfzh1["下单日期"] = dfzh1['order_create_time'].dt.date
dfzh1["下单日期"] = pd.to_datetime(dfzh1["下单日期"],errors="coerce")
dfzh1["下单日期"].head()

dfzh1["月份"]=dfzh1["下单日期"].dt.month
dfzh1["年份"]=dfzh1["下单日期"].dt.year
# df_total["下单月份"].value_counts()

0   2022-07-05
1   2022-07-13
2   2022-07-15
3   2022-07-16
4   2022-07-17
Name: 下单日期, dtype: datetime64[ns]

In [11]:
def getjidu(s):
    if s<4:
        return "1季度"
    elif 4<=s<7:
        return "2季度"
    elif 7<=s<10:
        return "3季度"
    else:
        return "4季度"
dfzh1["季度"]= dfzh1.apply(lambda x:getjidu(x["月份"]),axis=1)  
dfzh1["季度"].value_counts()

季度
2季度    15536
4季度    11205
3季度     9679
1季度     9478
Name: count, dtype: int64

## 1_21.获取_买断数据

In [12]:
sql1 = ''' --   买断信息 
with out_order as ( 
SELECT too.order_id,too.`status` 
,too.real_pay_money ,too.create_time,too.update_time
,too.pay_date ,too.actual_money 
,rank() over(partition by too.order_id order by too.pay_date desc) as rn 
from db_digua_business.t_order_out too
where  too.`status` not in (1)   -- 买断状态：1 未买断 2 已买断  3 部分买断 
-- and too.pay_status= 2    -- '支付状态 1、待支付 2、已支付 3、退款中 4、已退款 6、支付取消 7、已关闭',
-- GROUP BY 1 ,2
-- ORDER BY 5 desc,1
)
# note 20241126 增加distinct，买断可能分多次付款
# 181950     2
# 338547     2
# 1083822    2
# 303756     2
SELECT distinct t1.*
,t2.`status`
,case when t2.`status`=2 then '已买断' when  t2.`status`=3 and om.order_finish_date is not null then '已买断'  else '部分买断' end as if_outpay 
,om.order_finish_date
,case when t2.`status`=2 and om.order_finish_date is null then t1.pay_date else om.order_finish_date end as finish_date_new 
from (
SELECT too.order_id  -- ,too.`status` 
-- ,count(DISTINCT too.`status`) nn 
,sum(too.real_pay_money) as outpay_money     -- ,max(too.update_time) as outpay_time
,max(too.pay_date) as pay_date 
from out_order  too
GROUP BY 1 -- ,2
ORDER BY 2 desc ,1) t1 
left join out_order t2 on t1.order_id = t2.order_id and t2.rn=1 
left join db_digua_business.t_order om on t1.order_id = om.id 
;
'''

df_out = query(sql1)
df_out.shape

(5147, 7)

## 1_22.租后数据 关联 买断数据

In [13]:
dfzh2 = dfzh1.merge(df_out[['order_id', 'outpay_money', 'pay_date', 'status', 'if_outpay']], on = 'order_id', how = 'left')
dfzh2.shape
dfzh1.shape

(45898, 80)

(45898, 76)

In [14]:
dfzh2['finish_date_new'] = dfzh2.apply(lambda x:pd.to_datetime(x.pay_date, errors='coerce') if pd.isnull(x.order_finish_date) and x.status==2
                                       else pd.to_datetime(x.订单完成时间, errors='coerce') if pd.isnull(x.order_finish_date)  and pd.notnull(x.订单完成时间)
                                       else pd.to_datetime(x.order_finish_date, errors='coerce'),axis = 1 ) 
# dfzh2['finish_date_new'] = np.where((pd.isnull(dfzh2.order_finish_date)) & (dfzh2.status==2),pd.to_datetime(dfzh2.pay_date, errors='coerce'),pd.to_datetime(dfzh2.order_finish_date, errors='coerce'))
dfzh2['finish_date_new'].notnull().sum()

np.int64(6388)

In [15]:
dfzh2['if_outpay'] = dfzh2.apply(lambda x: '已买断' if x['status'] ==2 
                                 else  '已买断'  if  x['status'] ==3 and pd.notnull(x.finish_date_new) 
                                 else '部分买断' if x['status'] ==3 and pd.isnull(x.finish_date_new) 
                                 else '未知',axis =1)
dfzh2['if_outpay'].value_counts()

if_outpay
未知      40965
已买断      4827
部分买断      106
Name: count, dtype: int64

## 1_31.获取_分期取消数据

In [16]:
sql1 = '''-- 订单分期表  取消的分期
SELECT tos.order_id,count(*) as nn
,count(tos.reality_refund_date) as  reality_refund_date
,sum(tos.real_pay_money) as real_pay_money
,sum(tprm.purchase_amount)/count(tos.order_id) as purchase_amount
,avg(tprm.advance_periods) as  advance_periods
,avg(tprm.advance_sum) as advance_sum
from db_digua_business.t_order_stages tos 
inner join db_digua_business.t_postlease_receivables_monitoring   tprm 
on tos.order_id = tprm.order_id 
# 订单取消
where tos.`status`=4 
GROUP BY 1 order by 2 desc 
;
'''

df_cancel = query(sql1)
df_cancel.shape

(2751, 7)

## 1_32.租后数据关联分期取消数据

In [17]:
dfzh3 = dfzh2.merge(df_cancel[['order_id', 'nn', 'reality_refund_date', 'real_pay_money']],on = 'order_id',how = 'left')
dfzh3.shape
dfzh2.shape

# 不同还款状态下  完成时间的情况
# dfzh3.groupby(['rembursement_status']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count','finish_date_new':'count','pay_date':'count','nn':'count','outpay_money':'sum'})

# 不同订单状态下  完成时间的情况
# dfzh3.groupby(['订单状态']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count','finish_date_new':'count','pay_date':'count','nn':'count','outpay_money':'sum'})

# 有取消记录的用户分布情况
# dfzh3[dfzh3.nn>=1].groupby(['rembursement_status']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count'
                                                        #  ,'finish_date_new':'count','pay_date':'count','nn':'count','real_pay_money':'sum'})

# 有取消记录的用户分布情况
# dfzh3[dfzh3.nn>=1].groupby(['订单状态']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count'
                                                        #  ,'finish_date_new':'count','pay_date':'count','nn':'count','real_pay_money':'sum'})

(45898, 84)

(45898, 81)

## 1_41.按风控策略调整时间节点划分时间段

In [18]:
#按风控策略调整时间节点划分时间段
dfzh1.loc[:"2023-01-09","时间段"]=" -2023.1.9前"
dfzh1.loc["2023-01-10":"2023-03-03","时间段"]="2023.1.10-2023.3.3"
dfzh1.loc["2023-03-04":"2023-03-24","时间段"]="2023.3.4-2023.3.24"
dfzh1.loc["2023-03-25":"2023-04-28","时间段"]="2023.3.25-2023.4.28"
dfzh1.loc["2023-04-29":"2023-05-17","时间段"]="2023.4.29-2023.5.17"
dfzh1.loc["2023-05-18":"2023-05-31","时间段"]="2023.5.18-2023.5.31"
dfzh1.loc["2023-06-01":"2023-06-30","时间段"]="2023.6.1-2023.6.30"
dfzh1.loc["2023-07-01":"2023-08-11","时间段"]="2023.7.1-2023.8.11"
dfzh1.loc["2023-08-12":"2023-09-15","时间段"]="2023.8.12-2023.9.15"
dfzh1.loc["2023-09-16":"2023-09-22","时间段"]="2023.9.16-2023.9.22"
dfzh1.loc["2023-09-23":"2023-11-16","时间段"]="2023.9.23-2023.11.16"
dfzh1.loc["2023-11-17":"2023-11-28","时间段"]="2023.11.17-2023.11.28"
dfzh1.loc["2023-11-29":"2023-12-05","时间段"]="2023.11.29-2023.12.5"
dfzh1.loc["2023-12-06":"2024-01-05","时间段"]="2023.12.06-2024.1.5"
dfzh1.loc["2024-01-06":"2024-01-23","时间段"]="2024.1.6-2024.1.23"
dfzh1.loc["2024-01-24":"2024-2-21","时间段"]="2024.1.24-2024.2.21"
dfzh1.loc["2024-02-22":"2024-02-29","时间段"]="2024.2.22-2024.2.29"
dfzh1.loc["2024-03-01":"2024-03-12","时间段"]="2024.3.1-2024.3.12"
dfzh1.loc["2024-03-13":"2024-04-03","时间段"]="2024.3.13-2024.4.3"
dfzh1.loc["2024-04-04":"2024-04-10","时间段"]="2024.4.4-2024.4.10"
dfzh1.loc["2024-04-11":"2024-04-18","时间段"]="2024.4.11-2024.4.18"
dfzh1.loc["2024-04-19":"2024-04-24","时间段"]="2024.4.19-2024.4.24"
dfzh1.loc["2024-04-25":"2024-04-27","时间段"]="2024.4.25-2024.4.27"
dfzh1.loc["2024-04-28":,"时间段"]="2024.4.28- "

## 1_51.获取_账期数据（含续租）

In [19]:
sql_xz = ''' 
-- 状态,1:未支付  2:扣款失败 3:已支付 4：已取消 5、申请扣款、6已退款',
-- '是否续租账期,0非续租账期，1预授权账期，2自定义续租账期',
SELECT ymos.*
from db_rent.ya_merchant_order_stages ymos 
inner join db_digua_business.t_postlease_receivables_monitoring tprm 
on ymos.order_id = tprm.order_id 
;
'''

df_xzfq = query(sql_xz)
df_xzfq.shape

(604771, 23)

In [20]:
# note 20241227 号卡 应付日期更新
# 分期表
sql = '''-- 租后应收监控  
SELECT distinct hk.order_number, tos.order_id, tos.refund_date as refund_date_1
from db_digua_business.t_order_stages tos
inner join
    (select distinct om.id as order_id, om.order_number
    from db_digua_business.t_order om
    left join db_digua_business.t_platform_activity tpa 
    on tpa.id=om.activity_id
    where tpa.type = 4) hk
on tos.order_id = hk.order_id
'''
tmp_hk = query(sql)
tmp_hk['refund_date_ym'] = tmp_hk['refund_date_1'].dt.strftime('%Y-%m')
tmp_hk.shape
tmp_hk[:2]

df_xzfq['refund_date_ym'] = df_xzfq['refund_date'].dt.strftime('%Y-%m')
df_xzfq = df_xzfq.merge(tmp_hk[['order_id', 'refund_date_1', 'refund_date_ym']], on = ['order_id', 'refund_date_ym'], how = 'left')
df_xzfq['refund_date_2'] = np.where(df_xzfq.refund_date_1.notnull(), df_xzfq.refund_date_1, df_xzfq.refund_date)
df_xzfq.shape

df_xzfq.drop(columns=['refund_date_1','refund_date_ym','refund_date'],inplace = True)
df_xzfq = df_xzfq.rename(columns = {'refund_date_2':'refund_date'})
df_xzfq.shape

(4872, 4)

,order_number,order_id,refund_date_1,refund_date_ym
0,A20240927154904302,1533488,2024-09-30,2024-09
1,A20240927154904302,1533488,2024-10-30,2024-10


(604771, 26)

(604771, 23)

In [21]:
# 数据检查
# tmp1 = df_xzfq.drop_duplicates(subset = ['order_id'])[['order_id']]
# tmp2 = tmp_hk.drop_duplicates(subset = ['order_id'])[['order_id', 'order_number']]
# tmp1.shape
# tmp2.shape

# tmp3 = tmp1.merge(tmp2, on = 'order_id', how = 'inner')
# tmp3.shape

# tmp4 = tmp3.merge(df_xzfq, on = 'order_id', how = 'inner')
# tmp4.shape

# tmp4["日"]=tmp4["refund_date"].dt.day
# tmp4["日"].value_counts()
# tmp4[tmp4.日.isin([2,11])][['order_number','refund_date']]

# df_xzfq[df_xzfq.refund_date_2 != df_xzfq.refund_date].shape
# df_xzfq[df_xzfq.refund_date_2 == df_xzfq.refund_date].shape
# df_xzfq[df_xzfq.refund_date_2 != df_xzfq.refund_date][['order_id', 'refund_date', 'refund_date_1', 'refund_date_2', 'refund_date_ym']]
# 以下四个订单的应付日期<15号，订单分期表更正了  正常的订单的应付日期
# A20240929094315174 1539249  已退款
# A2024092919033313  1541755  正常
# A2024092919182056  1541798  已退款
# A2024100810242488  1568606  已退款

# tmp_hk[tmp_hk.order_number.isin(['A2024100810242488'])]
# df_xzfq[df_xzfq.order_id.isin([1539249,1541755,1541798,1568606])][['order_id', 'refund_date', 'refund_date_1', 'refund_date_2', 'refund_date_ym']]

In [22]:
# df_xz_concat[df_xz_concat.order_number.isin(['A2024080919264964','A2024092919033313','A20241218184711224'])][['order_number','应付日期','实付日期','逾期天数']]
# A2024080919264964
# A2024092919033313
# tmp = df_xz_concat[df_xz_concat.逾期天数 > 30].drop_duplicates(subset=['order_number'])[['order_number']]
# tmp = tmp.merge(df_hk, on = 'order_number', how = 'inner')
# tmp.shape
# tmp

## 1_52.重命名

In [23]:
df_xzfq = df_xzfq.rename(columns={'money':'当前应付租金','part_payment':'实付金额','refund_date':'应付日期','reality_refund_date':'实付日期','sort':'当前期数'})

## 1_53.实还金额核查

In [24]:
# total_check = df_ddfq.merge(df_xzfq,on=['订单ID','当前期数'],how='inner')
# total_check.info()

# total_check1= total_check[(total_check.实付金额_y < total_check.实付金额_x)][['订单ID','订单号','应付日期_x','实付日期_x','实付日期_y','实付金额_x','实付金额_y','状态','status']]    #   #(total_check.应付日期_x.notnull()) & (total_check.应付日期_y.isnull())
# total_check1= total_check[(total_check.实付金额 < total_check.实际支付金额)][['订单ID','订单号','还款日期','实际还款日期','实付日期','实际支付金额','实付金额','状态','status']]
# total_check1

# total_check2= total_check[(total_check.实付金额_y > total_check.实付金额_x)][['订单ID','订单号','应付日期_x','实付日期_x','实付日期_y','实付金额_x','实付金额_y','状态','status']]    #   #(total_check.应付日期_x.notnull()) & (total_check.应付日期_y.isnull())
# total_check2= total_check[(total_check.实付金额 > total_check.实际支付金额)][['订单ID','订单号','还款日期','实际还款日期','实付日期','实际支付金额','实付金额','状态','status']]
# total_check2

# total_check2.to_excel('E:/myfile/业务相关数据/total_check_0709-1.xlsx')

## 1_61.数据合并

In [25]:
df_xz_concat = df_xzfq.merge(dfzh3,how='right',on ='order_id')
# df_xz_concat = df_fq.merge(dfzh1,how='right',right_on ='订单id',left_on='订单ID')
# note 20241126 增加
# df_xz_concat[df_xz_concat.当前应付租金.isnull()].shape
# df_xz_concat[df_xz_concat.当前应付租金.isna()].shape
# 实付日期为空，但当前应付租金为空
# 1681669 A20241104144812228
df_xz_concat.实付金额 = df_xz_concat.实付金额 + df_xz_concat.sesame_promo_money_pay + df_xz_concat.promo_money
df_xz_concat = df_xz_concat[df_xz_concat.当前应付租金.notnull()]

# order_id == 1681669 的当前期数为空
# 1681669	A20241104144812228
# 1669581	A2024110109350797
df_xz_concat['当前期数'].notnull().sum()
df_xz_concat.shape
df_xzfq.shape
dfzh3.shape

# df_xz_concat.drop_duplicates(subset=["order_id"]).shape
# dfzh3.drop_duplicates(subset=["order_id"]).shape

# df_xz_concat['order_id'].nunique()
# dfzh3['order_id'].nunique()
# df_xz_concat[["order_id","当前期数"]].info()

np.int64(604920)

(604920, 106)

(604771, 23)

(45898, 84)

## 1_62.去重

In [26]:
check = df_xz_concat[["order_id","当前期数"]]  # ,"逾期天数"
all_duplicates = check[check.duplicated(keep=False)]  
# all_duplicates.to_excel('E:/myfile/业务相关数据/all_duplicates.xlsx')
print(all_duplicates.sort_values(by=['order_id','当前期数']))

        order_id  当前期数
146271    869632   1.0
146283    869632   1.0
435207    869632   1.0
435219    869632   1.0
146272    869632   2.0
...          ...   ...
484775   2040621  23.0
431999   2040621  24.0
432023   2040621  24.0
484752   2040621  24.0
484776   2040621  24.0

[672 rows x 2 columns]


In [27]:
#删除重复订单
df_xz_concat.drop_duplicates(subset=["order_id","当前期数"],inplace=True)
df_xz_concat.shape

(604416, 106)

In [28]:
# 119321
# 1083822 303756
# all_duplicates[all_duplicates['order_id'] == 181950].shape
# df_xz_concat[df_xz_concat['order_id'] == 181950].shape

# df_xz_concat.to_excel('E:/myfile/业务相关数据/df_xz_concat.xlsx')

# 2.计算逾期天数

## 2_11.异常查询

In [29]:
# df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.status_x==1)][['order_number', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new']]

# 实付金额>0 实付日期为空 finish_date_new非空 :实付日期=订单完成时间
# df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.实付日期.isnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new']]

# part_payment==0 reality_refund_date.notnull() 还款状态== '续租中' part_payment=money 还款状态== '已逾期' reality_refund_date=pd.na
# dd_test = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test

# dd_test = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test

# 实付为0，实付日期非空，完成时间为空，无期数取消：实付日期清空
# dd_test = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test.订单状态值.value_counts()

# 实付日期为空，完成时间为空，取消期数>=1 ：状态=已取消
# dd_test = df_xz_concat[ (df_xz_concat.实付日期.isnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn>=1)][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test.订单状态值.value_counts()

# 部分还款，实付日期非空，完成时间为空 用户： 实付日期清空
# dd_test1 = df_xz_concat[(df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.notnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test1

# 部分还款，实付日期为空，完成时间非空 用户： 实付日期=完成时间
# dd_test1 = df_xz_concat[(df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.isnull()) & (df_xz_concat.finish_date_new.notnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test1.订单状态值.value_counts()

# 已买断 但无实付日期及金额：实付日回写
# dd_test1 = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.isnull()) & (df_xz_concat.finish_date_new.notnull())& (df_xz_concat.outpay_money>0)][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]
# # dd_test1.订单状态值.value_counts()
# dd_test1

## 2_12.实还日期修正

In [30]:
df_xz_concat['实付日期'] = pd.to_datetime(df_xz_concat['实付日期']).dt.date
df_xz_concat['finish_date_new'] = pd.to_datetime(df_xz_concat['finish_date_new']).dt.date

In [31]:
# （1）部分还款 用户 实付日期非空 且非买断 非完成 ： 实付日期清空  
df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.notnull()) & (df_xz_concat.finish_date_new.isnull()),pd.NaT,df_xz_concat.实付日期)

df_xz_concat[(df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.notnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']].tail()     

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
347021,A2024110708472957,1692436,457.98,457.98,2025-04-10,2025-04-06,3.0,6.0,还款中,NaT,4,NaT,NaN,NaT
347022,A2024110708472957,1692436,457.98,457.98,2025-05-10,2025-05-10,3.0,7.0,还款中,NaT,4,NaT,NaN,NaT
478429,A2025032020360980,2259696,666.32,666.32,2025-03-23,2025-03-20,3.0,1.0,还款中,NaT,4,NaT,NaN,NaT
478430,A2025032020360980,2259696,666.32,666.32,2025-04-23,2025-04-19,3.0,2.0,还款中,NaT,4,NaT,NaN,NaT
478431,A2025032020360980,2259696,666.32,666.32,2025-05-23,2025-05-19,3.0,3.0,还款中,NaT,4,NaT,NaN,NaT


In [32]:
# （2）已完成 还款 用户  实付日期 补充 订单完成时间       实付金额>0  实付日期为空  finish_date_new非空  :实付日期=订单完成时间finish_date_new
df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额>0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.notnull()),df_xz_concat.finish_date_new,df_xz_concat.实付日期1)

df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']].tail()       

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
384101,A20241210113506313,1839860,695.21,665.65,2025-05-13,NaT,1.0,6.0,还款中,NaT,4,NaT,NaN,NaT
451644,A20250223144805320,2133904,974.62,300.00,2025-05-26,NaT,1.0,4.0,还款中,NaT,4,NaT,8.0,NaT
478429,A2025032020360980,2259696,666.32,666.32,2025-03-23,2025-03-20,3.0,1.0,还款中,NaT,4,NaT,NaN,NaT
478430,A2025032020360980,2259696,666.32,666.32,2025-04-23,2025-04-19,3.0,2.0,还款中,NaT,4,NaT,NaN,NaT
478431,A2025032020360980,2259696,666.32,666.32,2025-05-23,2025-05-19,3.0,3.0,还款中,NaT,4,NaT,NaN,NaT


In [33]:
# （3）租完即送用户 A2023081019051714 补还款时间 
df_xz_concat['实付日期1'] = np.where((df_xz_concat.order_number=='A2023081019051714') & (df_xz_concat.当前期数==12),'2024-07-22',df_xz_concat['实付日期1'])
df_xz_concat[(df_xz_concat.order_number=='A2023081019051714') & (df_xz_concat.当前期数==12)][['实付日期1']]

,实付日期1
60964,2024-07-22


In [34]:
# （4）未还款 且 未完成 未取消用户 ： 实付日期清空
df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn.isnull()),pd.NaT,df_xz_concat.实付日期1)

df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']].tail()

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
8862,A2023021015233353,125911,633.21,0.0,2023-03-13,2023-03-18,2.0,2.0,已逾期,NaT,5,NaT,NaN,NaT
9558,A2023021414250354,132517,334.26,0.0,2023-03-17,2023-03-18,2.0,2.0,已逾期,NaT,5,NaT,NaN,NaT
9636,A2023021504342232,133713,396.66,0.0,2023-03-18,2023-03-18,2.0,2.0,已逾期,NaT,5,NaT,NaN,NaT
10650,A2023022114312437,145022,556.04,0.0,2023-03-24,2023-03-20,2.0,2.0,已逾期,NaT,5,NaT,NaN,NaT
12255,A2023030700325746,166163,288.77,0.0,2024-01-10,2023-04-17,1.0,11.0,已买断,NaT,8,NaT,NaN,NaT


In [35]:
# （5）实付为0，实付日期为空，但 完成时间非空，买断金额>0： 实付日期回写

df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额==0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.notnull())& (df_xz_concat.outpay_money>0),df_xz_concat.finish_date_new,df_xz_concat.实付日期1)

df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期1.notnull()) & (df_xz_concat.finish_date_new.notnull())& (df_xz_concat.outpay_money>0)][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']] .tail()

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
223303,A2024060716074819,1111255,1085.68,0.0,2025-05-10,NaT,1.0,12.0,已买断,2024-12-14 14:59:21,8,2024-12-14,7.0,2024-12-14
231547,A20240620152351112,1150772,770.50,0.0,2026-02-24,NaT,1.0,21.0,已买断,2025-04-01 09:53:49,8,2025-04-01,NaN,2025-04-01
231548,A20240620152351112,1150772,514.32,0.0,2026-03-24,NaT,1.0,22.0,已买断,2025-04-01 09:53:49,8,2025-04-01,NaN,2025-04-01
447127,A2025021919082939,2114551,974.62,0.0,2025-12-22,NaT,1.0,11.0,已买断,2025-05-22 10:15:51,8,2025-05-22,9.0,2025-05-22
505548,A20250411105344370,2386604,1082.94,0.0,2026-02-14,NaT,1.0,11.0,已买断,2025-04-19 12:35:48,8,2025-04-19,10.0,2025-04-19


In [36]:
# （6）部分还款 
# df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期1'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

In [37]:
# （7）未完成 但含有分期取消的 订单
# check = df_xz_concat[(df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.isnull()) & df_xz_concat.nn>=1][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期1'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]
# check.订单状态值.value_counts()

In [38]:
# （8）实付0，实付日期为空，但 完成时间非空：为取消订单，实付日期=完成时间
df_xz_concat['实付日期new'] = np.where( (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.notnull()),df_xz_concat.finish_date_new,df_xz_concat.实付日期1)
df_xz_concat[ (df_xz_concat.实付日期new.isnull()) & (df_xz_concat.finish_date_new.notnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期1'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期1,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,outpay_money


In [39]:
# （9）属于部分还款 :无实付日期
# dd_test2 = df_xz_concat[(df_xz_concat.实付金额 >0) & (df_xz_concat.实付日期new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期new'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]
# dd_test2

In [40]:
# （10）df_xz_concat[df_xz_concat.应付日期.isnull()][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期new'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

In [41]:
# （11）A202207050851373、A20241104144812228 应付日期修复
df_xz_concat['应付日期'] = np.where((df_xz_concat.应付日期.isnull()) & (df_xz_concat.order_number.isin(['A202207050851373','A20241104144812228'])),pd.to_datetime('2023-07-05') ,df_xz_concat.应付日期 )
df_xz_concat[df_xz_concat.应付日期.isnull()][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期new'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期new,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,outpay_money


In [42]:
# （12）实付金额>0  实付日期.isnull()  还款状态==已买断  :  逾期天数=0  实付日期=订单完成时间
# 实付金额>0  实付日期.isnull()  还款状态==已逾期  :实付金额 =0  
# 实付金额==0  实付日期.notnull()  还款状态== '续租中' 实付金额=当前应付租金   还款状态== '已逾期' 实付日期=pd.na
# 0<实付金额<当前应付租金  实付日期.notnull()  还款状态== '已逾期'  :  实付金额=0  & 实付日期=pd.na

# df_xz_concat['实付金额new'] = df_xz_concat.apply(lambda x:
#     0 if x.实付金额>0 and pd.isna(x.实付日期) and x.还款状态=='已逾期'
#     else 0 if x.实付金额>0 and x.实付金额<x.当前应付租金 and pd.notna(x.实付日期) and x.还款状态=='已逾期'
#     else x.当前应付租金 if x.实付金额==0 and pd.notna(x.实付日期) and x.还款状态=='续租中'
#     else x.实付金额
#     ,axis = 1)

In [43]:
# （13）df_xz_concat['实付日期new'] = df_xz_concat.apply(lambda x:
#     x.订单完成时间 if x.实付金额>0 and pd.isna(x.实付日期) and x.还款状态=='已买断'
#     else np.nan if x.实付金额>=0 and x.实付金额 < x.当前应付租金 and pd.notna(x.实付日期) and x.还款状态=='已逾期'
#     else x.实付日期
#     ,axis = 1)

## 2_13.获取逾期截止时间

In [44]:
df_xz_concat['now_date'] = pd.Timestamp(dt.now().date() )
df_xz_concat['now_date'].head()

0   2025-06-06
1   2025-06-06
2   2025-06-06
3   2025-06-06
4   2025-06-06
Name: now_date, dtype: datetime64[s]

## 2_14.重新定义状态

In [45]:
df_xz_concat[(df_xz_concat.实付日期new.isnull()) & (~df_xz_concat.订单状态值.isin([8,10]))][['rembursement_status']].value_counts()
df_xz_concat[(df_xz_concat.实付日期new.isnull())& (pd.isna(df_xz_concat.finish_date_new))][['订单状态']].value_counts()

rembursement_status
还款中                    104281
已逾期                     60320
未到首期还款日                 44174
续租中                     14504
检测中                        36
Name: count, dtype: int64

订单状态 
租赁中      213200
已退款        6804
待归还        6097
待收货        3643
已完成         217
待发货         184
订单取消         84
检测中          62
待商家收货        15
Name: count, dtype: int64

In [46]:
conditions = [  
    ((pd.isna(df_xz_concat['实付日期new'])) & (pd.isna(df_xz_concat['finish_date_new'])) & (df_xz_concat['订单状态值'].isin([8, 10]))),  
    ((pd.to_datetime(df_xz_concat['now_date']) <= pd.to_datetime(df_xz_concat['应付日期'])) ),  
    ((pd.notna(df_xz_concat['实付日期new'])) & (pd.to_datetime(df_xz_concat['实付日期new']) <= pd.to_datetime(df_xz_concat['应付日期']))),  
    ((pd.notna(df_xz_concat['实付日期new'])) & (pd.to_datetime(df_xz_concat['实付日期new']) > pd.to_datetime(df_xz_concat['应付日期']))) ,
    ((pd.isna(df_xz_concat['实付日期new'])) & (pd.to_datetime(df_xz_concat['now_date']) > pd.to_datetime(df_xz_concat['应付日期'])) )  

]  
  
choices = ['已取消', '未到还款日', '正常还款', '已逾期支付','已逾期']  
  
# 使用numpy.where来应用条件  
df_xz_concat['状态'] = np.select(conditions, choices, default='其他')

df_xz_concat['状态'].value_counts()

状态
正常还款     291285
未到还款日    222225
已逾期支付     42212
已逾期       41673
已取消        7021
Name: count, dtype: int64

In [47]:
# df_xz_concat['实付日期new'] = pd.to_datetime(df_xz_concat['实付日期new'], errors='coerce')
# df_xz_concat['应付日期'] = pd.to_datetime(df_xz_concat['实付日期new'], errors='coerce')

# # E:/myfile/业务相关数据/风控相关数据输出_{Today}.xlsx
# df_xz_concat.to_excel('E:/myfile/业务相关数据/报错表0902.xlsx')

## 2_15.重新定义逾期天数

In [48]:
df_xz_concat['实付日期new'].isnull().sum()
empty_rows = df_xz_concat[df_xz_concat['应付日期'].isna()]
empty_rows.order_number  

np.int64(230336)

Series([], Name: order_number, dtype: object)

In [49]:
# 异常值删除，应付时间为空
df_xz_concat.shape
df_xz_concat=df_xz_concat[~df_xz_concat['应付日期'].isnull()]
df_xz_concat.shape

(604416, 110)

(604416, 110)

In [50]:
#实付日期为空且完成时间为空  订单状态为已退款或已完成  :  逾期天数=0 
#note 20241127 逾期天数 截止目前的逾期天数 ；逾期一直未还的客户，应付日期越靠前，逾期天数越高
# A202207181530165
# A2024092919033313
# df_xz_concat[df_xz_concat.order_number == 'A202207181530165'][['order_number','逾期天数','应付日期','']]
# df_xz_concat[df_xz_concat.order_number == 'A2024092919033313'][['应付日期','实付日期','实付日期new','实付日期1','finish_date_new','当前期数','逾期天数','状态'
# ,'实付金额','当前应付租金','sesame_promo_money_pay','promo_money']]

df_xz_concat['逾期天数'] = df_xz_concat.apply(lambda x:
    0 if x['状态'] in ['未到还款日','正常还款','已取消']  
    else (pd.to_datetime(x['实付日期new']) - pd.to_datetime(x['应付日期'])).days  if pd.to_datetime(x['实付日期new']) >= pd.to_datetime(x['应付日期'])
    else  (pd.to_datetime(x['now_date']) - pd.to_datetime(x['应付日期'])).days   if   pd.to_datetime(x['now_date']) > pd.to_datetime(x['应付日期']) and pd.isna(x['实付日期new'])
    else 0 
    ,  axis=1)

# df_xz_concat['逾期天数'].value_counts()
df_xz_concat[df_xz_concat['逾期天数']>0]['rembursement_status'].value_counts()

rembursement_status
已逾期        60485
还款中         8677
已买断         7638
续租中         5792
已完成         1103
检测中          136
未到首期还款日       41
Name: count, dtype: int64

In [51]:
# liudan
# # 定义一个函数来计算逾期天数
# def calculate_overdue_days(row):
#     if row['状态'] in ['未到还款日', '正常还款', '已取消']:
#         return 0
    
#     # 转换日期为 datetime 类型，并处理 None 和 NaN
#     实付日期new = pd.to_datetime(row['实付日期new'], errors='coerce')
#     应付日期 = pd.to_datetime(row['应付日期'], errors='coerce')
#     now_date = pd.to_datetime(row['now_date'], errors='coerce')

#     if pd.isna(实付日期new) or pd.isna(应付日期):
#         return 0
    
#     if 实付日期new >= 应付日期:
#         return (实付日期new - 应付日期).days
    
#     if pd.isna(实付日期new) and now_date > 应付日期:
#         return (now_date - 应付日期).days
    
#     return 0

# # 应用函数并创建新的 '逾期天数' 列
# df_xz_concat['逾期天数'] = df_xz_concat.apply(calculate_overdue_days, axis=1)

# # 显示结果
# print(df_xz_concat['逾期天数'].value_counts())

## 2_16.逾期天数处理

In [52]:
def over_due_days(df):  
    # 转换日期为pandas的日期类型
    df['实付日期'] = pd.to_datetime(df['实付日期'])  
    df['应付日期'] = pd.to_datetime(df['应付日期'])  
      
    # 创建一个新的列来存储结果  
    df['逾期天数new'] = np.nan  # 初始化为NaN  
      
    # 应用逻辑判断  
    mask1 = df['状态'].isin(['已逾期', '未到还款日', '已取消','已提前支付'])  
    mask2 = df['实付日期'].isnull()  
    mask3 = (df['实付日期'] - df['应付日期']).dt.days <= 0  
      
    # 更新计算天数列  
    df.loc[mask1, '逾期天数new'] = df.loc[mask1, '逾期天数']  
    df.loc[~mask1 & mask2, '逾期天数new'] = 0   
    df.loc[~mask1 & ~mask2 & mask3, '逾期天数new'] = 0  
    df.loc[~mask1 & ~mask2 & ~mask3, '逾期天数new'] = (df['实付日期'] - df['应付日期']).dt.days  
      
    # 实付日期为空时设置计算天数为0：  
    # df.loc[df['实付日期'].isnull(), '逾期天数new'] = 0  
      
    return df 

    # df_ddfq = over_due_days(df_ddfq)

## 2_17.稽查

In [53]:
# df_ddfq.columns
# df_xz_concat.shape
# check_fq_df = df_xz_concat.merge(df_ddfq,on=['订单ID','当前期数'],how='left')
# check_fq_df.shape
# check_fq_df.columns
# check_fq_df[['订单ID', '当前应付租金_x', '实付金额','应付日期_x', '实付日期_x','status', '当前期数', 'is_relet','订单号_x','下单月份', '下单日期','还款状态', '实付金额new', '实付日期new', '逾期天数_x','状态_x'
            #  ,'当前应付租金_y', '实际支付金额', '应付日期_y', '实付日期_y', '状态_y', '逾期天数_y','逾期天数new']].to_excel('E:/myfile/业务相关数据/check_fq_df.xlsx')


## 2_18.确认是否到表现期

In [54]:
# import os
# file_name = os.path.basename(f_path_ck)  

# df_xz_concat['date_str'] = file_name.split('excel')[1].split('.')[0]

# df_xz_concat['date_str'].head(1)

In [55]:
# df_xz_concat['now_date'] = pd.Timestamp(dt.now().date() )
# df_xz_concat['now_date']

In [56]:
# dfzh1 = over_due_days(dfzh1)

df_xz_concat['now_date'] = pd.Timestamp(dt.now().date() )
df_xz_concat['应付日期'] = pd.to_datetime(df_xz_concat['应付日期']).dt.date
# df_xz_concat['agr_days'] = (pd.to_datetime(df_xz_concat['now_date']) - df_xz_concat['应付日期']).dt.days 

In [57]:
# from datetime import timedelta
df_xz_concat['agr_1d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=1)
df_xz_concat['agr_4d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=4)
df_xz_concat['agr_7d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=7)
df_xz_concat['agr_15d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=15)
df_xz_concat['agr_30d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=30)

df_xz_concat[['agr_30d','agr_15d','agr_7d','agr_4d','agr_1d','应付日期']].head()

,agr_30d,agr_15d,agr_7d,agr_4d,agr_1d,应付日期
0,2022-08-04,2022-07-20,2022-07-12,2022-07-09,2022-07-06,2022-07-05
1,2022-09-04,2022-08-20,2022-08-12,2022-08-09,2022-08-06,2022-08-05
2,2022-10-05,2022-09-20,2022-09-12,2022-09-09,2022-09-06,2022-09-05
3,2022-11-04,2022-10-20,2022-10-12,2022-10-09,2022-10-06,2022-10-05
4,2022-12-05,2022-11-20,2022-11-12,2022-11-09,2022-11-06,2022-11-05


In [58]:
import os

# file_name = os.path.basename(f_path_ck)  
df_xz_concat['date_str'] = df_xz_concat['now_date']
df_xz_concat['agr_1d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_1d) <= pd.to_datetime(df_xz_concat.date_str),1,0)

df_xz_concat['agr_4d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_4d) <= pd.to_datetime(df_xz_concat.date_str),1,0)
df_xz_concat['agr_7d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_7d) <= pd.to_datetime(df_xz_concat.date_str),1,0)
df_xz_concat['agr_15d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_15d) <= pd.to_datetime(df_xz_concat.date_str),1,0)
df_xz_concat['agr_30d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_30d) <= pd.to_datetime(df_xz_concat.date_str),1,0)


df_xz_concat[['agr_30d','应付日期','date_str','agr_30d_cum']].head(5)
# df_xz_concat[(df_xz_concat.agr_7d_cum == 0) & (df_xz_concat.agr_4d_cum == 1)].head()
# df_xz_concat[df_xz_concat.order_id == 254588]

,agr_30d,应付日期,date_str,agr_30d_cum
0,2022-08-04,2022-07-05,2025-06-06,1
1,2022-09-04,2022-08-05,2025-06-06,1
2,2022-10-05,2022-09-05,2025-06-06,1
3,2022-11-04,2022-10-05,2025-06-06,1
4,2022-12-05,2022-11-05,2025-06-06,1


In [59]:
# note 20241211 数据检查
# df_xz_concat[(df_xz_concat.应付日期>='2024-12-01')&(df_xz_concat.应付日期<='2024-12-31')&(df_xz_concat.order_number.isin(
# ['A20241101174043208'
# ,'A2024110313351882'
# ,'A20241103155713186'
# ,'A2024110318311457'
# ,'A20241104145450273'
# ,'A20241104164506212'
# ,'A20241105105325369'
# ,'A20241105154302243'
# ,'A2024110515050030'
# ,'A2024110610041421'
# ,'A20241106144302240'
# ,'A2024110709254879'
# ,'A20241107103123201'
# ,'A20241107195535157'
# ,'A20241108104251268'
# ,'A20241108155751456'
# ,'A2024110715142073'
# ,'A2024110817092834'
# ]))][['order_number','agr_1d_cum','agr_4d_cum','agr_7d_cum','agr_4d','应付日期','date_str','agr_1d','下单日期']]

## 2_19.观察日定义 每月月底时间

In [60]:
from pandas.tseries.offsets import MonthEnd
def mob_date(df):
    df['下单日期'] = pd.to_datetime(df['下单日期'])
    df['实付日期new'] = pd.to_datetime(df['实付日期new'])
    # note n 续租的没有统计
    for n in range(0,13):  
        # df[f'mob_date_{n}'] = df['下单日期'] + MonthEnd(n+1)
        # note 20241205 修改 月末的的跳级了，所以不要加1，mob0对齐
        # df[f'mob_date_{n}'] = np.where(df["下单日期"].dt.day.isin([28,29,30,31]), df['下单日期'] + MonthEnd(n), df['下单日期'] + MonthEnd(n+1))
        df[f'mob_date_{n}'] = np.where((df["下单日期"].dt.day == 31)|(df["下单日期"].isin(['2023-02-28','2024-02-29','2025-02-28']))|((df["下单日期"].dt.month.isin([4,6,9,11]))&(df["下单日期"].dt.day == 30)), df['下单日期'] + MonthEnd(n), df['下单日期'] + MonthEnd(n+1))
        df[f'mob_date_{n}'] = pd.to_datetime(df[f'mob_date_{n}'])
        df[f'paid_date_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],pd.NaT,df['实付日期new'])
        df[f'paid_money_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],np.nan,df['实付金额'])
        df[f'paid_date_mob{n}'] = pd.to_datetime(df[f'paid_date_mob{n}'])
    return df

df_xz_concat = mob_date(df_xz_concat)


# A2023072814030812  2023-7-28 加收二期，月末为账单日
# A2023073012170638  2023-7-30 3+1，2023.8无账单日
# A2023070115535442  2023-7-1
# A2023073114301076  2023-7-31 1元+1期，续租

# A202301011402371 2023-01-01
# A202301290815565 2023-01-29
# A2023013100341134 2023-01-31
# A2023022801142011 2023-02-28
# A2023022709283120  2023-02-27
# A202302050022319  2023-02-05
# A2023040503314720 2023-04-05
# A2023042803470516  2023-04-28
# A2023043000524068 2023-04-30
# A2023063003392242 2023-06-30
# A202306280105556 2023-06-28
# A202306050324407  2023-06-05
# A202310050100222 2023-10-05
# A2023102901224646 2023-10-29
# A2023103101255825 2023-10-31

In [61]:
df_xz_concat['date_str']  = pd.to_datetime( df_xz_concat['date_str'] )
df_xz_concat['应付日期'] = pd.to_datetime(df_xz_concat['应付日期']) 

## 2_20.更新逾期天数

In [62]:
# （1）逾期天数校正
# note 20241213
# 首期实际付款日期为空的订单，为首期取消或者待发货的，贷后表现逾期天数更新为0（各期更新为0）

# 把符合条件的订单拿出来，逾期天数全部更新为0
sql_xz = ''' 
SELECT distinct tprm.order_number,1 as label
# ,ymos.reality_refund_date,ymos.sort as '当前期数',ymos.status
from db_rent.ya_merchant_order_stages ymos 
inner join db_digua_business.t_postlease_receivables_monitoring tprm 
on ymos.order_id = tprm.order_id 
where ymos.sort=1 and ymos.reality_refund_date is null
;
'''
tmp = query(sql_xz)
tmp.shape
# tmp[:2]

df_xz_concat = df_xz_concat.merge(tmp, on = 'order_number', how = 'left')
df_xz_concat['label'] = df_xz_concat['label'].fillna(0)
df_xz_concat['label'].value_counts()

df_xz_concat[df_xz_concat['label'] == 1].drop_duplicates(subset = 'order_number').shape

df_xz_concat['逾期天数'] = np.where(df_xz_concat['label'] == 1, 0, df_xz_concat['逾期天数'])
# df_xz_concat['逾期天数_new'] = np.where(df_xz_concat['label'] == 1, 0, df_xz_concat['逾期天数'])
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']].shape
# df_xz_concat[(df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']) & (df_xz_concat.逾期天数 > 30)].shape
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']][['order_number','逾期天数','逾期天数_new']]
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']].逾期天数.max()
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']].逾期天数.min()
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']][['order_number','逾期天数','逾期天数_new']].to_excel('tmp.xlsx', index = False)

# A202207181530165
# A2024092919033313 号卡
# A2023011718580545 vintage分子是月末状态，FSTQPD是发生过逾期
# df_xz_concat[df_xz_concat.order_number == 'A2023011718580545'][['order_number','应付日期','实付日期','实付金额','逾期天数']]
# df_mob_total[df_mob_total.order_number == 'A2023011718580545'][['order_number','下单日期','paid_money_mob','overdue_days_mob','当前应付租金','MOB','overdue_days_1','overdue_days_4','overdue_days_7','overdue_days_15','overdue_days_31']]

# （2）2025.2.21
df_xz_concat['逾期天数'] = np.where(df_xz_concat.当前期数 == 1, 0, df_xz_concat.逾期天数)

(1101, 2)

label
0.0    591288
1.0     13128
Name: count, dtype: int64

(1080, 162)

# 3.逾期加工

In [63]:
# （1）历史最大逾期--续租的含续租，但未续租的（即归还逾期的）没有统计在内
df_maxoverdue = df_xz_concat.groupby(['order_number','order_id','true_name','id_card_num','mobile','下单日期','search_time','归属渠道','是否二手','是否带锁']).agg({'逾期天数':'max'}).reset_index()
df_maxoverdue.shape

df_tmp = df_xz_concat[df_xz_concat.逾期天数 > 0].groupby(['order_number']).agg({'当前期数':'min'}).reset_index()
df_tmp = df_tmp.rename(columns = {'当前期数': '首次逾期期次'})

df_maxoverdue = df_maxoverdue.merge(df_tmp, on = 'order_number', how = 'left')
df_maxoverdue[:1]
del df_tmp

(45878, 11)

,order_number,order_id,true_name,id_card_num,mobile,下单日期,search_time,归属渠道,是否二手,是否带锁,逾期天数,首次逾期期次
0,A202207050851373,8924,胡水飘,362202199805282550,13027948681,2022-07-05,2022-07,未知渠道,二手,未带锁,4,4.0


In [64]:
# （2）FPD SPD TPD
df_xz_concat['当前期数'] = df_xz_concat['当前期数'].astype(int)

df_fstq = df_xz_concat[["order_id","当前期数","逾期天数"]].pivot(index="order_id", columns="当前期数", values="逾期天数")
df_fstq = df_fstq.reset_index()
df_fstq.shape

# df_fstq = df_fstq.rename(columns = {2: 'FPD'})
df_fstq['FPD'] = df_fstq[2]
df_fstq.loc[:,"SPD"] = np.where( df_fstq['FPD'] == 0, df_fstq[3], 0)
df_fstq.loc[:,"TPD"] = np.where((df_fstq['FPD'] == 0) & (df_fstq[3] == 0), df_fstq[4], 0)

# df_fstq = df_fstq[['order_id','FPD','SPD','TPD']]
df_fstq[df_fstq.order_id.isin([508404,1678492,1692916,12449])][['order_id','FPD','SPD','TPD']]
# df_xz_concat[df_xz_concat.order_number == 'A2023073114301076'][['order_id','下单日期','当前期数','应付日期']]
# A2023073114301076 508404
# df_fstq.FPD.max()

(45879, 37)

当前期数,order_id,FPD,SPD,TPD
28,12449,0.0,11.0,0.0
3719,508404,0.0,0.0,0.0
24300,1678492,182.0,0.0,0.0
24564,1692916,8.0,0.0,0.0


In [65]:
# # （3）第2-4期逾期天数
# df_fstq = df_fstq.rename(columns = {2: '第二期逾期天数', 3: '第三期逾期天数', 4: '第四期逾期天数'})
# df_fstq = df_fstq[['order_id','FPD', 'SPD', 'TPD', '第二期逾期天数','第三期逾期天数','第四期逾期天数']]
# tmp = df_xz_concat[df_xz_concat['当前期数'] == 1][['order_id','应付日期']]
# df_fstq = df_fstq.merge(tmp, on = 'order_id', how = 'left')
# df_fstq = df_fstq.fillna(0)
# df_fstq[df_fstq.order_id.isin([508404,1678492,1692916,12449])]
# # df_xz_concat[df_xz_concat.order_number == 'A2023073114301076'][['order_id','下单日期','当前期数','应付日期']]
# # A2023073114301076 508404
# # df_fstq.FPD.max()


# # df_fstq = df_fstq.rename(columns = {2: '第二期逾期天数', 3: '第三期逾期天数', 4: '第四期逾期天数', 5:'第五期逾期天数',  6:'第六期逾期天数', 7:'第七期逾期天数'})
# # df_fstq = df_fstq[['order_id','FPD', 'SPD', 'TPD', '第二期逾期天数','第三期逾期天数','第四期逾期天数','第五期逾期天数','第六期逾期天数','第七期逾期天数']]
# # tmp = df_xz_concat[df_xz_concat['当前期数'] == 1][['order_id','应付日期']]
# # df_fstq = df_fstq.merge(tmp, on = 'order_id', how = 'left')
# # df_fstq = df_fstq.fillna(0)

# （3）
# （3-1）第2-12期逾期天数
df_fstq = df_fstq.fillna(0)
for var in range(2,13):
    if var == 2:
        df_fstq['mob12dpd_max'] = df_fstq[var]
    else:
        # print (var)
        df_fstq['mob12dpd_max'] = np.where(df_fstq.mob12dpd_max >= df_fstq[var], df_fstq.mob12dpd_max, df_fstq[var])

for var in range(2,8):
    if var == 2:
        df_fstq['mob6dpd_max'] = df_fstq[var]
    else:
        df_fstq['mob6dpd_max'] = np.where(df_fstq.mob6dpd_max >= df_fstq[var], df_fstq.mob6dpd_max, df_fstq[var])

for var in range(2,5):
    if var == 2:
        df_fstq['mob3dpd_max'] = df_fstq[var]
    else:
        df_fstq['mob3dpd_max'] = np.where(df_fstq.mob3dpd_max >= df_fstq[var], df_fstq.mob3dpd_max, df_fstq[var])

# （3-2）第2-4期逾期天数
df_fstq = df_fstq.rename(columns = {2: '第二期逾期天数', 3: '第三期逾期天数', 4: '第四期逾期天数'})
# df_fstq = df_fstq[['order_id','FPD', 'SPD', 'TPD', '第二期逾期天数','第三期逾期天数','第四期逾期天数']]
df_fstq = df_fstq[['order_id','FPD', 'SPD', 'TPD', '第二期逾期天数','第三期逾期天数','第四期逾期天数','mob3dpd_max','mob6dpd_max','mob12dpd_max']]
tmp = df_xz_concat[df_xz_concat['当前期数'] == 1][['order_id','应付日期']]
df_fstq = df_fstq.merge(tmp, on = 'order_id', how = 'left')
df_fstq[df_fstq.order_id.isin([508404,1678492,1692916,12449])]
# df_xz_concat[df_xz_concat.order_number == 'A2023073114301076'][['order_id','下单日期','当前期数','应付日期']]
# A2023073114301076 508404
# df_fstq.FPD.max()

,order_id,FPD,SPD,TPD,第二期逾期天数,第三期逾期天数,第四期逾期天数,mob3dpd_max,mob6dpd_max,mob12dpd_max,应付日期
28,12449,0.0,11.0,0.0,0.0,11.0,9.0,11.0,19.0,19.0,2022-08-05
3719,508404,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,2023-08-03
24300,1678492,182.0,0.0,0.0,182.0,151.0,120.0,182.0,182.0,182.0,2024-11-06
24564,1692916,8.0,0.0,0.0,8.0,0.0,40.0,40.0,48.0,48.0,2024-11-10


In [66]:
# # （4）表关联 
# df_analy_data = df_maxoverdue.merge(df_fstq, on = 'order_id', how = 'left')
# df_analy_data = df_analy_data.rename(columns = {'逾期天数':'历史最大逾期天数','应付日期': '首期应还日期'})
# df_analy_data = df_analy_data.merge(df_zhys[['order_number','overdue_type']], on = 'order_number', how = 'left')

# df_analy_data.shape
# df_analy_data = df_analy_data.sort_values(by = ['order_number','overdue_type'])
# df_analy_data = df_analy_data.drop_duplicates(subset = 'order_number')
# # 以下6个订单号重复,overdue_type 不一样，保留未逾期的，因为 历史最大逾期天数 为0，P站显示为0
# # A2024020821520518    2
# # A2024010116302721    2
# # A2024012715442754    2
# # A202401030606401     2
# # A2024010216293470    2
# # A202402110902322     2
# df_analy_data.shape

# df_analy_data[df_analy_data.order_id.isin([508404,1678492,1692916,12449])]

# （4）表关联 
df_analy_data = df_maxoverdue.merge(df_fstq, on = 'order_id', how = 'left')
df_analy_data = df_analy_data.rename(columns = {'逾期天数':'历史最大逾期天数','应付日期': '首期应还日期'})
df_analy_data = df_analy_data.merge(df_zhys[['order_number','overdue_type']], on = 'order_number', how = 'left')

df_analy_data.shape
df_analy_data[df_analy_data.order_id.isin([508404,1678492,1692916,12449])]

(45891, 23)

,order_number,order_id,true_name,id_card_num,mobile,下单日期,search_time,归属渠道,是否二手,是否带锁,历史最大逾期天数,首次逾期期次,FPD,SPD,TPD,第二期逾期天数,第三期逾期天数,第四期逾期天数,mob3dpd_max,mob6dpd_max,mob12dpd_max,首期应还日期,overdue_type
28,A202208052103231,12449,赵旭,230184199803042615,13591343043,2022-08-05,2022-08,顶部搜索框的搜索结果页,全新,未带锁,19,3.0,0.0,11.0,0.0,0.0,11.0,9.0,11.0,19.0,19.0,2022-08-05,M6
3719,A2023073114301076,508404,邵光峰,371482199409260337,15136842042,2023-07-31,2023-07,芝麻租物,全新,未带锁,6,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,2023-08-03,
24310,A20241103155713186,1678492,张驰,152201200201300017,16223593777,2024-11-03,2024-11,单人聊天会话中的小程序消息卡片（分享）,全新,未带锁,182,2.0,182.0,0.0,0.0,182.0,151.0,120.0,182.0,182.0,182.0,2024-11-06,M6
24574,A20241107103123201,1692916,李星华,360427199610221519,15870818200,2024-11-07,2024-11,单人聊天会话中的小程序消息卡片（分享）,全新,未带锁,48,2.0,8.0,0.0,0.0,8.0,0.0,40.0,40.0,48.0,48.0,2024-11-10,M0


In [67]:
# （5）打标签
# df_analy_data['f2pd7'] = np.where(df_analy_data.第二期逾期天数 >= df_analy_data.第三期逾期天数, df_analy_data.第二期逾期天数, df_analy_data.第三期逾期天数)
# df_analy_data['f3pd7'] = np.where(df_analy_data.f2pd7 >= df_analy_data.第四期逾期天数, df_analy_data.f2pd7, df_analy_data.第四期逾期天数)

df_analy_data['mob2dpd'] = np.where(df_analy_data.第二期逾期天数 >= df_analy_data.第三期逾期天数, df_analy_data.第二期逾期天数, df_analy_data.第三期逾期天数)
# df_analy_data['mob3dpd'] = np.where(df_analy_data.mob2dpd >= df_analy_data.第四期逾期天数, df_analy_data.mob2dpd, df_analy_data.第四期逾期天数)

df_analy_data['y_fraud_fpd7'] = df_analy_data.apply(lambda x: 1 if x.FPD >= 8 
                                               else 0 if x.FPD >= 0 and x.FPD <= 7
                                               else 999, axis = 1)
df_analy_data['y_fraud_fpd15'] = df_analy_data.apply(lambda x: 1 if x.FPD >= 16
                                               else 0 if x.FPD >= 0 and x.FPD <= 15
                                               else 999, axis = 1)
df_analy_data['y_fraud_fpd30'] = df_analy_data.apply(lambda x: 1 if x.FPD >= 31 
                                               else 0 if x.FPD >= 0 and x.FPD <= 30
                                               else 999, axis = 1)

df_analy_data['y_fraud_mob2dpd7'] = df_analy_data.apply(lambda x: 1 if x.mob2dpd >= 8 
                                               else 0 if x.mob2dpd >= 0 and x.mob2dpd <= 7
                                               else 999, axis = 1)

df_analy_data['y_fraud_mob3dpd7'] = df_analy_data.apply(lambda x: 1 if x.mob3dpd_max >= 8 
                                               else 0 if x.mob3dpd_max >= 0 and x.mob3dpd_max <= 7
                                               else 999, axis = 1)
df_analy_data['y_credit_mob3dpd30'] = df_analy_data.apply(lambda x: 1 if x.mob3dpd_max >= 31 
                                               else 0 if x.mob3dpd_max >= 0 and x.mob3dpd_max <= 30
                                               else 999, axis = 1)

df_analy_data['y_credit_mob6dpd30'] = df_analy_data.apply(lambda x: 1 if x.mob6dpd_max >= 31 
                                               else 0 if x.mob6dpd_max >= 0 and x.mob6dpd_max <= 30
                                               else 999, axis = 1)

df_analy_data['y_credit_mob12dpd30'] = df_analy_data.apply(lambda x: 1 if x.mob12dpd_max >= 31 
                                               else 0 if x.mob12dpd_max >= 0 and x.mob12dpd_max <= 30
                                               else 999, axis = 1)

df_analy_data['y_credit'] = df_analy_data.apply(lambda x: 1 if x.历史最大逾期天数 >= 31 
                                               else 0 if x.历史最大逾期天数 >= 0 and x.历史最大逾期天数 <= 30
                                               else 999, axis = 1)
df_analy_data.shape

# # （5）打标签
# df_analy_data['f2pd7'] = np.where(df_analy_data.第二期逾期天数 >= df_analy_data.第三期逾期天数, df_analy_data.第二期逾期天数, df_analy_data.第三期逾期天数)
# df_analy_data['f3pd7'] = np.where(df_analy_data.f2pd7 >= df_analy_data.第四期逾期天数, df_analy_data.f2pd7, df_analy_data.第四期逾期天数)
# df_analy_data['y_fraud_fpd7'] = df_analy_data.apply(lambda x: 1 if x.FPD >= 8 
#                                                else 0 if x.FPD >= 0 and x.FPD <= 7
#                                                else 999, axis = 1)
# df_analy_data['y_fraud_f2pd7'] = df_analy_data.apply(lambda x: 1 if x.f2pd7 >= 8 
#                                                else 0 if x.f2pd7 >= 0 and x.f2pd7 <= 7
#                                                else 999, axis = 1)
# df_analy_data['y_fraud_f3pd7'] = df_analy_data.apply(lambda x: 1 if x.f3pd7 >= 8 
#                                                else 0 if x.f3pd7 >= 0 and x.f3pd7 <= 7
#                                                else 999, axis = 1)
# df_analy_data['y_credit_mob3pd30'] = df_analy_data.apply(lambda x: 1 if x.f3pd7 >= 31 
#                                                else 0 if x.f3pd7 >= 0 and x.f3pd7 <= 30
#                                                else 999, axis = 1)
# df_analy_data['y_credit'] = df_analy_data.apply(lambda x: 1 if x.历史最大逾期天数 >= 31 
#                                                else 0 if x.历史最大逾期天数 >= 0 and x.历史最大逾期天数 <= 30
#                                                else 999, axis = 1)
# df_analy_data.shape
# var_1 = ['order_number','下单日期','历史最大逾期天数','FPD','SPD','TPD','第二期逾期天数','第三期逾期天数','第四期逾期天数','overdue_type','f2pd7','f3pd7','y_fraud_fpd7','y_fraud_f2pd7','y_fraud_f3pd7','y_credit_mob3pd30','y_credit']
# df_analy_data[df_analy_data.order_id.isin([508404,1678492,1692916,12449])][var_1]



# df_analy_data['f2pd7'] = np.where(df_analy_data.第二期逾期天数 >= df_analy_data.第三期逾期天数, df_analy_data.第二期逾期天数, df_analy_data.第三期逾期天数)
# df_analy_data['f3pd7'] = np.where(df_analy_data.f2pd7 >= df_analy_data.第四期逾期天数, df_analy_data.f2pd7, df_analy_data.第四期逾期天数)
# # 新增
# df_analy_data['mob4dpd'] = np.where(df_analy_data.f3pd7 >= df_analy_data.第五期逾期天数, df_analy_data.f3pd7, df_analy_data.第五期逾期天数)
# df_analy_data['mob5dpd'] = np.where(df_analy_data.mob4dpd >= df_analy_data.第六期逾期天数, df_analy_data.mob4dpd, df_analy_data.第六期逾期天数)
# df_analy_data['mob6dpd'] = np.where(df_analy_data.mob5dpd >= df_analy_data.第七期逾期天数, df_analy_data.mob5dpd, df_analy_data.第七期逾期天数)

# df_analy_data['y_fraud_fpd7'] = df_analy_data.apply(lambda x: 1 if x.FPD >= 8 
#                                                else 0 if x.FPD >= 0 and x.FPD <= 7
#                                                else 999, axis = 1)
# df_analy_data['y_fraud_f2pd7'] = df_analy_data.apply(lambda x: 1 if x.f2pd7 >= 8 
#                                                else 0 if x.f2pd7 >= 0 and x.f2pd7 <= 7
#                                                else 999, axis = 1)
# df_analy_data['y_fraud_f3pd7'] = df_analy_data.apply(lambda x: 1 if x.f3pd7 >= 8 
#                                                else 0 if x.f3pd7 >= 0 and x.f3pd7 <= 7
#                                                else 999, axis = 1)
# df_analy_data['y_credit_mob3dpd30'] = df_analy_data.apply(lambda x: 1 if x.f3pd7 >= 31 
#                                                else 0 if x.f3pd7 >= 0 and x.f3pd7 <= 30
#                                                else 999, axis = 1)
# df_analy_data['y_credit'] = df_analy_data.apply(lambda x: 1 if x.历史最大逾期天数 >= 31 
#                                                else 0 if x.历史最大逾期天数 >= 0 and x.历史最大逾期天数 <= 30
#                                                else 999, axis = 1)
# # 新增
# df_analy_data['y_credit_mob3dpd15'] = df_analy_data.apply(lambda x: 1 if x.f3pd7 >= 15
#                                                else 0 if x.f3pd7 >= 0 and x.f3pd7 <= 14
#                                                else 999, axis = 1)
# df_analy_data['y_credit_mob6dpd30'] = df_analy_data.apply(lambda x: 1 if x.mob6dpd >= 31
#                                                else 0 if x.mob6dpd >= 0 and x.mob6dpd <= 30
#                                                else 999, axis = 1)


# df_analy_data.shape

(45891, 33)

In [68]:
# （6）数据检查
df_analy_data[df_analy_data.历史最大逾期天数 >0].shape
df_analy_data[df_analy_data.历史最大逾期天数 >30].shape
df_analy_data[df_analy_data.历史最大逾期天数 >90].shape
df_analy_data[df_analy_data.历史最大逾期天数 >180].shape
df_analy_data[(df_analy_data.历史最大逾期天数 >180)&(df_analy_data.下单日期 <= '2023-12-31')].shape
df_analy_data[(df_analy_data.历史最大逾期天数 >180)&(df_analy_data.下单日期 >= '2024-01-01')&(df_analy_data.下单日期 <= '2024-12-31')].shape
df_analy_data[(df_analy_data.历史最大逾期天数 >180)&(df_analy_data.下单日期 >= '2025-01-01')].shape

(17862, 33)

(8863, 33)

(6568, 33)

(4567, 33)

(2851, 33)

(1716, 33)

(0, 33)

# 4.规则命中数据整理

## 4_1.取规则命中数据

In [69]:
# （1）大表
sql1 = ''' -- 订单&风控信息  近10日数据   
SELECT date(om.create_time) as create_date,om.create_time,om.id as order_id ,om.order_number,om.all_money 
,om.status,om.order_method,om.order_type
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,case when locate('租物',pa.name)>0 or locate('租物',cc.name)>0 or locate('芝麻',pa.name)>0 or locate('芝麻',cc.name)>0  then '芝麻租物' when locate('抖音',pa.name)>0 then '抖音渠道' when locate('搜索',cc.name)>0 then '搜索渠道' else '其他渠道' end as channel_type 
,tod.sku_attributes,tod.product_name,tod.new_actual_money
,case when  locate('租完即送',tod.sku_attributes)>0 then '租完即送' else '租物归还' end as back_type
,om.user_mobile,tmu.true_name,tmu.id_card_num
,top.total_describes,tor.decision_result,om.cancel_reason
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips  

#2025.3.10
# 判断拒量是机审拒绝还是被出库前强拒，无拒绝理由： qvt_risk = '1'为机审强拒，qvt_risk='0' 且qvt_result='1'的为出库前强拒

,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.queue_verify_type') end,'"','') as qvt_risk
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.queue_verify_type') end,'"','') as qvt_result 

# is_vip = '1' & status_result = '0' 为免人审的单

,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.is_vip') end,'"','') as is_vip
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.status') end,'"','') as status_result

,cc.name as channel_name         -- 来源渠道
,cc.channel_type_id              -- 渠道id
,pa.name as activity_name        -- 活动名称
,om.merchant_id,om.merchant_name
,topay.total_freeze_fund_amount , tprm.purchase_amount, tprm.all_rental, tprm.order_number order_number2
,om.buy_service_product,tso.status as service_status 
, tprm.contract_price, tor.update_time, tomt.reason
,tor.jd_user_level_risk  -- 京享租评分等级

from db_digua_business.t_order  om 
LEFT JOIN db_digua_business.t_postlease_receivables_monitoring tprm on om.id=tprm.order_id
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
 -- 备注信息合并 
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
on om.id = top.order_id 
-- 服务信息
left join  db_digua_business.t_service_order tso  on om.id = tso.order_id 
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
-- 用户信息 
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
-- 商品信息
left join db_digua_business.t_order_details tod on om.id = tod.order_id
-- 免押信息  
left join (SELECT t.*,row_number() over(partition by t.order_id order by t.pay_date desc) as rn 
from db_digua_business.t_order_pay t 
where t.pay_type = 'ZFBYSQ' and t.item_type=1 and t.`status` in (2,5) and t.trade_no is not null )  topay 
on topay.order_id=om.id   and  topay.rn = 1 
-- 商家订单转移表
left join db_digua_business.t_order_merchant_transfer tomt on tomt.order_id=om.id
where om.user_mobile is not null 
and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静'
,'陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')
and date_format(om.create_time, '%Y-%m-%d')>='2023-10-01'
-- and om.create_time >= date_add(current_date, interval -3 month)
;
'''
df = query(sql1)
df.shape

(2508482, 43)

In [70]:
# （2）机审
# db_credit.risk：机审的订单，机审命中的
sql2 = '''
SELECT 
id_card, mobile, time, trace_id
, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status_risk  
, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.rejected') end,'"','') as rejected  

# 2025.3.18 识别联合运营订单 union_rent_tag = '1' 为联合运营的，前置拦截的表从 2025.3.25 开始也加上 union_rent_tag 的标签，存在"union_rent_tag"字段，且union_rent_tag==1，为联营订单
, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.union_rent_tag') end,'"','') as union_rent_tag
, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.union_rent_rejected') end,'"','') as union_rent_rejected
FROM `db_credit`.`risk`
'''
df_risk = query(sql2)
df_risk.shape

(364098, 8)

In [71]:
# （3）出库前-风控流
# 出库前风控流强拒
sql3 = '''
SELECT risk_trace_id trace_id, id_card, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status   FROM `db_credit`.risk_examine
'''
df_re = query(sql3)
# df_re = df_risk_examine.copy()
df_re.loc[:, 'time_re'] = pd.to_datetime(df_re.time.dt.date,errors="coerce")
df_re.rename(columns={'id_card': 'id_card_re', 'status': 'status_re'}, inplace=True)
df_re.shape
df_re[:1]

(106194, 5)

,trace_id,id_card_re,time,status_re,time_re
0,53ab4a0a13e7f82b5fc594a852900ff7,43072619940323311X,2024-01-25 17:11:18,0,2024-01-25


In [72]:
# （4）出库前-数控
# 蚂蚁数控强拒
sql_ra = ''' -- 996强拒表
select order_id, time, replace(case when JSON_VALID(result) THEN JSON_EXTRACT(result, '$.status') end,'"','') as status_ra  from db_credit.risk_alipay_interactive_prod_result
'''
df_ra = query(sql_ra)
df_ra.loc[:, 'time_ra'] = pd.to_datetime(df_ra.time.dt.date,errors="coerce")
df_ra = df_ra.sort_values(by='time', ascending=False).groupby('order_id').head(1)
df_ra.shape
df_ra[:1]

(106298, 4)

,order_id,time,status_ra,time_ra
106297,2833026,2025-06-06 19:10:55,0,2025-06-06


In [73]:
# （5）表合并
# df_risk 的订单 包含 df_re、df_ra 的，出现在 df_re、df_ra 的话，以 df_re、df_ra 的结果为准
# 前置拦截的不会出现在 df_risk
df = df.merge(df_risk[['trace_id', 'status_risk', 'union_rent_tag', 'union_rent_rejected']], on='trace_id', how='left').merge(df_re[['trace_id', 'status_re']], on='trace_id', how='left').merge(df_ra[['order_id', 'time_ra', 'status_ra']], on='order_id', how='left')
df.shape
df[:2]

(2508482, 49)

,create_date,create_time,order_id,order_number,all_money,status,order_method,order_type,status2,channel_type,sku_attributes,product_name,new_actual_money,back_type,user_mobile,true_name,id_card_num,total_describes,decision_result,cancel_reason,trace_id,rejected,result,tips,qvt_risk,qvt_result,is_vip,status_result,channel_name,channel_type_id,activity_name,merchant_id,merchant_name,total_freeze_fund_amount,purchase_amount,all_rental,order_number2,buy_service_product,service_status,contract_price,update_time,reason,jd_user_level_risk,status_risk,union_rent_tag,union_rent_rejected,status_re,time_ra,status_ra
0,2023-10-01,2023-10-01 00:01:42,656793,A202310010001411,7000.7,13,0,ZFB_ORDER,订单取消,芝麻租物,"[{""type"":""text"",""value"":""租完归还/可买断"",""key"":""租赁方案...",iPhone14ProMax 国行正品 顺丰包邮 现货速发 高通过率,12866.7,租物归还,18695096660,黄菲,420902199604180446,None,"{""code"":0,""msg"":""ok"",""rejected"":[],""result"":""数...",过期系统自动取消,6fc563d936c2a5600c53828ea138f05c,[],数据校验通过,[],None,None,None,None,芝麻租物,2.0,芝麻租物-苹果手机,14,迪瓜自营店,NaN,NaN,NaN,None,1,NaN,NaN,NaT,None,None,NaN,NaN,NaN,NaN,NaT,NaN
1,2023-10-01,2023-10-01 00:01:42,656793,A202310010001411,7000.7,13,0,ZFB_ORDER,订单取消,芝麻租物,"[{""type"":""text"",""value"":""租完归还/可买断"",""key"":""租赁方案...",iPhone14ProMax 国行正品 顺丰包邮 现货速发 高通过率,12866.7,租物归还,18695096660,黄菲,420902199604180446,None,"{""code"":0,""msg"":""ok"",""rejected"":[],""result"":""数...",过期系统自动取消,6fc563d936c2a5600c53828ea138f05c,[],数据校验通过,[],None,None,None,None,芝麻租物,2.0,芝麻租物-苹果手机,14,迪瓜自营店,NaN,NaN,NaN,None,1,NaN,NaN,NaT,None,None,NaN,NaN,NaN,NaN,NaT,NaN


In [74]:
# （6）
# 前置拦截
# df_reject_before = df[((df['status2']=='订单取消')&(df['拒绝理由'].notna()))|((df['status2']=='订单取消')&(df['total_describes'].notna()))]
# df_reject_before['拒绝理由'] = np.where(df_reject_before['拒绝理由'].isna(),   
#                                         df_reject_before['total_describes'].str.split('拒绝理由：').str[-1].str.split('"').str[0],df_reject_before['拒绝理由'])
# df_reject_before = df_reject_before[['create_time', 'order_number', '拒绝理由']]

## 4_2.数据处理

In [75]:
# （1）信息处理
df['拒绝理由'] = df["rejected"].str.replace("[","").str.replace("]","").str.replace('"','')
df['拒绝理由'] = np.where(df['拒绝理由']=='', df['total_describes'].str.split('原因：').str[-1].str.split('"').str[0], df['rejected'])

In [76]:
# （2）判断进件
df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")

In [77]:
# （3）日期处理
df['下单日期'] = df['create_time'].dt.date
df["下单日期"] = pd.to_datetime(df["下单日期"],errors="coerce")

In [78]:
# （4）订单预排序去重 剔除内部订单
dict_status_code = {
    "订单取消": 1,
    "待支付": 2,
    "已退款": 3, 
    "待审核": 4,
    "待发货": 5,
    "待收货": 6,
    "租赁中": 7,
    "已完成": 8
}
df["状态编码"]=df["status2"].map(dict_status_code)


df.sort_values(by=["下单日期","状态编码"],inplace=True)


#删除重复单号
df.drop_duplicates(subset=["order_id"],inplace=True)
#删除身份证空值行
df.dropna(subset=["id_card_num"],axis=0,inplace=True)
#去除刷单订单
df.drop(df[df['total_describes'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划-无优惠"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单曙光计划"].index,inplace=True)
df.drop(df[df['activity_name']=="线下门店3个月试行"].index,inplace=True)
#删除订单状态空值行
df.dropna(subset=["status2"],axis=0,inplace=True)


#删除重复订单
df.drop_duplicates(subset=["order_id"],inplace=True)
# note 20241209 删除此条件
# df.drop_duplicates(subset=["true_name","user_mobile","id_card_num","下单日期"],keep="last",inplace=True)


df.drop(df[df['true_name'].isin(["刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文"])].index,inplace=True)

df.shape

(2124038, 53)

In [79]:
# # （5）剔除商家数据--只保留自营租机业务数据
# df.drop(df[df['merchant_name']=="深圳优优大数据科技有限公司"].index,inplace=True)
# df.drop(df[df['merchant_name']=="优优2店"].index,inplace=True)
# df.drop(df[df['merchant_name']=="小豚租（代收）"].index,inplace=True)
# df.drop(df[df['merchant_name']=="苏州蚁诺宝"].index,inplace=True)
# df.drop(df[df['merchant_name']=="租着用电脑数码"].index,inplace=True)
# df.drop(df[df['merchant_name']=="北京海鸟窝科技有限公司"].index,inplace=True)

# df.drop(df[df['merchant_name']=="汇客好租"].index,inplace=True)
# df.drop(df[df['merchant_name']=="澄心优租"].index,inplace=True)
# df.drop(df[df['merchant_name']=="CPS渠道合作"].index,inplace=True)

# df.drop(df[df['sku_attributes'].str.contains(pat='探路者',regex=False)==True].index,inplace=True)
# df.drop(df[df['merchant_name']=="趣智数码"].index,inplace=True)
# df.drop(df[df['merchant_name']=="格木木二奢名品"].index,inplace=True)
# df.drop(df[df['merchant_name']=="广州康基贸易有限公司"].index,inplace=True)

# df.drop(df[df['merchant_name']=="线下小店"].index,inplace=True)

# df.drop(df[df['merchant_name'].str.contains(pat='探路者',regex=False)==True].index,inplace=True) 
# df.drop(df[df['merchant_name'] == "乙辉数码"].index, inplace=True)

# df.shape
df = clean.drop_merchant(df)
df = clean.drop_rejected_merchant(df)

In [80]:
# （6）定义审核状态
# 处理原因
df = df[df['sku_attributes'].notnull()]
df["取消原因"]=df["total_describes"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip()
df["电审拒绝原因"]=df["total_describes"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip()
df["无法联系原因"]=df["total_describes"].str.split("用户无法联系：").str[1].str.split("$").str[0].str.strip()
# 判断 前置拦截   机审强拒   出库前风控强拒
df['是否前置拦截'] = np.where((df.result.str.contains('id_card不得为空') )|(df.result.str.contains('mobile校验不通过')) |(df.result.str.contains('name校验不通过'))|(df.result.str.contains('年龄超过49岁或低于18岁'))|(df.result=='风控未通过')|(df.result=='黑名单用户'),1,0)


# df['是否机审强拒'] = np.where((df.result.str.contains('风控拒绝') & (~df.result.str.contains('命中出库前风控流强拒').fillna(False)) ),1,0)
# df['是否出库前风控强拒'] = np.where((df.result.str.contains('命中出库前风控流强拒').fillna(False)) |((df.total_describes.str.contains('蚂蚁数控风险等级').fillna(False)) & (~df.result.str.contains('黑名单用户').fillna(False))) ,1,0)

# 2025.3.10
df['是否机审强拒'] = np.where(df.status_risk=='1', 1, 0)
df['是否出库前风控强拒'] = np.where((df.status_re=='1')|(df.status_ra=='1'), 1, 0)

df.loc[:,"审核状态"]=df.apply(lambda x: clean.reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截'],x['是否机审强拒'],x['是否出库前风控强拒']),axis=1)

# df["机审强拒"] = np.where(df["审核状态"]=='机审强拒',1,0)
# df["出库前风控强拒"] = np.where(df["审核状态"]=='出库前风控强拒',1,0)

df.shape

(1994200, 60)

In [81]:
# （7）加工渠道、是否免审、是否拒量
# 2025.3.10
df =  df.merge(qudao_name_df,left_on = 'order_number',right_on = '订单号',how = 'left')
df["channel_name"] = np.where(df.渠道名称.notnull(),df.渠道名称,df["channel_name"])
df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")
df.loc[:,"归属渠道"]=df.apply(lambda x: clean.qudao_type(x["来源渠道"],x["activity_name"],x["order_method"], x['channel_type_id'], x['order_type']),axis=1)

df['是否免人审'] = np.where((df.is_vip=='1')&(df.status_result=='0'), 1 , 0)

# df['是否拒量'] = np.where(df.qvt_risk == '1', 1 , 0)
# df['是否拒量'] = np.where((df.qvt_risk == '0') & (df.qvt_result == '1'), 1 , df.是否拒量)
df['是否拒量'] = np.where(df.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True, 1, 0)

In [82]:
# （8）添加拒量被拒理由
# 判断拒量是机审拒绝还是被出库前强拒，无拒绝理由： qvt_risk = '1'为机审强拒，qvt_risk='0' 且qvt_result='1'的为出库前强拒
# 有些为真实拒绝原因
# 回捞命中订单恢复记录豁免的强拒项，2025.3.6 上线
df['拒绝理由'] = np.where((df.拒绝理由 == '[') & (df.qvt_risk == '1'), '命中机审强拒进回捞池', df.拒绝理由)
df['拒绝理由'] = np.where((df.拒绝理由 == '[') & (df.qvt_risk == '0') & (df.qvt_result == '1'), '命中出库前强拒进回捞池', df.拒绝理由)
df.shape
df[:1]

# 以下3个订单检查到是拒量回捞，但未识别出来
# A20241212093821140
# A20241214161430122
# A20241220184558214
# df[df.order_number == 'A20241220184558214'][['拒绝理由','qvt_risk','qvt_result']]

# df[(df.qvt_risk == '1')&(df.拒绝理由 != '命中机审强拒进回捞池')].下单日期.min()
# df[(df.qvt_risk == '0') & (df.qvt_result == '1')&(df.拒绝理由 != '命中出库前强拒进回捞池')].下单日期.min()
# df[(df.qvt_risk == '1')][['order_number','拒绝理由','status_risk']].拒绝理由.value_counts()
# df[(df.qvt_risk == '0') & (df.qvt_result == '1')][['order_number','拒绝理由','status_risk']].拒绝理由.value_counts()

(1994200, 66)

,create_date,create_time,order_id,order_number,all_money,status,order_method,order_type,status2,channel_type,sku_attributes,product_name,new_actual_money,back_type,user_mobile,true_name,id_card_num,total_describes,decision_result,cancel_reason,trace_id,rejected,result,tips,qvt_risk,qvt_result,is_vip,status_result,channel_name,channel_type_id,activity_name,merchant_id,merchant_name,total_freeze_fund_amount,purchase_amount,all_rental,order_number2,buy_service_product,service_status,contract_price,update_time,reason,jd_user_level_risk,status_risk,union_rent_tag,union_rent_rejected,status_re,time_ra,status_ra,拒绝理由,进件,下单日期,状态编码,取消原因,电审拒绝原因,无法联系原因,是否前置拦截,是否机审强拒,是否出库前风控强拒,审核状态,订单号,渠道名称,来源渠道,归属渠道,是否免人审,是否拒量
0,2023-10-01,2023-10-01 00:01:42,656793,A202310010001411,7000.7,13,0,ZFB_ORDER,订单取消,芝麻租物,"[{""type"":""text"",""value"":""租完归还/可买断"",""key"":""租赁方案...",iPhone14ProMax 国行正品 顺丰包邮 现货速发 高通过率,12866.7,租物归还,18695096660,黄菲,420902199604180446,None,"{""code"":0,""msg"":""ok"",""rejected"":[],""result"":""数...",过期系统自动取消,6fc563d936c2a5600c53828ea138f05c,[],数据校验通过,[],None,None,None,None,芝麻租物,2.0,芝麻租物-苹果手机,14,迪瓜自营店,NaN,NaN,NaN,None,1,NaN,NaN,NaT,None,None,NaN,NaN,NaN,NaN,NaT,NaN,None,未进件,2023-10-01,1.0,None,None,None,0,0,0,进件前取消,NaN,NaN,芝麻租物,芝麻租物,0,0


In [83]:
# （9）添加商家拒量被拒理由
# 添加小蚂蚁租机被拒理由
# 小蚂蚁租机 的用 df_risk 的 拒绝理由填充
df2 = df.copy()
# df2.drop(df2[df2['merchant_name']=="小蚂蚁租机"].index,inplace=True)
df2.drop(df2[df2['merchant_name'].isin(["小蚂蚁租机",'人人享租','崇胜数码','兴鑫兴通讯'])].index,inplace=True)

# 匹配小蚂蚁数据
# df_xmy = df[df['merchant_name'] == '小蚂蚁租机']
df_xmy = df[df['merchant_name'].isin(["小蚂蚁租机",'人人享租','崇胜数码','兴鑫兴通讯'])]

# 机审强拒数据重命名
# df_risk.rename(columns={'time': 'time_risk', 'status': 'status_risk', 'rejected': 'rejected_risk'}, inplace=True)
df_risk.rename(columns={'time': 'time_risk', 'rejected': 'rejected_risk'}, inplace=True)
df_risk['拒绝理由_risk'] = df_risk["rejected_risk"].str.replace("[","").str.replace("]","").str.replace('"','')
# 对小蚂蚁数据和机审强拒数据进行拼接
df_risk_all = pd.merge(df_xmy, df_risk, left_on='id_card_num', right_on='id_card', how='inner')
# 计算出库前强拒的订单数
df_risk_all2 = df_risk_all[(df_risk_all['time_risk'] < df_risk_all['update_time'])]
# 进行排序并取到最近的一个订单
df_risk_all2 = df_risk_all2.sort_values(['order_id', 'time_risk'], ascending=[True, False]).groupby('order_id').head(1)
# # df_risk_all2 = df_risk_all2[df_risk_all2['status_risk'] == '1']
df_risk_all2 = df_risk_all2[df_risk_all2['status_risk_y'] == '1']
df_risk_all2.shape
# 拼接小蚂蚁数据获取全新的拒绝理由
# status_risk 为空 但 拒绝理由不为空
df_risk_new = df.merge(df_risk_all2[['order_number', '拒绝理由_risk']], on='order_number', how='left')
df_risk_new['拒绝理由'] = np.where((df_risk_new['拒绝理由'].isna()) & df_risk_new['拒绝理由_risk'].notna(), df_risk_new['拒绝理由_risk'], df_risk_new['拒绝理由'])
df_risk_new['search_time'] = pd.to_datetime(df_risk_new["下单日期"]).dt.strftime('%Y-%m')
df_risk_new = df_risk_new.drop(columns = ['拒绝理由_risk'])
df_risk_new.shape
df_risk_new[:1]

# 860459
# df[df.拒绝理由.notnull()].shape
# 868083
# df_risk_new[df_risk_new.拒绝理由.notnull()].shape
# df_risk_new[df_risk_new.order_number == 'A2024110614100051'][['拒绝理由','status_risk']]

(0, 75)

(1994200, 67)

,create_date,create_time,order_id,order_number,all_money,status,order_method,order_type,status2,channel_type,sku_attributes,product_name,new_actual_money,back_type,user_mobile,true_name,id_card_num,total_describes,decision_result,cancel_reason,trace_id,rejected,result,tips,qvt_risk,qvt_result,is_vip,status_result,channel_name,channel_type_id,activity_name,merchant_id,merchant_name,total_freeze_fund_amount,purchase_amount,all_rental,order_number2,buy_service_product,service_status,contract_price,update_time,reason,jd_user_level_risk,status_risk,union_rent_tag,union_rent_rejected,status_re,time_ra,status_ra,拒绝理由,进件,下单日期,状态编码,取消原因,电审拒绝原因,无法联系原因,是否前置拦截,是否机审强拒,是否出库前风控强拒,审核状态,订单号,渠道名称,来源渠道,归属渠道,是否免人审,是否拒量,search_time
0,2023-10-01,2023-10-01 00:01:42,656793,A202310010001411,7000.7,13,0,ZFB_ORDER,订单取消,芝麻租物,"[{""type"":""text"",""value"":""租完归还/可买断"",""key"":""租赁方案...",iPhone14ProMax 国行正品 顺丰包邮 现货速发 高通过率,12866.7,租物归还,18695096660,黄菲,420902199604180446,None,"{""code"":0,""msg"":""ok"",""rejected"":[],""result"":""数...",过期系统自动取消,6fc563d936c2a5600c53828ea138f05c,[],数据校验通过,[],None,None,None,None,芝麻租物,2.0,芝麻租物-苹果手机,14,迪瓜自营店,NaN,NaN,NaN,None,1,NaN,NaN,NaT,None,None,NaN,NaN,NaN,NaN,NaT,NaN,None,未进件,2023-10-01,1.0,None,None,None,0,0,0,进件前取消,NaN,NaN,芝麻租物,芝麻租物,0,0,2023-10


In [84]:
# （10）
#进件的数据
# 机审强拒&出库前强拒规则明细，1代表命中
df_jj = df_risk_new[df_risk_new['进件']=='进件']
# df_reject = df_jj[(df_jj['机审强拒']==1)|(df_jj['出库前风控强拒']==1)][['create_time', 'order_id', 'order_number', '拒绝理由', 'total_describes', 'result']]
df_reject = df_jj[['create_time', 'order_id', 'order_number', '拒绝理由', 'total_describes', 'result','trace_id','审核状态']]
df_reject_risk = risk.model_reject(df_reject, 'order_number')
# df_reject_risk_merge = df_reject[['order_number','order_id','trace_id','拒绝理由','审核状态']].merge(df_reject_risk, on='order_number', how='left')
# 2025.3.25
var_need = ['order_number','order_id','trace_id','下单日期','search_time','是否免人审','是否拒量','归属渠道','union_rent_tag','union_rent_rejected','拒绝理由','审核状态']
df_reject_risk_merge = df_risk_new[df_risk_new['进件']=='进件'][var_need].merge(df_reject_risk, on='order_number', how='left')
df_reject_risk_merge.shape
df_reject_risk_merge.tail(1)

# 拒绝理由 == '[' 的可能 可能是拒件 也可能是出库， 看 审核状态
# df_reject_risk_merge[df_reject_risk_merge.拒绝理由 == '['][:2]
# df_reject_risk_merge[df_reject_risk_merge.拒绝理由 == '['].tail(2)
# 295034

(291605, 38)

,order_number,order_id,trace_id,下单日期,search_time,是否免人审,是否拒量,归属渠道,union_rent_tag,union_rent_rejected,拒绝理由,审核状态,评分等级D且综合风险等级为3,综合风险等级为3,命中法院失信/限高被执行人,命中刑事案件,命中借贷纠纷,命中融安分低于680,命中高风险名单,命中风险勘测,蚁盾分>=80,命中30天多头>13并且90天多头>39,履约历史等级1且为搜索渠道,云商分低于496,评分等级低于D,评分等级D且非免押客户,非免押用户,命中自有模型强拒,非芝麻租物进件且履约历史等级=1强拒,命中策略240703_14强拒,命中策略240703_4强拒,命中策略240703_10强拒,命中拒绝_通过加强包240801强拒,蚂蚁数控风险等级=996强拒,蚂蚁数控风险等级=998强拒,命中出库前风控流强拒,命中抖音_240829策略强拒,240927策略强拒
291604,A20250606174831588,2832754,cf50138ca53f54e110e4821011e51874,2025-06-06,2025-06,1,0,搜索渠道,None,None,[,出库,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
# （11）
# df_reject_before：前置命中的数据
# 前置拦截拒规则明细，1代表命中
df_reject_before = df[((df['status2']=='订单取消')&(df['拒绝理由'].notna()))|((df['status2']=='订单取消')&(df['total_describes'].notna()))]
df_reject_before['拒绝理由'] = np.where(df_reject_before['拒绝理由'].isna(), np.where(df_reject_before['total_describes'].str.contains('拒绝理由：'),df_reject_before['total_describes'].str.split('拒绝理由：').str[-1].str.split('"').str[0], df_reject_before['拒绝理由'])  
                                        ,df_reject_before['拒绝理由'])
# df_reject_before = df_reject_before[['create_time', 'order_id', 'order_number','trace_id', 'status2', '拒绝理由', 'total_describes', 'result','审核状态']]
# 2025.3.25
var_1 = ['create_time', 'order_id', 'order_number', 'status2', '拒绝理由', 'total_describes', 'result','trace_id','审核状态','下单日期','是否免人审','是否拒量','归属渠道','union_rent_tag','union_rent_rejected']
df_reject_before = df_reject_before[var_1]
df_reject_before_risk = risk.pre_reject(df_reject_before, 'order_number')

# 2025.3.25
# df_reject_before_risk_merge = df_reject_before[['order_number','order_id','trace_id','拒绝理由','审核状态']].merge(df_reject_before_risk, on='order_number', how='right')
var_need = ['order_number','order_id','trace_id','下单日期','search_time','是否免人审','是否拒量','归属渠道','union_rent_tag','union_rent_rejected','拒绝理由','审核状态']
df_reject_before['search_time'] = pd.to_datetime(df_reject_before["下单日期"]).dt.strftime('%Y-%m')
df_reject_before_risk_merge = df_reject_before[var_need].merge(df_reject_before_risk, on='order_number', how='right')

df_reject_before_risk_merge.shape
df_reject_before_risk_merge[:1]   
# 643314
# 699375

(665478, 26)

,order_number,order_id,trace_id,下单日期,search_time,是否免人审,是否拒量,归属渠道,union_rent_tag,union_rent_rejected,拒绝理由,审核状态,年龄超过49岁或低于18岁,命中法院失信/限高被执行人,命中刑事案件,命中借贷纠纷,命中融安分低于680,命中高风险名单,身份证命中黑名单,冰鉴火眸分<500,命中强拒加入临时黑名单,当日下单次数大于等于5次,当月下单次数大于等于10次,命中特殊地区 新疆|西藏,评分等级低于D,命中TD212强拒
0,A2023100100065712,656804,ec38b9c6d4fa3f36a542001b116e9efb,2023-10-01,2023-10,0,0,芝麻租物,NaN,NaN,[冰鉴火眸分<500命中强拒],前置拦截,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
# （12）进件和前置拦截-合并
# df_rule_data：前置被拒绝的和进件的样本数据
df_rule_data = df_reject_risk_merge.merge(df_reject_before_risk_merge, on = ['order_number','order_id','trace_id'], how = 'outer')
df_rule_data = df_rule_data.rename(columns = {'拒绝理由_x':'机审拒绝','拒绝理由_y':'前置拒绝'})

# df_rule_data[df_rule_data.审核状态_x == df_rule_data.审核状态_y].shape
# df_reject_before_risk_merge.审核状态.value_counts()
df_rule_data['审核状态'] = np.where(pd.isnull(df_rule_data.审核状态_x), df_rule_data.审核状态_y, df_rule_data.审核状态_x)
df_rule_data = df_rule_data.drop(columns = ['审核状态_x','审核状态_y'])

df_reject_risk_merge.shape
df_reject_before_risk_merge.shape
df_rule_data.shape
# df_rule_data[:1]

# df_rule_data[df_rule_data.机审拒绝.notnull()][:20]
# df_rule_data[df_rule_data.order_number == 'A2023010909584423']

df_rule_data['下单日期'] = np.where(pd.isnull(df_rule_data.下单日期_x), df_rule_data.下单日期_y, df_rule_data.下单日期_x)
df_rule_data['search_time'] = np.where(pd.isnull(df_rule_data.search_time_x), df_rule_data.search_time_y, df_rule_data.search_time_x)
df_rule_data['是否免人审'] = np.where(pd.isnull(df_rule_data.是否免人审_x), df_rule_data.是否免人审_y, df_rule_data.是否免人审_x)
df_rule_data['是否拒量'] = np.where(pd.isnull(df_rule_data.是否拒量_x), df_rule_data.是否拒量_y, df_rule_data.是否拒量_x)
df_rule_data['归属渠道'] = np.where(pd.isnull(df_rule_data.归属渠道_x), df_rule_data.归属渠道_y, df_rule_data.归属渠道_x)
df_rule_data['union_rent_tag'] = np.where(pd.isnull(df_rule_data.union_rent_tag_x), df_rule_data.union_rent_tag_y, df_rule_data.union_rent_tag_x)
df_rule_data['union_rent_rejected'] = np.where(pd.isnull(df_rule_data.union_rent_rejected_x), df_rule_data.union_rent_rejected_y, df_rule_data.union_rent_rejected_x)

df_rule_data = df_rule_data.drop(columns = ['下单日期_x','下单日期_y','search_time_x','search_time_y','是否免人审_x','是否免人审_y','是否拒量_x','是否拒量_y','归属渠道_x','归属渠道_y','union_rent_tag_x','union_rent_tag_y','union_rent_rejected_x','union_rent_rejected_y'])
df_rule_data.shape
df_rule_data[:1]

(291605, 38)

(665478, 26)

(957083, 60)

(957083, 53)

,order_number,order_id,trace_id,机审拒绝,评分等级D且综合风险等级为3,综合风险等级为3,命中法院失信/限高被执行人_x,命中刑事案件_x,命中借贷纠纷_x,命中融安分低于680_x,命中高风险名单_x,命中风险勘测,蚁盾分>=80,命中30天多头>13并且90天多头>39,履约历史等级1且为搜索渠道,云商分低于496,评分等级低于D_x,评分等级D且非免押客户,非免押用户,命中自有模型强拒,非芝麻租物进件且履约历史等级=1强拒,命中策略240703_14强拒,命中策略240703_4强拒,命中策略240703_10强拒,命中拒绝_通过加强包240801强拒,蚂蚁数控风险等级=996强拒,蚂蚁数控风险等级=998强拒,命中出库前风控流强拒,命中抖音_240829策略强拒,240927策略强拒,前置拒绝,年龄超过49岁或低于18岁,命中法院失信/限高被执行人_y,命中刑事案件_y,命中借贷纠纷_y,命中融安分低于680_y,命中高风险名单_y,身份证命中黑名单,冰鉴火眸分<500,命中强拒加入临时黑名单,当日下单次数大于等于5次,当月下单次数大于等于10次,命中特殊地区 新疆|西藏,评分等级低于D_y,命中TD212强拒,审核状态,下单日期,search_time,是否免人审,是否拒量,归属渠道,union_rent_tag,union_rent_rejected
0,A2023100100065712,656804,ec38b9c6d4fa3f36a542001b116e9efb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[冰鉴火眸分<500命中强拒],NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,前置拦截,2023-10-01,2023-10,0.0,0.0,芝麻租物,NaN,NaN


In [87]:
# （13）进件和前置拦截-明细

df_rule_data = df_rule_data.fillna(0)
df_rule_data[:1]

df_rule_data.shape
# df_rule_data.drop_duplicates(subset=['order_id']).shape
# df_rule_data.drop_duplicates(subset=['trace_id']).shape
# df_rule_data.drop_duplicates(subset=['order_id','trace_id']).shape
# # (395509, 44)
# df_rule_data[df_rule_data.trace_id == 0].shape
# df_rule_data[df_rule_data.order_id == 0].shape
# df_rule_data[(df_rule_data.trace_id == 0) & (df_rule_data.order_id == 0)].shape
# df_rule_data[(df_rule_data.trace_id != 0) & (df_rule_data.order_id != 0)][['order_id','trace_id']][:2]
# trace_id 重复， order_id不重复，中间表有缓存
# df_rule_data[df_rule_data.trace_id == '441581199811230010'][['order_id','trace_id']]
# 441581199811230010                     10
# 947affface19108aada119d330bac244        7

,order_number,order_id,trace_id,机审拒绝,评分等级D且综合风险等级为3,综合风险等级为3,命中法院失信/限高被执行人_x,命中刑事案件_x,命中借贷纠纷_x,命中融安分低于680_x,命中高风险名单_x,命中风险勘测,蚁盾分>=80,命中30天多头>13并且90天多头>39,履约历史等级1且为搜索渠道,云商分低于496,评分等级低于D_x,评分等级D且非免押客户,非免押用户,命中自有模型强拒,非芝麻租物进件且履约历史等级=1强拒,命中策略240703_14强拒,命中策略240703_4强拒,命中策略240703_10强拒,命中拒绝_通过加强包240801强拒,蚂蚁数控风险等级=996强拒,蚂蚁数控风险等级=998强拒,命中出库前风控流强拒,命中抖音_240829策略强拒,240927策略强拒,前置拒绝,年龄超过49岁或低于18岁,命中法院失信/限高被执行人_y,命中刑事案件_y,命中借贷纠纷_y,命中融安分低于680_y,命中高风险名单_y,身份证命中黑名单,冰鉴火眸分<500,命中强拒加入临时黑名单,当日下单次数大于等于5次,当月下单次数大于等于10次,命中特殊地区 新疆|西藏,评分等级低于D_y,命中TD212强拒,审核状态,下单日期,search_time,是否免人审,是否拒量,归属渠道,union_rent_tag,union_rent_rejected
0,A2023100100065712,656804,ec38b9c6d4fa3f36a542001b116e9efb,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[冰鉴火眸分<500命中强拒],0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,前置拦截,2023-10-01,2023-10,0.0,0.0,芝麻租物,0,0


(957083, 53)

In [88]:
# （14）进件数据
# df_risk_new：申请数据
# 仅取进件的数据， 拒绝理由 == '[' 的可能 可能是拒件 也可能是出库， 看 审核状态
df_jj = df_risk_new[df_risk_new['进件']=='进件']
df_jj = df_jj[['order_number','下单日期','search_time','归属渠道','是否免人审','审核状态','拒绝理由','是否前置拦截','是否机审强拒','是否出库前风控强拒','是否拒量','union_rent_tag','union_rent_rejected','user_mobile','id_card_num','jd_user_level_risk']]
df_jj = df_jj.rename(columns = {'union_rent_tag': '是否联合运营', 'union_rent_rejected': '联合运营命中风控'})
df_jj.shape
df_jj.tail()

# 释放空间
del df
del df2
del df_reject_risk_merge
del df_reject_before_risk_merge
del df_risk 
del df_re 
del df_ra 
del df_xmy 
del df_risk_all2 
del df_reject_before 
del df_reject_before_risk

(291605, 16)

,order_number,下单日期,search_time,归属渠道,是否免人审,审核状态,拒绝理由,是否前置拦截,是否机审强拒,是否出库前风控强拒,是否拒量,是否联合运营,联合运营命中风控,user_mobile,id_card_num,jd_user_level_risk
1994195,A20250606172213281,2025-06-06,2025-06,芝麻租物,1,出库,[,0,0,0,0,None,None,13926641633,441802200611033844,None
1994196,A20250606173534423,2025-06-06,2025-06,芝麻租物,1,出库,[,0,0,0,0,None,None,13284873791,15232720000401773X,None
1994197,A20250606173638443,2025-06-06,2025-06,芝麻租物,1,出库,[,0,0,0,0,None,None,15082519816,513221198907060610,None
1994198,A20250606174508544,2025-06-06,2025-06,京东渠道,0,出库,[,0,0,0,0,None,None,17673168721,331082200407211415,"{""authStatus"":""SUCCESS"",""riskLevel"":""D"",""black..."
1994199,A20250606174831588,2025-06-06,2025-06,搜索渠道,1,出库,[,0,0,0,0,None,None,15670210353,411526199611180741,None


In [89]:
# （15）数据检查
# df_jj.shape
# df_jj[:1]
# df_jj.是否拒量.value_counts()
# df_jj.是否联合运营.value_counts()
# df_jj[df_jj.是否联合运营 == '1']
# df_rule_data[:1]

# df_risk.shape
# df_risk[:3]
# df_risk.union_rent_tag.value_counts()
# df_risk[df_risk.union_rent_tag == '1'].shape

# df[df.union_rent_tag == '1'].shape

# df_risk[df_risk.union_rent_tag == '1'][['trace_id']]
# df[df.union_rent_tag == '1'][['trace_id']]

# 以下三个 trace_id 在 df_risk，但不在 df
# 8f80fc9a5c1b864e9e41260de3a86273
# 8ff42b701a73228a59f360e851b630aa
# 47cf27a705f026fadcefef09090ef2df

# df.shape
# df[df.trace_id == '8ff42b701a73228a59f360e851b630aa']

# 免押数据
# df_jj.拒绝理由.value_counts()
# tmp = df_jj.groupby(['拒绝理由']).agg({'order_number': 'count'}).reset_index()
# tmp = tmp.sort_values(by = 'order_number', ascending = False)
# tmp.to_excel('tmp.xlsx', index = False)

# df_jj[df_jj.拒绝理由.isin(['[命中非免押用户拒绝]','命中非免押用户拒绝'])].shape

# tmp = df_jj[(df_jj.search_time == '2025-02') & (~df_jj.拒绝理由.isin(['[命中非免押用户拒绝]','命中非免押用户拒绝']))][['order_number','下单日期','search_time','归属渠道']]
# tmp.shape
# tmp.to_excel('免押进件_202502.xlsx', index = False)
# tmp[:5]

# 17135 - 1351= 15784

# # 联合运营订单漏斗
# sql = '''
# SELECT  id as order_id, order_number, create_time, union_rent_tag, status
# from  db_digua_business.t_order
# where union_rent_tag = 'Y'
# ;
# '''
# tmp1 = query(sql)
# tmp1.shape

# # 有3单未关联到，为测试白名单
# # 拒绝理由 为前置拒绝理由，union_rent_rejected 为机审拒绝，因为联合运营的机审豁免了，所以单独存放机审拒绝
# # ,om.user_mobile,tmu.true_name,tmu.id_card_num
# var_1 = ['order_number','拒绝理由','审核状态','下单日期','search_time','union_rent_rejected','union_rent_tag','product_name','进件','user_mobile','id_card_num']
# df_lhyy_1 = tmp1.merge(df_risk_new[var_1], on = 'order_number', how = 'inner')
# df_lhyy_1 = df_lhyy_1[df_lhyy_1.下单日期 >= '2025-03-18']
# df_lhyy_1.shape
# df_lhyy_1[:2]
# df_lhyy_1 = df_lhyy_1.merge(df_out_data[['order_number','grade','bj_qy_score','br_ra_score','rentOnlineDetailLevelOverall']], on = 'order_number', how = 'left')
# df_lhyy_1['grade_new'] = df_lhyy_1.apply(lambda x: 'F' if x.bj_qy_score <= 400 and x.br_ra_score <= 799
#                                          else 'E' if x.bj_qy_score <= 400 and x.br_ra_score > 799

#                                          else 'F' if x.bj_qy_score > 400 and x.bj_qy_score <= 470 and x.br_ra_score <= 679
#                                          else 'D' if x.bj_qy_score > 400 and x.bj_qy_score <= 470 and x.br_ra_score > 679 and x.br_ra_score <= 799
#                                          else 'C' if x.bj_qy_score > 400 and x.bj_qy_score <= 470 and x.br_ra_score > 799 and x.br_ra_score <= 839
#                                          else 'B' if x.bj_qy_score > 400 and x.bj_qy_score <= 470 and x.br_ra_score > 839 and x.br_ra_score <= 879
#                                          else 'A' if x.bj_qy_score > 400 and x.bj_qy_score <= 470 and x.br_ra_score > 879

#                                          else 'F' if x.bj_qy_score > 470 and x.bj_qy_score <= 500 and x.br_ra_score <= 679
#                                          else 'D' if x.bj_qy_score > 470 and x.bj_qy_score <= 500 and x.br_ra_score > 679 and x.br_ra_score <= 799
#                                          else 'C' if x.bj_qy_score > 470 and x.bj_qy_score <= 500 and x.br_ra_score > 799 and x.br_ra_score <= 839
#                                          else 'B' if x.bj_qy_score > 470 and x.bj_qy_score <= 500 and x.br_ra_score > 839 and x.br_ra_score <= 879
#                                          else 'A' if x.bj_qy_score > 470 and x.bj_qy_score <= 500 and x.br_ra_score > 879
                                         
#                                          else 'F' if x.bj_qy_score > 500 and x.bj_qy_score <= 525 and x.br_ra_score <= 679
#                                          else 'D' if x.bj_qy_score > 500 and x.bj_qy_score <= 525 and x.br_ra_score > 679 and x.br_ra_score <= 709
#                                          else 'C' if x.bj_qy_score > 500 and x.bj_qy_score <= 525 and x.br_ra_score > 709 and x.br_ra_score <= 769
#                                          else 'B' if x.bj_qy_score > 500 and x.bj_qy_score <= 525 and x.br_ra_score > 769 and x.br_ra_score <= 879
#                                          else 'A' if x.bj_qy_score > 500 and x.bj_qy_score <= 525 and x.br_ra_score > 879 and x.br_ra_score <= 910
#                                          else 'S' if x.bj_qy_score > 500 and x.bj_qy_score <= 525 and x.br_ra_score > 910

#                                          else 'F' if x.bj_qy_score > 525 and x.bj_qy_score <= 535 and x.br_ra_score <= 679
#                                          else 'C' if x.bj_qy_score > 525 and x.bj_qy_score <= 535 and x.br_ra_score > 679 and x.br_ra_score <= 769
#                                          else 'B' if x.bj_qy_score > 525 and x.bj_qy_score <= 535 and x.br_ra_score > 769 and x.br_ra_score <= 799
#                                          else 'A' if x.bj_qy_score > 525 and x.bj_qy_score <= 535 and x.br_ra_score > 799 and x.br_ra_score <= 879
#                                          else 'S' if x.bj_qy_score > 525 and x.bj_qy_score <= 535 and x.br_ra_score > 879

#                                          else 'F' if x.bj_qy_score > 535 and x.bj_qy_score <= 560 and x.br_ra_score <= 679
#                                          else 'C' if x.bj_qy_score > 535 and x.bj_qy_score <= 560 and x.br_ra_score > 679 and x.br_ra_score <= 769
#                                          else 'B' if x.bj_qy_score > 535 and x.bj_qy_score <= 560 and x.br_ra_score > 769 and x.br_ra_score <= 799
#                                          else 'A' if x.bj_qy_score > 535 and x.bj_qy_score <= 560 and x.br_ra_score > 799 and x.br_ra_score <= 839
#                                          else 'S' if x.bj_qy_score > 535 and x.bj_qy_score <= 560 and x.br_ra_score > 839
                                         
#                                          else 'F' if x.bj_qy_score > 560 and x.bj_qy_score <= 575 and x.br_ra_score <= 679
#                                          else 'C' if x.bj_qy_score > 560 and x.bj_qy_score <= 575 and x.br_ra_score > 679 and x.br_ra_score <= 739
#                                          else 'B' if x.bj_qy_score > 560 and x.bj_qy_score <= 575 and x.br_ra_score > 739 and x.br_ra_score <= 799
#                                          else 'A' if x.bj_qy_score > 560 and x.bj_qy_score <= 575 and x.br_ra_score > 799 and x.br_ra_score <= 839
#                                          else 'S' if x.bj_qy_score > 560 and x.bj_qy_score <= 575 and x.br_ra_score > 839

#                                          else 'F' if x.bj_qy_score > 575 and x.bj_qy_score <= 600 and x.br_ra_score <= 679
#                                          else 'C' if x.bj_qy_score > 575 and x.bj_qy_score <= 600 and x.br_ra_score > 679 and x.br_ra_score <= 709
#                                          else 'B' if x.bj_qy_score > 575 and x.bj_qy_score <= 600 and x.br_ra_score > 709 and x.br_ra_score <= 799
#                                          else 'A' if x.bj_qy_score > 575 and x.bj_qy_score <= 600 and x.br_ra_score > 799 and x.br_ra_score <= 839
#                                          else 'S' if x.bj_qy_score > 575 and x.bj_qy_score <= 600 and x.br_ra_score > 839

#                                          else 'F' if x.bj_qy_score > 600 and x.bj_qy_score <= 680 and x.br_ra_score <= 679
#                                          else 'B' if x.bj_qy_score > 600 and x.bj_qy_score <= 680 and x.br_ra_score > 679 and x.br_ra_score <= 769
#                                          else 'A' if x.bj_qy_score > 600 and x.bj_qy_score <= 680 and x.br_ra_score > 769 and x.br_ra_score <= 839
#                                          else 'S' if x.bj_qy_score > 600 and x.bj_qy_score <= 680 and x.br_ra_score > 839

#                                          else 'F' if x.bj_qy_score > 680 and x.br_ra_score <= 679
#                                          else 'B' if x.bj_qy_score > 680 and x.br_ra_score > 679 and x.br_ra_score <= 709
#                                          else 'A' if x.bj_qy_score > 680 and x.br_ra_score > 709 and x.br_ra_score <= 799
#                                          else 'S' if x.bj_qy_score > 680 and x.br_ra_score > 799

#                                          else 'other', axis = 1)
# df_analy_data['是否出库'] = 1
# df_lhyy_1 = df_lhyy_1.merge(df_analy_data[['order_number','是否出库']], on = 'order_number', how = 'left')
# df_lhyy_1.to_excel('llyy_tmp.xlsx', index = False)


# # 拒量
# df_risk_new[df_risk_new['是否拒量'] == 1].shape
# df_risk_new[(df_risk_new['是否拒量'] == 1) & (df_risk_new.下单日期>='2025-03-17') & (df_risk_new.下单日期<='2025-03-23')].审核状态.value_counts()
# # df_risk_new[(df_risk_new['是否拒量'] == 1) & (df_risk_new.下单日期>='2025-03-17') & (df_risk_new.下单日期<='2025-03-23')].drop_duplicates(subset = ['id_card_num']).拒绝理由.value_counts()
# df_risk_new[(df_risk_new['是否拒量'] == 1) & (df_risk_new.下单日期>='2025-03-17') & (df_risk_new.下单日期<='2025-03-23')].拒绝理由.value_counts()

# tmp = df_risk_new[df_risk_new['是否拒量'] == 1][['下单日期','order_number','tips','审核状态']]
# tmp['是否出库'] = np.where(tmp.审核状态 == '出库', 1, 0)
# # tmp['策略命中等级'] = tmp['tips'].str.extract(r'(策略241205命中\(\d+\)?|策略241212命中\(\d+\)?|命中自有模型回捞策略|回捞策略250330命中?)')[0]
# tmp['策略命中等级'] = tmp['tips'].str.extract(r'(策略241205命中\(\d+\)?|策略241212命中?|命中自有模型回捞策略|回捞策略250330命中?)')[0]
# tmp = tmp[['下单日期','order_number','策略命中等级','是否出库']]
# # tmp.to_excel('拒量数据_20250422.xlsx', index = False)
# tmp.to_excel('拒量数据_20250507.xlsx', index = False)

#八派
# var_1 = ['order_number','拒绝理由','审核状态','下单日期','search_time','union_rent_rejected','union_rent_tag','product_name','进件','user_mobile','id_card_num']
# condition_1 = (df_risk_new.归属渠道== '八派信息')
# df_risk_new[condition_1][var_1].to_excel('八派.xlsx', index = False)

# 聚量派
# var_1 = ['order_number','拒绝理由','审核状态','下单日期','search_time','union_rent_rejected','union_rent_tag','product_name','进件','user_mobile','id_card_num']
# condition_1 = (df_risk_new.归属渠道== '聚量派')
# df_risk_new[condition_1][var_1].to_excel('聚量派.xlsx', index = False)

# 蚁盾分
# df_risk_new[(df_risk_new.拒绝理由.isin(['[命中蚁盾分>50]','[蚁盾分>=80]','[命中蚁盾分>50, 蚁盾分>=80]']))&(df_risk_new.下单日期>='2025-03-01')].groupby(['下单日期']).agg({'order_number':'count'})




# 5.自变量

## （5-1-1）中间表

In [90]:
# （5-1-1）中间表
# trace_df 关联其他表，有缓存的用 trace_id 关联，没缓存的用 order_trace_id 关联
# 2025.3.19：目前存在以下情况，各数据源的表有数据，但中间表没有，主要是美宜租、小蚂蚁或者23年及之前的数据，非免押也会流到小蚂蚁，到小蚂蚁之后会调贷前风控，中间表只含自营订单，故会出现数据偏差
sql1='''-- trace关联表 
SELECT * 
from db_credit.order_association
;
'''

trace_df =query(sql1)
trace_df.shape
trace_df[trace_df.order_id.isnull()].shape
trace_df[trace_df.order_number.isnull()].shape
trace_df[trace_df.trace_id.isnull()].shape
# trace_df[trace_df.trace_id.isnull()].tail()
# trace_df.drop_duplicates(subset=['trace_id']).shape

# trace_id 有重复
# order_trace_id 有重复，没有空
# order_id 无重复，没有空
# trace_df[trace_df['order_trace_id'] == '441581199811230010']

# 三者非空的时候，order_trace_id = risk_trace_id，但 trace_id 不一定和risk_trace_id/order_trace_id相等
# tmp1 = trace_df[(trace_df.trace_id.notna()) & (trace_df.order_trace_id.notna()) & (trace_df.risk_trace_id.notna())]
# tmp1.shape
# tmp1[tmp1.trace_id== tmp1.order_trace_id].shape
# tmp1[tmp1.trace_id== tmp1.risk_trace_id].shape
# tmp1[tmp1.order_trace_id== tmp1.risk_trace_id].shape
# tmp1[tmp1.trace_id!= tmp1.risk_trace_id][:1]

# （2）区别用哪个模型
# 0：没有使用模型
# 1：ascore2311
# 2：ascore_yl2311
# 3：ascore2401
# 4: ascore2501
sql1='''-- risk关联表 
SELECT trace_id, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.is_model') end,'"','') as is_model
from db_credit.risk
;
'''
risk_df_model = query(sql1)
risk_df_model.shape
# risk_df_model[:2]

# 这个关联方式有问题
# trace_df = trace_df.merge(risk_df_model, on = 'trace_id', how = 'left')
# trace_df.shape
# trace_df[:1]

(2302173, 5)

(0, 5)

(0, 5)

(26711, 5)

(364103, 2)

## （5-2-1）火眸分

In [91]:
# （5-2-1）火眸分
sql1='''-- 风控-冰鉴-火眸分
SELECT * 
from db_credit.risk_icekredit_cds
;
'''

bj_hm_df =query(sql1)
bj_hm_df.shape

bj_hm_df['bj_hm_score'] = bj_hm_df['rs'].apply(lambda x: json.loads(x).get('result', {}).get('card_device_score22', {}).get('bj_score', None))  
bj_hm_df['bj_hm_score'] = bj_hm_df['bj_hm_score'].astype(int)
bj_hm_df = bj_hm_df[['trace_id', 'bj_hm_score']]



df_out_data = trace_df.merge(bj_hm_df, on = 'trace_id', how = 'left')
df_out_data.shape
df_out_data[:1]

del bj_hm_df

(1248459, 5)

(2302173, 6)

,order_id,order_number,order_trace_id,trace_id,risk_trace_id,bj_hm_score
0,88929,A202301090005022,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,567.0


## （5-3-1）芝麻租物-风控专业版/风控进阶版

In [92]:
# （5-3-1）芝麻租物-风控专业版/风控进阶版 --调用周期-7天
# 细化综合风险等级 rentOnlineDetailLevelOverall  1-2024.9-2024.12切流调用，2025.1.1开始切流100%；2-线下测试 203.10-2024.10，仅租物渠道
# 综合风险等级 rentOnlineRiskLevelOverall
# 共租行为等级 rentOnlineRiskLevelMultiRent
# 履约历史等级 rentOnlineRiskLevelFulfill

sql1='''-- 芝麻租物风控专业版
-- 95184	95184	95184
SELECT *
from db_credit.risk_alipay_auth_query
;
'''

zm_fk_df = query(sql1)
zm_fk_df = zm_fk_df[['trace_id', 'rentOnlineRiskLevelOverall', 'rentOnlineRiskLevelMultiRent', 'rentOnlineRiskLevelFulfill','rentOnlineDetailLevelOverall']]
zm_fk_df.shape
zm_fk_df[zm_fk_df.trace_id.isnull()].shape
# zm_fk_df[:1]

# outer 关联出来的 order_number 都是空值，没意义
# df_out_data = trace_df.merge(zm_fk_df, on = 'trace_id', how = 'outer')
df_out_data = df_out_data.merge(zm_fk_df, on = 'trace_id', how = 'left')
df_out_data.shape
df_out_data[:1]

del zm_fk_df

(301530, 5)

(0, 5)

(2302173, 10)

,order_id,order_number,order_trace_id,trace_id,risk_trace_id,bj_hm_score,rentOnlineRiskLevelOverall,rentOnlineRiskLevelMultiRent,rentOnlineRiskLevelFulfill,rentOnlineDetailLevelOverall
0,88929,A202301090005022,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,567.0,NaN,NaN,NaN,NaN


## （5-3-2）细化综合风险等级

In [93]:
# （5-3-2）细化综合风险等级-补充
# 2023.10及之后的样本，本次选取 2023.10-2024.10 的进件（出库），trace_id、逾期标签（M1+、M3+、FPD31+）、品类（全新、二手、非手机）
# df_analy_data[df_analy_data.order_id.isin([508404,1678492,1692916,12449])]

# 2025.1.1全面调用，2024.11-2024.12的样本为随机线上调用的（2024.09-2024.12线上随机调用）

# 返回结果 outer_order_id 去重
path = 'F:/租后表分析数据_ld/三方数据/'
df_lhyy_in = pd.read_excel(path + 'lhyy_sample_data_diguazuji_return.xlsx')
df_lhyy_in = df_lhyy_in.rename(columns = {'outer_order_id': 'out_order_no'})
df_lhyy_in.shape
df_lhyy_in.drop_duplicates(subset = ['out_order_no']).shape

# # （2-3-1）取 trace_id
# sql1='''-- trace关联表 
# SELECT trace_id, order_number
# from db_credit.order_association
# ;
# '''

# trace_df =query(sql1)
# trace_df.shape

# （2-3-2）关联 out_order_no
sql1='''-- 芝麻租物风控专业版
-- 95184	95184	95184
SELECT trace_id,out_order_no
from db_credit.risk_alipay_auth_query
;
'''
zm_fk_df = query(sql1)
zm_fk_df.shape

# （2-3-3）表关联
trace_df = trace_df.merge(zm_fk_df, on = 'trace_id', how = 'left')
trace_df.shape

# 发散，有些没有租后表现
df_lhyy_in = df_lhyy_in.merge(trace_df, on = 'out_order_no', how = 'inner')
df_lhyy_in.shape
df_lhyy_in.drop_duplicates(subset = ['trace_id']).shape
# df_lhyy_in[df_lhyy_in.out_order_no == 'A2024082909142423']
df_lhyy_in.drop_duplicates(subset = ['out_order_no']).shape
df_lhyy_in.drop_duplicates(subset = ['order_number']).shape

df_lhyy_in = df_lhyy_in[['order_number','zm_lvl']]
df_lhyy_in['zm_lvl'] = df_lhyy_in.apply(lambda x: '0' if x.zm_lvl == 'R0' 
                                        else '1' if x.zm_lvl == 'R1'
                                        else '2' if x.zm_lvl == 'R2'
                                        else '3' if x.zm_lvl == 'R3'
                                        else '4' if x.zm_lvl == 'R4'
                                        else '5' if x.zm_lvl == 'R5'
                                        else '6' if x.zm_lvl == 'R6'
                                        else x.zm_lvl, axis = 1)
# df_lhyy_in[:1]

# 关联到主表
df_out_data = df_out_data.merge(df_lhyy_in, on = 'order_number', how = 'left')
# 同时存在,以线上调用的结果为准
df_out_data['rentOnlineDetailLevelOverall'] = np.where(pd.isnull(df_out_data.rentOnlineDetailLevelOverall), df_out_data.zm_lvl, df_out_data.rentOnlineDetailLevelOverall)
df_out_data = df_out_data.drop(columns = ['zm_lvl'])
df_out_data.shape
df_out_data[:1]

# del trace_df
del zm_fk_df
del df_lhyy_in

# 17941
# tmp = df_lhyy_in.merge(df_analy_data, on = 'order_number', how = 'inner')
# tmp.shape

(17902, 2)

(17902, 2)

(301530, 2)

(2302173, 6)

(24751, 7)

(17902, 7)

(17902, 7)

(24751, 7)

(2302173, 10)

,order_id,order_number,order_trace_id,trace_id,risk_trace_id,bj_hm_score,rentOnlineRiskLevelOverall,rentOnlineRiskLevelMultiRent,rentOnlineRiskLevelFulfill,rentOnlineDetailLevelOverall
0,88929,A202301090005022,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,567.0,NaN,NaN,NaN,NaN


## （5-3-3）蚂蚁链-蚁盾分1.0

In [94]:
# （5-3-3）蚂蚁链-蚁盾分1.0
sql1='''-- 风控-蚂蚁链租赁风险分
-- 122905	122905	122905
SELECT trace_id, score as my_zl_score
from db_credit.risk_antchain
;
'''

my_zl_df=query(sql1)
my_zl_df['my_zl_score'] = my_zl_df['my_zl_score'].astype(float)
my_zl_df.shape

# 11454 条关联不到 	order_id	order_number	
# df_out_data = df_out_data.merge(my_zl_df, on = 'trace_id', how = 'outer')
df_out_data = df_out_data.merge(my_zl_df, on = 'trace_id', how = 'left')
df_out_data.shape
df_out_data[:1] 

del my_zl_df 

(339008, 2)

(2302173, 11)

,order_id,order_number,order_trace_id,trace_id,risk_trace_id,bj_hm_score,rentOnlineRiskLevelOverall,rentOnlineRiskLevelMultiRent,rentOnlineRiskLevelFulfill,rentOnlineDetailLevelOverall,my_zl_score
0,88929,A202301090005022,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,567.0,NaN,NaN,NaN,NaN,1.419


## （5-3-4）蚂蚁链-蚁盾分2.0

In [95]:
# （5-3-4）蚂蚁链-蚁盾分2.0
# 1-2024.12线下回溯，样本详情：
# （1）2024.9-2024.11 的出库订单 --评估稳定性、区分度
# （2）2024.12.1-2024.12.17 的进件 --评估单条规则命中率、独立命中率
# afz_v1_score ：租赁通用分，取值 -1 表示库无记录
# path_my = 'C:/Users/Administrator/Desktop/mine/7_三方数据产品/1_蚂蚁/2_租赁分洞察分/1.20241218/'
path_my = path
data_my_out = pd.read_excel(path_my + 'my_sample_20241218.xlsx')
data_my_in_1 = pd.read_csv(path_my + '租赁通用分V1_迪瓜_241220.csv')

data_my_in_1 = data_my_in_1[['certno_hash','mob_hash','apply_date','afz_v1_score']]
data_my_in_1['afz_v1_score'] = np.where(data_my_in_1.afz_v1_score == -1, -999, data_my_in_1.afz_v1_score)
data_my_in_1["apply_date"] = data_my_in_1["apply_date"].astype('str')
data_my_in_1[data_my_in_1.afz_v1_score == -999].shape
data_my_in_1 = data_my_in_1.drop_duplicates(subset = ['certno_hash','mob_hash','apply_date','afz_v1_score'])

data_my_out = data_my_out[['order_number','order_id', '下单日期', '归属渠道','idcard_sha256','mobile_sha256']]
data_my_out = data_my_out.rename(columns = {'idcard_sha256': 'certno_hash', 'mobile_sha256': 'mob_hash'})
data_my_out["apply_date"] = data_my_out["下单日期"].astype('str')
data_my_out["apply_date"] = data_my_out["apply_date"].str.replace('-','')
data_my_out['search_time'] = pd.to_datetime(data_my_out["下单日期"]).dt.strftime('%Y-%m')

data_my_out.shape
data_my_in_1.shape

data_my = data_my_out.merge(data_my_in_1, on = ['certno_hash','mob_hash','apply_date'], how = 'left')
# data_my.drop(columns=['apply_date','certno_hash','mob_hash'],inplace = True)
data_my = data_my[['order_number','afz_v1_score']]
data_my.shape
data_my[:2]

del data_my_in_1
del data_my_out

# 2-线下回溯 2024.12.16-2025.1.31 有租后表现的 + 2025.2.1-2025.2.19 免押进件（2025.2.19 上线）
# path_my = 'C:/Users/Administrator/Desktop/mine/7_三方数据产品/1_蚂蚁/2_租赁分洞察分/2.20250320/'
path_my = path
data_my_out = pd.read_excel(path_my + 'my_sample_20250320.xlsx')
data_my_in_1 = pd.read_csv(path_my + '租赁通用分V1_20250401_DGXJ_ee2_zl_250326.csv')

data_my_in_1 = data_my_in_1[['certno_hash','mob_hash','apply_date','afz_v1_score']]
data_my_in_1['afz_v1_score'] = np.where(data_my_in_1.afz_v1_score == -1, -999, data_my_in_1.afz_v1_score)
data_my_in_1["apply_date"] = data_my_in_1["apply_date"].astype('str')
data_my_in_1[data_my_in_1.afz_v1_score == -999].shape
data_my_in_1 = data_my_in_1.drop_duplicates(subset = ['certno_hash','mob_hash','apply_date','afz_v1_score'])

data_my_out = data_my_out[['order_number','order_id', '下单日期', '归属渠道','idcard_sha256','mobile_sha256']]
data_my_out = data_my_out.rename(columns = {'idcard_sha256': 'certno_hash', 'mobile_sha256': 'mob_hash'})
data_my_out["apply_date"] = data_my_out["下单日期"].astype('str')
data_my_out["apply_date"] = data_my_out["apply_date"].str.replace('-','')
data_my_out['search_time'] = pd.to_datetime(data_my_out["下单日期"]).dt.strftime('%Y-%m')

data_my_out.shape
data_my_in_1.shape

data_my_1 = data_my_out.merge(data_my_in_1, on = ['certno_hash','mob_hash','apply_date'], how = 'left')
# data_my.drop(columns=['apply_date','certno_hash','mob_hash'],inplace = True)
data_my_1 = data_my_1[['order_number','afz_v1_score']]
data_my_1.shape

data_my = pd.concat([data_my,data_my_1])
data_my.shape
data_my = data_my.drop_duplicates(subset = 'order_number')
data_my.shape

del data_my_in_1
del data_my_out

# 3-关联到主表
# 17763
# tmp = df_out_data[['order_number']].merge(data_my[['order_number']], on = 'order_number', how = 'inner')
df_out_data = df_out_data.merge(data_my, on = 'order_number', how = 'left')



# 4-线上调 2025-02-19
sql1='''-- 风控-蚂蚁链租赁风险分
-- 122905	122905	122905
SELECT trace_id, score as my_zl_score2
from db_credit.risk_antchain_ato
;
'''

my_zl_df_2 = query(sql1)
my_zl_df_2['my_zl_score2'] = my_zl_df_2['my_zl_score2'].astype(float)
my_zl_df_2.shape

df_out_data = df_out_data.merge(my_zl_df_2, on = 'trace_id', how = 'left')
# df_out_data['my_zl_score2'] = np.where(pd.isnull(df_out_data.afz_v1_score), df_out_data.my_zl_score2, df_out_data.afz_v1_score)
df_out_data['my_zl_score2'] = np.where(pd.isnull(df_out_data.my_zl_score2), df_out_data.afz_v1_score, df_out_data.my_zl_score2)
df_out_data = df_out_data.drop(columns = ['afz_v1_score'])
df_out_data.shape
df_out_data[:1] 

del my_zl_df_2 
del data_my

# df_out_data[df_out_data.my_zl_score2 > 0][['order_number','my_zl_score2']][:5]
# order_number	my_zl_score2
# 1657388	A20250212142919191	77.3086
# 1658578	A20250212174156265	68.2489
# 1659580	A2025021300115318	52.5108
# 1660252	A2025021310090343	51.0409
# 1660308	A20250213102333106	56.4121

(16, 4)

(17764, 8)

(17757, 4)

(17764, 2)

,order_number,afz_v1_score
0,A20240901005306132,47.5799
1,A20240901005509136,57.9553


...

(30069, 2)

(30069, 2)

(96584, 2)

(2302173, 12)

,order_id,order_number,order_trace_id,trace_id,risk_trace_id,bj_hm_score,rentOnlineRiskLevelOverall,rentOnlineRiskLevelMultiRent,rentOnlineRiskLevelFulfill,rentOnlineDetailLevelOverall,my_zl_score,my_zl_score2
0,88929,A202301090005022,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,567.0,NaN,NaN,NaN,NaN,1.419,NaN


## （5-3-5）百融-风险勘测

In [96]:
# （5-3-5）百融-风险勘测

## （5-3-6）百融-云商分

In [97]:
# （5-3-6）百融-云商分
sql1='''-- 风控-百融贷前-云商分
-- 101331	101331	101331
SELECT *
from db_credit.risk_bairong_scoreysstd
;
'''

br_ys_df =query(sql1)
br_ys_df.shape

br_ys_df['br_ys_score'] = br_ys_df['query_rs'].apply(lambda x: json.loads(x).get('Score', {}).get('scoreysstd', None))  
br_ys_df['br_ys_score'] = br_ys_df['br_ys_score'].astype(int)
br_ys_df = br_ys_df[['trace_id', 'br_ys_score']]

# 3263 条关联不到 	order_id	order_number	
# df_out_data = df_out_data.merge(br_ys_df, on = 'trace_id', how = 'outer')
df_out_data = df_out_data.merge(br_ys_df, on = 'trace_id', how = 'left')
df_out_data.shape
df_out_data[:1]

# 在 br_ys_df 有，但在 trace_df 没有，3000多条，2024.12 有511条
# bc74db481731377076457110184c188f	
# e3f5baf3d90ea794fdb6fb3a28dd8dc5

# df_out_data[df_out_data.trace_id == 'bc74db481731377076457110184c188f']
# br_ys_df[br_ys_df.trace_id == 'bc74db481731377076457110184c188f']

# df_out_data[df_out_data.trace_id == 'e3f5baf3d90ea794fdb6fb3a28dd8dc5']
# br_ys_df[br_ys_df.trace_id == 'e3f5baf3d90ea794fdb6fb3a28dd8dc5']

del br_ys_df

(311462, 6)

(2302173, 13)

,order_id,order_number,order_trace_id,trace_id,risk_trace_id,bj_hm_score,rentOnlineRiskLevelOverall,rentOnlineRiskLevelMultiRent,rentOnlineRiskLevelFulfill,rentOnlineDetailLevelOverall,my_zl_score,my_zl_score2,br_ys_score
0,88929,A202301090005022,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,567.0,NaN,NaN,NaN,NaN,1.419,NaN,NaN


## （5-3-7）百融-融安分

In [98]:
# （5-3-7）百融-融安分
# 仅取融安分-大表
sql1='''-- 风控-百融 
SELECT rb.trace_id
    ,replace(case when JSON_VALID(rb.query_rs) THEN JSON_EXTRACT(rb.query_rs, '$.Score.scoreprime') end,'"','') as br_ra_score
from db_credit.risk_bairong rb 
-- LIMIT 100 
;
'''

br_ra_df =query(sql1)
br_ra_df['br_ra_score'] = br_ra_df['br_ra_score'].astype(float)
br_ra_df.shape

# 百融-融安分A-主流-标准款-小表
# db_credit.risk_bairong_scoreprime
sql1='''-- 风控-百融 
SELECT trace_id, query_rs, query_time
from db_credit.risk_bairong_scoreprime
;
'''
br_ra_df_1 = query(sql1)
# br_ra_df_1['search_time'] = br_ra_df_1['query_time'].dt.strftime('%Y-%m')

br_ra_df_1['br_ra_score'] = br_ra_df_1['query_rs'].apply(lambda x: json.loads(x).get('RiskStrategy', {}).get('Score', {}).get('scoreprime', None))  
br_ra_df_1['br_ra_score'] = pd.to_numeric(br_ra_df_1['br_ra_score'].replace('"null"', np.nan), errors='coerce')
br_ra_df_1 = br_ra_df_1[['trace_id','br_ra_score']]

# 合并
br_ra_df = pd.concat([br_ra_df, br_ra_df_1])
br_ra_df.shape
br_ra_df = br_ra_df.drop_duplicates(subset = 'trace_id')

# 关联
# df_out_data = df_out_data.drop(columns = 'br_ra_score')
df_out_data = df_out_data.merge(br_ra_df,on = 'trace_id',how='left')
df_out_data.shape
df_out_data[:1]

del br_ra_df
del br_ra_df_1

# 在 br_ra_df 有，但在 trace_df 没有，25000条左右，2024.12 有511条
# sql1='''-- 风控-百融 
# SELECT rb.trace_id
#     ,replace(case when JSON_VALID(rb.query_rs) THEN JSON_EXTRACT(rb.query_rs, '$.Score.scoreprime') end,'"','') as br_ra_score
#     ,query_time
# from db_credit.risk_bairong rb 
# # LIMIT 100 
# ;
# '''

# br_ra_df =query(sql1)
# br_ra_df["下单日期"] = br_ra_df['query_time'].dt.date
# br_ra_df["下单日期"] = pd.to_datetime(br_ra_df["下单日期"],errors="coerce")
# br_ra_df['search_time'] = pd.to_datetime(br_ra_df["下单日期"]).dt.strftime('%Y-%m')
# br_ra_df.shape
# br_ra_df.drop_duplicates(subset = 'trace_id').shape

# tmp1 = df_out_data.merge(br_ra_df,on = 'trace_id',how='right')
# tmp1.shape
# tmp1[tmp1.br_ra_score_x.isnull()].shape
# tmp1[tmp1.br_ra_score_x.isnull()].drop_duplicates(subset = 'trace_id').shape
# tmp1[tmp1.br_ra_score_x.isnull()].drop_duplicates(subset = 'trace_id').search_time.value_counts()

# tmp1[(tmp1.br_ra_score_x.isnull())&(tmp1.search_time == '2025-01')][:3]
# 1979578101591333f0a05da8bcfacf16
# a1ca07b662b4ce93755f69421cc86ba3
# 1784690257903de4551080402ee0a996
# df_out_data[df_out_data.trace_id == '1979578101591333f0a05da8bcfacf16']
# br_ra_df[br_ra_df.trace_id == '1979578101591333f0a05da8bcfacf16']

# df_out_data[df_out_data.trace_id == 'a1ca07b662b4ce93755f69421cc86ba3']
# br_ra_df[br_ra_df.trace_id == 'a1ca07b662b4ce93755f69421cc86ba3']

(409712, 2)

(428993, 2)

(2302173, 14)

,order_id,order_number,order_trace_id,trace_id,risk_trace_id,bj_hm_score,rentOnlineRiskLevelOverall,rentOnlineRiskLevelMultiRent,rentOnlineRiskLevelFulfill,rentOnlineDetailLevelOverall,my_zl_score,my_zl_score2,br_ys_score,br_ra_score
0,88929,A202301090005022,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,567.0,NaN,NaN,NaN,NaN,1.419,NaN,NaN,813.0


## （5-3-8-1）自有模型-ascore2311

In [99]:
# （5-3-8-1）自有模型-ascore2311 
sql1='''-- 风控-自有模型-ascore2311 
SELECT * 
from db_credit.risk_private_model
;
'''

mx_2311_df =query(sql1)
mx_2311_df.shape

mx_2311_df['model_score_2311'] = pd.to_numeric(mx_2311_df['score'], errors='coerce')

mx_2311_df = mx_2311_df[['trace_id', 'model_score_2311']]
mx_2311_df[:1]

(284982, 7)

,trace_id,model_score_2311
0,12f8bdc71e41d304140df3a262cc4951,19.514414


## （5-3-8-2）自有模型-2401

In [100]:
# （5-3-8-2）自有模型-2401
sql1='''-- 风控-自有模型-2401
SELECT * 
from db_credit.risk_private_model_2401
;
'''

mx_2401_df =query(sql1)
mx_2401_df.shape

mx_2401_df['model_score_2401'] = pd.to_numeric(mx_2401_df['score'], errors='coerce')
mx_2401_df = mx_2401_df[['trace_id', 'model_score_2401']]
mx_2401_df[:1]

(262384, 7)

,trace_id,model_score_2401
0,ba170e63115f61a35da1ea8e662eda36,16.362452


## （5-3-8-3）自有模型-yl

In [101]:
# （5-3-8-3）自有模型-yl 
sql1='''-- 风控-自有模型-yl
SELECT * 
from db_credit.risk_private_model_yl
;
'''

mx_yl_df =query(sql1)
mx_yl_df.shape

mx_yl_df['model_score_yl'] = pd.to_numeric(mx_yl_df['score'], errors='coerce')
mx_yl_df = mx_yl_df[['trace_id', 'model_score_yl']]
mx_yl_df[:1]

(284901, 7)

,trace_id,model_score_yl
0,12f8bdc71e41d304140df3a262cc4951,16.430973


## （5-3-8-4）自有模型-2501

In [102]:
# （5-3-8-4）自有模型-2501
sql1='''-- 风控-自有模型-2501
SELECT * 
from db_credit.risk_private_model_2501_base
;
'''

mx_2501_df =query(sql1)
mx_2501_df.shape

mx_2501_df['model_score_2501'] = pd.to_numeric(mx_2501_df['score'], errors='coerce')
mx_2501_df = mx_2501_df[['trace_id', 'model_score_2501']]
mx_2501_df[:1]

# 线下调用
# 自有模型-2501：250121上线，上线后2个星期试运行，20250210增量运行，20250218增量运行改为并行，随机切流30%
# 20250121-20250205左右的数据不要用，入模有个变量类型有误，导致模型评分有差异
# 监控自2025.2.18起并行的样本

(110479, 7)

,trace_id,model_score_2501
0,b2358986620820d3e7b53201311b7eae,24.780494


## （5-3-8-5）模型关联

In [103]:
# （5-3-8-5）模型关联
df_model = mx_2311_df.merge(mx_2401_df, on = 'trace_id', how='outer')
df_model.shape

df_model = df_model.merge(mx_yl_df, on = 'trace_id', how='outer')
df_model.shape

df_model = df_model.merge(mx_2501_df, on = 'trace_id', how='outer')
df_model.shape

# df_model.tail()
df_model = df_model.merge(risk_df_model, on='trace_id', how='left')
df_model.shape

# A202407222000361 1277221 ee3e7156005b1e040f7ca4a85455a796，在 df_model，不在 trace_df，往前回溯的时候少量数据侧漏
df_out_data = df_out_data.merge(df_model, left_on='order_trace_id', right_on='trace_id', how='left')
# df_out_data = trace_df.merge(df_model, left_on='order_trace_id', right_on='trace_id', how='left')
# df_out_data = trace_df.merge(df_model, left_on='order_trace_id', right_on='trace_id', how='outer')
# 185431
# df_out_data = trace_df.merge(df_model, left_on='order_trace_id', right_on='trace_id', how='inner')
df_out_data.shape

df_out_data.drop('trace_id_y', axis=1, inplace=True)
df_out_data.rename(columns={'trace_id_x': 'trace_id'}, inplace=True)
df_out_data.shape
df_out_data[:2]

# risk_trace_id 可能有空， 但 order_trace_id 较全面
# df_out_data[df_out_data.order_trace_id == df_out_data.risk_trace_id].shape
# df_out_data[(df_out_data.order_trace_id != df_out_data.risk_trace_id)].shape
# df_out_data[(df_out_data.order_trace_id != df_out_data.risk_trace_id) & (df_out_data.risk_trace_id.notnull())].shape
# df_out_data[(df_out_data.order_trace_id != df_out_data.risk_trace_id) & (df_out_data.order_trace_id.notnull())].shape

# 下面是另外一种关联模式，结果一样的，只是结果表的trace_id为 trace_df 的 order_trace_id
# df_model = mx_2311_df[['trace_id', 'model_score_2311']].merge(mx_2401_df[['trace_id', 'model_score_2401']], on = 'trace_id', how='outer')
# df_model.shape

# df_model = df_model.merge(mx_yl_df[['trace_id', 'model_score_yl']], on = 'trace_id', how='outer')
# df_model.shape

# # df_model = df_model.merge(risk_df, on='', how='inner')
# df_model = df_model.merge(risk_df_model, on='trace_id', how='left')
# df_model.shape
# df_model[:2]

# temp = df_out_data.merge(df_model, left_on = 'order_trace_id', right_on = 'trace_id', how = 'inner')
# temp = df_out_data.merge(df_model, on = 'trace_id', how = 'inner') -- 0
# temp.shape
# temp[:5]
# temp[temp.model_score_2311_x !=temp.model_score_2311_y].shape
# temp[(temp.model_score_yl_x !=temp.model_score_yl_y)&(temp.model_score_yl_x.notnull())].shape
# temp[(temp.model_score_2401_x !=temp.model_score_2401_y)&(temp.model_score_2401_x.notnull())].shape

# trace_df[trace_df.order_trace_id == '63d59ed51cd8ebe9bcdf88974f293b59']
# trace_df[trace_df.trace_id == '63d59ed51cd8ebe9bcdf88974f293b59']
# risk_df_model[risk_df_model.trace_id == '63d59ed51cd8ebe9bcdf88974f293b59']

# df_out_data[df_out_data.trace_id == '63d59ed51cd8ebe9bcdf88974f293b59']
# df_out_data[df_out_data.order_trace_id == '63d59ed51cd8ebe9bcdf88974f293b59']

# df_model[df_model.trace_id == '63d59ed51cd8ebe9bcdf88974f293b59']

del mx_2311_df
del mx_2401_df
del mx_yl_df
del df_model

(284984, 3)

(284986, 4)

(284988, 5)

(284988, 6)

(2302173, 20)

(2302173, 19)

,order_id,order_number,order_trace_id,trace_id,risk_trace_id,bj_hm_score,rentOnlineRiskLevelOverall,rentOnlineRiskLevelMultiRent,rentOnlineRiskLevelFulfill,rentOnlineDetailLevelOverall,my_zl_score,my_zl_score2,br_ys_score,br_ra_score,model_score_2311,model_score_2401,model_score_yl,model_score_2501,is_model
0,88929,A202301090005022,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,567.0,NaN,NaN,NaN,NaN,1.419,NaN,NaN,813.0,NaN,NaN,NaN,NaN,NaN
1,88931,A202301090006044,198ce32e8b7d8ca6d1ad1f9f3a9d2177,198ce32e8b7d8ca6d1ad1f9f3a9d2177,198ce32e8b7d8ca6d1ad1f9f3a9d2177,627.0,NaN,NaN,NaN,NaN,23.133,NaN,NaN,752.0,NaN,NaN,NaN,NaN,NaN


## （5-3-9）冰鉴-青云分

In [104]:
# （5-3-9）冰鉴-青云分
sql1='''-- 风控-冰鉴-青云分
SELECT * 
from db_credit.risk_icekredit_qy
;
'''

bj_qy_df =query(sql1)
bj_qy_df.shape

bj_qy_df['bj_qy_score'] = bj_qy_df['rs'].apply(lambda x: json.loads(x).get('result', {}).get('qing_yun22', {}).get('bj_score', None))  
bj_qy_df['bj_qy_score'] = bj_qy_df['bj_qy_score'].astype(int)
bj_qy_df = bj_qy_df[['trace_id', 'bj_qy_score']]


df_out_data = df_out_data.merge(bj_qy_df, on = 'trace_id', how = 'left')
df_out_data.shape
df_out_data[:1]

del bj_qy_df 

(355845, 5)

(2302173, 20)

,order_id,order_number,order_trace_id,trace_id,risk_trace_id,bj_hm_score,rentOnlineRiskLevelOverall,rentOnlineRiskLevelMultiRent,rentOnlineRiskLevelFulfill,rentOnlineDetailLevelOverall,my_zl_score,my_zl_score2,br_ys_score,br_ra_score,model_score_2311,model_score_2401,model_score_yl,model_score_2501,is_model,bj_qy_score
0,88929,A202301090005022,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,567.0,NaN,NaN,NaN,NaN,1.419,NaN,NaN,813.0,NaN,NaN,NaN,NaN,NaN,636.0


## （5-3-10）评分等级

In [105]:
# （5-3-10）评分等级
sql1='''
SELECT * 
from db_credit.risk
;
'''

risk_df =query(sql1)
risk_df.shape

df_out_data = df_out_data.merge(risk_df[['trace_id','grade']], left_on='order_trace_id', right_on='trace_id', how='left')
df_out_data.drop('trace_id_y', axis=1, inplace=True)
df_out_data.rename(columns={'trace_id_x': 'trace_id'}, inplace=True)
df_out_data.shape

df_out_data.shape
df_out_data[:1]

del risk_df

(364121, 11)

(2302173, 21)

(2302173, 21)

,order_id,order_number,order_trace_id,trace_id,risk_trace_id,bj_hm_score,rentOnlineRiskLevelOverall,rentOnlineRiskLevelMultiRent,rentOnlineRiskLevelFulfill,rentOnlineDetailLevelOverall,my_zl_score,my_zl_score2,br_ys_score,br_ra_score,model_score_2311,model_score_2401,model_score_yl,model_score_2501,is_model,bj_qy_score,grade
0,88929,A202301090005022,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,567.0,NaN,NaN,NaN,NaN,1.419,NaN,NaN,813.0,NaN,NaN,NaN,NaN,NaN,636.0,A


## （5-3-11）全景雷达-新颜

In [106]:
# （5-3-11）全景雷达-新颜
# 76个变量
# （1）取数
sql1='''-- 风控-新颜-全景雷达
SELECT * 
from db_credit.risk_xinyan_qjld
;
'''

xy_qj_df =query(sql1)
xy_qj_df.shape
xy_qj_df[:1]

# xy_qj_df.time.max()
# Timestamp('2024-09-21 11:53:09')

# （2）新颜数据处理
def expand_json_columns(df, json_col,prefix='xy_qj_',sep='.'):
    # 只对非空的 JSON 列进行解析
    df['result_parsed'] = df[json_col].apply(lambda x:json.loads(x) if pd.notna(x) else None )

    # 对解析出来的 JSON 字段进行展开，并添加前缀
    df_json_expanded = pd.json_normalize(df['result_parsed'].dropna(),sep=sep)

    # 保留最内层字段名（去除层级前缀）
    df_json_expanded.columns = [col.split(sep)[-1] for col in df_json_expanded.columns]

    # 为所有字段名添加前缀
    df_json_expanded = df_json_expanded.add_prefix(prefix)

    # 将展开的字段拼接到原始 DataFrame 中
    df_final = pd.concat([df,df_json_expanded], axis=1)

     # 删除中间生成的列
    df_final = df_final.drop(columns='result_parsed')
    return  df_final

xy_qj_df = expand_json_columns(xy_qj_df,'rs','xy_qj_')

# xy_obj_cols = ['xy_qj_A22160007','xy_qj_B22170007','xy_qj_B22170008','xy_qj_B22170009','xy_qj_B22170010','xy_qj_B22170011',
# 'xy_qj_B22170031','xy_qj_B22170032','xy_qj_B22170033','xy_qj_B22170034',  #--百分比
# 'xy_qj_B22170040','xy_qj_B22170041','xy_qj_B22170042','xy_qj_B22170043','xy_qj_B22170044','xy_qj_B22170050','xy_qj_B22170054']

# xy_qj_B22170034  去除百分比后  转为float  
xy_qj_df['xy_qj_B22170034'] = pd.to_numeric(xy_qj_df['xy_qj_B22170034'].str.replace('%', ''), errors='coerce') / 100
# xy_qj_df['xy_qj_B22170034'].value_counts()

cols = ['xy_qj_A22160001','xy_qj_A22160002', 'xy_qj_A22160003', 'xy_qj_A22160004',
       'xy_qj_A22160005', 'xy_qj_A22160006', 
       'xy_qj_A22160008', 'xy_qj_A22160009', 'xy_qj_A22160010','xy_qj_C22180001', 'xy_qj_C22180002', 'xy_qj_C22180003',
       'xy_qj_C22180004', 'xy_qj_C22180005', 'xy_qj_C22180006','xy_qj_C22180007', 'xy_qj_C22180008', 'xy_qj_C22180009',
       'xy_qj_C22180010', 'xy_qj_C22180011', 'xy_qj_C22180012','xy_qj_B22170001', 'xy_qj_B22170002', 'xy_qj_B22170003',
       'xy_qj_B22170004', 'xy_qj_B22170005', 'xy_qj_B22170006',
        'xy_qj_B22170012',
       'xy_qj_B22170013', 'xy_qj_B22170014', 'xy_qj_B22170015','xy_qj_B22170016', 'xy_qj_B22170017', 'xy_qj_B22170018',
       'xy_qj_B22170019', 'xy_qj_B22170020', 'xy_qj_B22170021','xy_qj_B22170022', 'xy_qj_B22170023', 'xy_qj_B22170024',
       'xy_qj_B22170025', 'xy_qj_B22170026', 'xy_qj_B22170027','xy_qj_B22170028', 'xy_qj_B22170029', 'xy_qj_B22170030',
        'xy_qj_B22170035', 'xy_qj_B22170036',
       'xy_qj_B22170037', 'xy_qj_B22170038', 'xy_qj_B22170039',
        'xy_qj_B22170045',
       'xy_qj_B22170046', 'xy_qj_B22170047', 'xy_qj_B22170048',
       'xy_qj_B22170049',  'xy_qj_B22170051',
       'xy_qj_B22170052', 'xy_qj_B22170053']

for col in cols:
    xy_qj_df[col] = pd.to_numeric(xy_qj_df[col], errors='coerce')

xy_qj_df  = xy_qj_df.drop(columns=['id', 'params', 'rs', 'time', 'xy_qj_code', 'xy_qj_message', 'xy_qj_requestId',
       'xy_qj_requestTime', 'xy_qj_behavior_report_detail','xy_qj_current_report_detail'])   # ,inplace=True

xy_qj_df.shape
xy_qj_df[:1]

# # （3）关联
# df_out_data = df_out_data.merge(xy_qj_df, on='trace_id', how='left')
# df_out_data.shape
# df_out_data[:1]

(68236, 5)

,id,trace_id,params,rs,time
0,4,1d321ebae2a913a47c3e3c75ab300500,"{""idNo"": ""330501198903132044"", ""name"": ""王玲"", ""...","{""code"": ""success"", ""data"": {""apply_report_det...",2023-11-03 13:43:53


(68236, 77)

,trace_id,xy_qj_A22160001,xy_qj_A22160002,xy_qj_A22160003,xy_qj_A22160004,xy_qj_A22160005,xy_qj_A22160006,xy_qj_A22160007,xy_qj_A22160008,xy_qj_A22160009,xy_qj_A22160010,xy_qj_C22180001,xy_qj_C22180002,xy_qj_C22180003,xy_qj_C22180004,xy_qj_C22180005,xy_qj_C22180006,xy_qj_C22180007,xy_qj_C22180008,xy_qj_C22180009,xy_qj_C22180010,xy_qj_C22180011,xy_qj_C22180012,xy_qj_B22170001,xy_qj_B22170002,xy_qj_B22170003,xy_qj_B22170004,xy_qj_B22170005,xy_qj_B22170006,xy_qj_B22170007,xy_qj_B22170008,xy_qj_B22170009,xy_qj_B22170010,xy_qj_B22170011,xy_qj_B22170012,xy_qj_B22170013,xy_qj_B22170014,xy_qj_B22170015,xy_qj_B22170016,xy_qj_B22170017,xy_qj_B22170018,xy_qj_B22170019,xy_qj_B22170020,xy_qj_B22170021,xy_qj_B22170022,xy_qj_B22170023,xy_qj_B22170024,xy_qj_B22170025,xy_qj_B22170026,xy_qj_B22170027,xy_qj_B22170028,xy_qj_B22170029,xy_qj_B22170030,xy_qj_B22170031,xy_qj_B22170032,xy_qj_B22170033,xy_qj_B22170034,xy_qj_B22170035,xy_qj_B22170036,xy_qj_B22170037,xy_qj_B22170038,xy_qj_B22170039,xy_qj_B22170040,xy_qj_B22170041,xy_qj_B22170042,xy_qj_B22170043,xy_qj_B22170044,xy_qj_B22170045,xy_qj_B22170046,xy_qj_B22170047,xy_qj_B22170048,xy_qj_B22170049,xy_qj_B22170050,xy_qj_B22170051,xy_qj_B22170052,xy_qj_B22170053,xy_qj_B22170054
0,1d321ebae2a913a47c3e3c75ab300500,542.0,73.0,118.0,88.0,8.0,257.0,2023-11,22.0,60.0,101.0,0.0,0.0,0.0,0.0,0.0,0.0,48.0,91.0,57600.0,17075.0,25500.0,84.0,524.0,4.0,13.0,34.0,70.0,131.0,"[50000,+)","[50000,+)","[50000,+)","[50000,+)","[50000,+)",9.0,21.0,20.0,20.0,4.0,10.0,23.0,36.0,48.0,36.0,48.0,0.0,0.0,2.0,2.0,2.0,0.0,0.0,0.0,"[500,1000)","[500,1000)","[500,1000)",0.97,2.0,9.0,18.0,29.0,54.0,"[20000,30000)","[50000,+)","[50000,+)","[50000,+)","[50000,+)",12.0,38.0,94.0,245.0,416.0,"(0,7]",84.0,127.0,719.0,2023-11


## （5-3-12）全景雷达-微聚未来

In [107]:
# （5-3-12）全景雷达-微聚未来
# 76个变量
# （1）
sql1='''-- 风控-微聚未来-全景雷达
SELECT * 
from db_credit.risk_weisuper_qjld
;
'''

xy_risk_weisuper_df =query(sql1)
xy_risk_weisuper_df.shape
xy_risk_weisuper_df[:1]
# xy_risk_weisuper_df.time.min()
# Timestamp('2024-09-21 12:11:08')

# # （2）微聚未来数据处理
def expand_json_columns(df, json_col,prefix='xy_qj_',sep='.'):
    # 只对非空的 JSON 列进行解析
    df['result_parsed'] = df[json_col].apply(lambda x:json.loads(x) if pd.notna(x) else None )

    # 对解析出来的 JSON 字段进行展开，并添加前缀
    df_json_expanded = pd.json_normalize(df['result_parsed'].dropna(),sep=sep)

    # 保留最内层字段名（去除层级前缀）
    df_json_expanded.columns = [col.split(sep)[-1] for col in df_json_expanded.columns]

    # 为所有字段名添加前缀
    df_json_expanded = df_json_expanded.add_prefix(prefix)

    # 将展开的字段拼接到原始 DataFrame 中
    df_final = pd.concat([df,df_json_expanded], axis=1)

     # 删除中间生成的列
    df_final = df_final.drop(columns='result_parsed')
    return  df_final

xy_risk_weisuper_df = expand_json_columns(xy_risk_weisuper_df,'rs','xy_qj_')

# # xy_obj_cols = ['xy_qj_A22160007','xy_qj_B22170007','xy_qj_B22170008','xy_qj_B22170009','xy_qj_B22170010','xy_qj_B22170011',
# # 'xy_qj_B22170031','xy_qj_B22170032','xy_qj_B22170033','xy_qj_B22170034',  #--百分比
# # 'xy_qj_B22170040','xy_qj_B22170041','xy_qj_B22170042','xy_qj_B22170043','xy_qj_B22170044','xy_qj_B22170050','xy_qj_B22170054']

# # xy_qj_B22170034  去除百分比后  转为float  
xy_risk_weisuper_df['xy_qj_B22170034'] = pd.to_numeric(xy_risk_weisuper_df['xy_qj_B22170034'].str.replace('%', ''), errors='coerce') / 100
# # xy_risk_weisuper_df['xy_qj_B22170034'].value_counts()

cols = ['xy_qj_A22160001','xy_qj_A22160002', 'xy_qj_A22160003', 'xy_qj_A22160004',
       'xy_qj_A22160005', 'xy_qj_A22160006', 
       'xy_qj_A22160008', 'xy_qj_A22160009', 'xy_qj_A22160010','xy_qj_C22180001', 'xy_qj_C22180002', 'xy_qj_C22180003',
       'xy_qj_C22180004', 'xy_qj_C22180005', 'xy_qj_C22180006','xy_qj_C22180007', 'xy_qj_C22180008', 'xy_qj_C22180009',
       'xy_qj_C22180010', 'xy_qj_C22180011', 'xy_qj_C22180012','xy_qj_B22170001', 'xy_qj_B22170002', 'xy_qj_B22170003',
       'xy_qj_B22170004', 'xy_qj_B22170005', 'xy_qj_B22170006',
        'xy_qj_B22170012',
       'xy_qj_B22170013', 'xy_qj_B22170014', 'xy_qj_B22170015','xy_qj_B22170016', 'xy_qj_B22170017', 'xy_qj_B22170018',
       'xy_qj_B22170019', 'xy_qj_B22170020', 'xy_qj_B22170021','xy_qj_B22170022', 'xy_qj_B22170023', 'xy_qj_B22170024',
       'xy_qj_B22170025', 'xy_qj_B22170026', 'xy_qj_B22170027','xy_qj_B22170028', 'xy_qj_B22170029', 'xy_qj_B22170030',
        'xy_qj_B22170035', 'xy_qj_B22170036',
       'xy_qj_B22170037', 'xy_qj_B22170038', 'xy_qj_B22170039',
        'xy_qj_B22170045',
       'xy_qj_B22170046', 'xy_qj_B22170047', 'xy_qj_B22170048',
       'xy_qj_B22170049',  'xy_qj_B22170051',
       'xy_qj_B22170052', 'xy_qj_B22170053']

for col in cols:
    xy_risk_weisuper_df[col] = pd.to_numeric(xy_risk_weisuper_df[col], errors='coerce')

# xy_risk_weisuper_df  = xy_risk_weisuper_df.drop(columns=['id', 'params', 'rs', 'time', 'xy_qj_code', 'xy_qj_message', 'xy_qj_requestId',
#        'xy_qj_requestTime', 'xy_qj_behavior_report_detail','xy_qj_current_report_detail'])   # ,inplace=True

xy_risk_weisuper_df  = xy_risk_weisuper_df.drop(columns=['id', 'params', 'rs', 'time', 'xy_qj_returnMsg', 'xy_qj_behavior_report_detail','xy_qj_current_report_detail'
                                                         , 'xy_qj_returnCode', 'xy_qj_apply_report_detail'])   # ,inplace=True
xy_risk_weisuper_df.shape
xy_risk_weisuper_df[:1]

# # （3）关联
xy_risk_qjld_df = pd.concat([xy_qj_df, xy_risk_weisuper_df])
xy_risk_qjld_df.shape
xy_risk_qjld_df = xy_risk_qjld_df.drop_duplicates(subset = ['trace_id'])
xy_risk_qjld_df.shape

df_out_data = df_out_data.merge(xy_risk_qjld_df, on='trace_id', how='left')
df_out_data.shape
df_out_data[:1]

# del xy_risk_qjld_df
del xy_qj_df
del xy_risk_weisuper_df

(147639, 5)

,id,trace_id,params,rs,time
0,1,17d29544198be5e82a0ca291a4a6203c,"{""idNo"": ""864a128343f3f0998c647a89e2af8393"", ""...","{""data"": {""apply_report_detail"": {""A22160001"":...",2024-09-21 12:11:08


(147639, 77)

,trace_id,xy_qj_A22160001,xy_qj_A22160002,xy_qj_A22160003,xy_qj_A22160004,xy_qj_A22160005,xy_qj_A22160006,xy_qj_A22160007,xy_qj_A22160008,xy_qj_A22160009,xy_qj_A22160010,xy_qj_C22180001,xy_qj_C22180002,xy_qj_C22180003,xy_qj_C22180004,xy_qj_C22180005,xy_qj_C22180006,xy_qj_C22180007,xy_qj_C22180008,xy_qj_C22180009,xy_qj_C22180010,xy_qj_C22180011,xy_qj_C22180012,xy_qj_B22170001,xy_qj_B22170002,xy_qj_B22170003,xy_qj_B22170004,xy_qj_B22170005,xy_qj_B22170006,xy_qj_B22170007,xy_qj_B22170008,xy_qj_B22170009,xy_qj_B22170010,xy_qj_B22170011,xy_qj_B22170012,xy_qj_B22170013,xy_qj_B22170014,xy_qj_B22170015,xy_qj_B22170016,xy_qj_B22170017,xy_qj_B22170018,xy_qj_B22170019,xy_qj_B22170020,xy_qj_B22170021,xy_qj_B22170022,xy_qj_B22170023,xy_qj_B22170024,xy_qj_B22170025,xy_qj_B22170026,xy_qj_B22170027,xy_qj_B22170028,xy_qj_B22170029,xy_qj_B22170030,xy_qj_B22170031,xy_qj_B22170032,xy_qj_B22170033,xy_qj_B22170034,xy_qj_B22170035,xy_qj_B22170036,xy_qj_B22170037,xy_qj_B22170038,xy_qj_B22170039,xy_qj_B22170040,xy_qj_B22170041,xy_qj_B22170042,xy_qj_B22170043,xy_qj_B22170044,xy_qj_B22170045,xy_qj_B22170046,xy_qj_B22170047,xy_qj_B22170048,xy_qj_B22170049,xy_qj_B22170050,xy_qj_B22170051,xy_qj_B22170052,xy_qj_B22170053,xy_qj_B22170054
0,17d29544198be5e82a0ca291a4a6203c,535.0,76.0,58.0,31.0,9.0,155.0,2024-09,10.0,47.0,67.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,14.0,11400.0,3514.0,2500.0,85.0,547.0,1.0,5.0,8.0,10.0,13.0,"(0,500)","[3000,5000)","[5000,10000)","[5000,10000)","[10000,20000)",7.0,3.0,0.0,0.0,1.0,5.0,7.0,9.0,10.0,9.0,10.0,0.0,0.0,2.0,3.0,3.0,0.0,0.0,0.0,"(0,500)","[2000,3000)","[2000,3000)",0.77,1.0,4.0,16.0,22.0,22.0,"[1000,2000)","[3000,5000)","[3000,5000)","[5000,10000)","[5000,10000)",11.0,24.0,35.0,63.0,68.0,"(0,7]",84.0,12.0,665.0,2024-09


(215875, 77)

(215782, 77)

(2302173, 97)

,order_id,order_number,order_trace_id,trace_id,risk_trace_id,bj_hm_score,rentOnlineRiskLevelOverall,rentOnlineRiskLevelMultiRent,rentOnlineRiskLevelFulfill,rentOnlineDetailLevelOverall,my_zl_score,my_zl_score2,br_ys_score,br_ra_score,model_score_2311,model_score_2401,model_score_yl,model_score_2501,is_model,bj_qy_score,grade,xy_qj_A22160001,xy_qj_A22160002,xy_qj_A22160003,xy_qj_A22160004,xy_qj_A22160005,xy_qj_A22160006,xy_qj_A22160007,xy_qj_A22160008,xy_qj_A22160009,xy_qj_A22160010,xy_qj_C22180001,xy_qj_C22180002,xy_qj_C22180003,xy_qj_C22180004,xy_qj_C22180005,xy_qj_C22180006,xy_qj_C22180007,xy_qj_C22180008,xy_qj_C22180009,xy_qj_C22180010,xy_qj_C22180011,xy_qj_C22180012,xy_qj_B22170001,xy_qj_B22170002,xy_qj_B22170003,xy_qj_B22170004,xy_qj_B22170005,xy_qj_B22170006,xy_qj_B22170007,xy_qj_B22170008,xy_qj_B22170009,xy_qj_B22170010,xy_qj_B22170011,xy_qj_B22170012,xy_qj_B22170013,xy_qj_B22170014,xy_qj_B22170015,xy_qj_B22170016,xy_qj_B22170017,xy_qj_B22170018,xy_qj_B22170019,xy_qj_B22170020,xy_qj_B22170021,xy_qj_B22170022,xy_qj_B22170023,xy_qj_B22170024,xy_qj_B22170025,xy_qj_B22170026,xy_qj_B22170027,xy_qj_B22170028,xy_qj_B22170029,xy_qj_B22170030,xy_qj_B22170031,xy_qj_B22170032,xy_qj_B22170033,xy_qj_B22170034,xy_qj_B22170035,xy_qj_B22170036,xy_qj_B22170037,xy_qj_B22170038,xy_qj_B22170039,xy_qj_B22170040,xy_qj_B22170041,xy_qj_B22170042,xy_qj_B22170043,xy_qj_B22170044,xy_qj_B22170045,xy_qj_B22170046,xy_qj_B22170047,xy_qj_B22170048,xy_qj_B22170049,xy_qj_B22170050,xy_qj_B22170051,xy_qj_B22170052,xy_qj_B22170053,xy_qj_B22170054
0,88929,A202301090005022,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,567.0,NaN,NaN,NaN,NaN,1.419,NaN,NaN,813.0,NaN,NaN,NaN,NaN,NaN,636.0,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## （5-3-13）融360

In [108]:
# （5-3-13）融360
# 线下测试+线上调取

# 1-本次测试有欺诈和信用产品
# （1）2023.7-2024.9 有贷后表现的订单
# path_360 = 'C:/Users/Administrator/Desktop/mine/7_三方数据产品/2_360/2_20241218/1.样本发送/'
path_360 = path
data_360_sample = pd.read_excel(path_360 + 'ceshi_sample_20241218.xlsx')
data_360_sample = data_360_sample[['order_number','idcard_md5','mobile_md5','下单日期']]
data_360_sample.shape

# path_360 = 'C:/Users/Administrator/Desktop/mine/7_三方数据产品/2_360/2_20241218/2.数据返回/维客壹佰/'
data_360_1 = pd.read_csv(path_360 + '维客壹佰-201048占信分臻藏版-支付风险V3.0-100%.csv')
# data_360_2 = pd.read_csv(path_360 + '维客壹佰-201053综合行为评估报告V2.4-100%.csv')
# data_360_3 = pd.read_csv(path_360 + '维客壹佰-201054综合行为评估报告V2.2-100%.csv', encoding = 'gbk')
# data_360_4 = pd.read_csv(path_360 + '维客壹佰-201055综合行为评估报告V2.3-100%.csv', encoding = 'gbk')
data_360_5 = pd.read_csv(path_360 + '维客壹佰-202098占信分臻藏版-行为风险V3.0-99.83%.csv', encoding = 'gbk')
data_360_6 = pd.read_csv(path_360 + '维客壹佰-202100占信分臻藏版-支付风险V3.0-100%.csv', encoding = 'gbk')
# data_360_7 = pd.read_csv(path_360 + '维客壹佰-202123占信分标准版-消费风险V3.0-99.97%.csv')


data_360_1 = data_360_1.rename(columns = {'score':'score_201048'})
data_360_1 = data_360_1[['idcard_md5','mobile_md5','下单日期','score_201048']]

data_360_5 = data_360_5.rename(columns = {'score':'score_202098'})
data_360_5 = data_360_5[['idcard_md5','mobile_md5','下单日期','score_202098']]

data_360_6 = data_360_6.rename(columns = {'score':'score_202100'})
data_360_6 = data_360_6[['idcard_md5','mobile_md5','下单日期','score_202100']]

# data_360_1.shape
# data_360_5.shape
# data_360_6.shape

data_360_1.drop_duplicates(subset = ['idcard_md5','mobile_md5','下单日期'], inplace=True)
data_360_5.drop_duplicates(subset = ['idcard_md5','mobile_md5','下单日期'], inplace=True)
data_360_6.drop_duplicates(subset = ['idcard_md5','mobile_md5','下单日期'], inplace=True)

data_360_score =  data_360_1.merge(data_360_5, on = ['idcard_md5','mobile_md5','下单日期'], how = 'outer')
data_360_score =  data_360_score.merge(data_360_6, on = ['idcard_md5','mobile_md5','下单日期'], how = 'outer')
# data_360_score.shape

data_360_score =  data_360_sample.merge(data_360_score, on = ['idcard_md5','mobile_md5','下单日期'], how = 'left')
data_360_score = data_360_score.fillna(-999)
data_360_score = data_360_score.drop(columns= ['idcard_md5', 'mobile_md5','下单日期'])
data_360_score.shape
# data_360_score[:2]

# 2-线下回溯：# 本次回溯：2024.10-2025.1 有贷后表现的订单，2025.2.1-2025.2.25 进件免押（2025.2.25上线）
# path = 'C:/Users/Administrator/Desktop/mine/7_三方数据产品/2_360/3_20250320/1.数据发送/'
data_360_sample_1 = pd.read_excel(path + 'ceshi_sample_20250320.xlsx')
data_360_sample_1 = data_360_sample_1[['order_number','idcard_md5','mobile_md5','下单日期']]
data_360_sample_1.shape
# data_360_sample_1[:1]

# path = 'C:/Users/Administrator/Desktop/mine/7_三方数据产品/2_360/3_20250320/2.数据返回/维客壹佰/'
data_360_1 = pd.read_excel(path + '维客壹佰-201048占控分-支付风险评估-100%.xlsx')
data_360_5 = pd.read_excel(path + '维客壹佰-202098占信分臻藏版-行为风险V3.0-99.99%.xlsx')
data_360_6 = pd.read_excel(path + '维客壹佰-202100占信分臻藏版-支付风险V3.0-100%.xlsx')

data_360_1 = data_360_1.rename(columns = {'score':'score_201048'})
data_360_1 = data_360_1[['idcard_md5','mobile_md5','下单日期','score_201048']]

data_360_5 = data_360_5.rename(columns = {'score':'score_202098'})
data_360_5 = data_360_5[['idcard_md5','mobile_md5','下单日期','score_202098']]

data_360_6 = data_360_6.rename(columns = {'score':'score_202100'})
data_360_6 = data_360_6[['idcard_md5','mobile_md5','下单日期','score_202100']]

data_360_1.drop_duplicates(subset = ['idcard_md5','mobile_md5','下单日期'], inplace=True)
data_360_5.drop_duplicates(subset = ['idcard_md5','mobile_md5','下单日期'], inplace=True)
data_360_6.drop_duplicates(subset = ['idcard_md5','mobile_md5','下单日期'], inplace=True)

data_360_score_1 =  data_360_1.merge(data_360_5, on = ['idcard_md5','mobile_md5','下单日期'], how = 'outer')
data_360_score_1 =  data_360_score_1.merge(data_360_6, on = ['idcard_md5','mobile_md5','下单日期'], how = 'outer')

data_360_score_1 =  data_360_sample_1.merge(data_360_score_1, on = ['idcard_md5','mobile_md5','下单日期'], how = 'left')
data_360_score_1 = data_360_score_1.fillna(-999)
data_360_score_1 = data_360_score_1.drop(columns= ['idcard_md5', 'mobile_md5','下单日期'])
data_360_score_1.shape
data_360_score_1[:1]

# 线下调用合并
data_360_score = pd.concat([data_360_score, data_360_score_1])
data_360_score.drop_duplicates(subset = ['order_number'], inplace=True)

data_360_score.shape

# 3-线上调用-2025.2.25上线
sql1='''
SELECT trace_id, score as score_201048
from db_credit.risk_zhanrong_201048
;
'''
df_201048 = query(sql1)
df_201048.shape

sql1='''
SELECT trace_id, score as score_202098
from db_credit.risk_zhanrong_202098
;
'''
df_202098 = query(sql1)
df_202098.shape

sql1='''
SELECT trace_id, score as score_202100
from db_credit.risk_zhanrong_202100
;
'''
df_202100 = query(sql1)
df_202100.shape

df_online_360 = df_201048.merge(df_202098, on = 'trace_id', how = 'outer')
df_online_360 = df_online_360.merge(df_202100, on = 'trace_id', how = 'outer')
df_online_360.shape
df_online_360[:1]

# 4-合并
# 部分order_number没有trace_id

df_out_data = df_out_data.merge(data_360_score, on='order_number', how='left')
df_out_data.shape

df_out_data = df_out_data.merge(df_online_360, on='trace_id', how='left')
df_out_data.shape

df_out_data['score_201048'] = np.where(pd.notnull(df_out_data.score_201048_y), df_out_data.score_201048_y, df_out_data.score_201048_x)
df_out_data['score_202098'] = np.where(pd.notnull(df_out_data.score_202098_y), df_out_data.score_202098_y, df_out_data.score_202098_x)
df_out_data['score_202100'] = np.where(pd.notnull(df_out_data.score_202100_y), df_out_data.score_202100_y, df_out_data.score_202100_x)

df_out_data = df_out_data.drop(columns = ['score_201048_x','score_201048_y','score_202098_x','score_202098_y','score_202100_x','score_202100_y'])
df_out_data.shape
df_out_data[:1]

del df_online_360
del df_202100
del df_202098
del df_201048
del data_360_score_1
del data_360_score
del data_360_sample_1
del data_360_sample
del data_360_1
del data_360_5
del data_360_6

(19669, 4)

(19669, 4)

(22444, 4)

(22444, 4)

,order_number,score_201048,score_202098,score_202100
0,A2024100100112314,595,687.0,651


...

,trace_id,score_201048,score_202098,score_202100
0,00001bdaba337a4f0174b5eeee2dcfad,516,436,486


(2302173, 100)

(2302173, 103)

(2302173, 100)

,order_id,order_number,order_trace_id,trace_id,risk_trace_id,bj_hm_score,rentOnlineRiskLevelOverall,rentOnlineRiskLevelMultiRent,rentOnlineRiskLevelFulfill,rentOnlineDetailLevelOverall,my_zl_score,my_zl_score2,br_ys_score,br_ra_score,model_score_2311,model_score_2401,model_score_yl,model_score_2501,is_model,bj_qy_score,grade,xy_qj_A22160001,xy_qj_A22160002,xy_qj_A22160003,xy_qj_A22160004,xy_qj_A22160005,xy_qj_A22160006,xy_qj_A22160007,xy_qj_A22160008,xy_qj_A22160009,xy_qj_A22160010,xy_qj_C22180001,xy_qj_C22180002,xy_qj_C22180003,xy_qj_C22180004,xy_qj_C22180005,xy_qj_C22180006,xy_qj_C22180007,xy_qj_C22180008,xy_qj_C22180009,xy_qj_C22180010,xy_qj_C22180011,xy_qj_C22180012,xy_qj_B22170001,xy_qj_B22170002,xy_qj_B22170003,xy_qj_B22170004,xy_qj_B22170005,xy_qj_B22170006,xy_qj_B22170007,xy_qj_B22170008,xy_qj_B22170009,xy_qj_B22170010,xy_qj_B22170011,xy_qj_B22170012,xy_qj_B22170013,xy_qj_B22170014,xy_qj_B22170015,xy_qj_B22170016,xy_qj_B22170017,xy_qj_B22170018,xy_qj_B22170019,xy_qj_B22170020,xy_qj_B22170021,xy_qj_B22170022,xy_qj_B22170023,xy_qj_B22170024,xy_qj_B22170025,xy_qj_B22170026,xy_qj_B22170027,xy_qj_B22170028,xy_qj_B22170029,xy_qj_B22170030,xy_qj_B22170031,xy_qj_B22170032,xy_qj_B22170033,xy_qj_B22170034,xy_qj_B22170035,xy_qj_B22170036,xy_qj_B22170037,xy_qj_B22170038,xy_qj_B22170039,xy_qj_B22170040,xy_qj_B22170041,xy_qj_B22170042,xy_qj_B22170043,xy_qj_B22170044,xy_qj_B22170045,xy_qj_B22170046,xy_qj_B22170047,xy_qj_B22170048,xy_qj_B22170049,xy_qj_B22170050,xy_qj_B22170051,xy_qj_B22170052,xy_qj_B22170053,xy_qj_B22170054,score_201048,score_202098,score_202100
0,88929,A202301090005022,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,567.0,NaN,NaN,NaN,NaN,1.419,NaN,NaN,813.0,NaN,NaN,NaN,NaN,NaN,636.0,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## （5-3-14）极信-银联

In [109]:
# （5-3-14）极信-银联
# 30 个变量
sql1='''-- 风控-极信-银联
SELECT * 
from db_credit.risk_jixin
;
'''

jx_yl_df =query(sql1)
jx_yl_df.shape

jx_yl_df['jx_yl_UPPC011'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPC011'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None) 
jx_yl_df['jx_yl_UPPC036'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPC036'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPC098'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPC098'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPC134'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPC134'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPC173'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPC173'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPC196'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPC196'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPD002'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPD002'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPD004'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPD004'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPD005'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPD005'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPD007'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPD007'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPD009'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPD009'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPD028'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPD028'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPD029'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPD029'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPD068'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPD068'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPD069'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPD069'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPD075'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPD075'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPD091'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPD091'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPD097'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPD097'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPD098'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPD098'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPD100'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPD100'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPE061'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPE061'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPE074'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPE074'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPE077'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPE077'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPE079'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPE079'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPE082'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPE082'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPE086'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPE086'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPE090'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPE090'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPG046'] = jx_yl_df['rs_decode'].apply(lambda 
                                                        x: json.loads(x)['data']['resultData'][0]['UPPG046'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPG050'] = jx_yl_df['rs_decode'].apply(lambda 
                                                        x: json.loads(x)['data']['resultData'][0]['UPPG050'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  
jx_yl_df['jx_yl_UPPG054'] = jx_yl_df['rs_decode'].apply(lambda x: 
                                                        json.loads(x)['data']['resultData'][0]['UPPG054'] 
                                                        if x and 'data' in json.loads(x) and 'resultData' in json.loads(x)['data'] and json.loads(x)['data']['resultData'] else None)  

jx_yl_df.drop(columns=['id',  'params', 'rs', 'rs_decode', 'time'],inplace = True)
cols = ['jx_yl_UPPC011', 'jx_yl_UPPC036', 'jx_yl_UPPC098',
       'jx_yl_UPPC134', 'jx_yl_UPPC173', 'jx_yl_UPPC196', 'jx_yl_UPPD002',
       'jx_yl_UPPD004', 'jx_yl_UPPD005', 'jx_yl_UPPD007', 'jx_yl_UPPD009',
       'jx_yl_UPPD028', 'jx_yl_UPPD029', 'jx_yl_UPPD068', 'jx_yl_UPPD069',
       'jx_yl_UPPD075', 'jx_yl_UPPD091', 'jx_yl_UPPD097', 'jx_yl_UPPD098',
       'jx_yl_UPPD100', 'jx_yl_UPPE061', 'jx_yl_UPPE074', 'jx_yl_UPPE077',
       'jx_yl_UPPE079', 'jx_yl_UPPE082', 'jx_yl_UPPE086', 'jx_yl_UPPE090',
       'jx_yl_UPPG046', 'jx_yl_UPPG050', 'jx_yl_UPPG054']
for col in cols:
    jx_yl_df[col] = pd.to_numeric(jx_yl_df[col].replace('"null"', np.nan), errors='coerce')


df_out_data = df_out_data.merge(jx_yl_df, on = 'trace_id', how = 'left')
df_out_data.shape
# df_out_data[:1]

# del jx_yl_df

(253006, 6)

(2302173, 130)

## （5-3-15）京享租
<!-- 已在大表取数 -->
<!-- jd_user_level_risk -->

## （5-4-1）芝麻数控-交互安全风控

In [110]:
# （5-4-1）芝麻数控-交互安全风控
# 目前使用到 risk_t0, risk_t1、risk_t3、risk_t7 没有使用
# zm_jh_t0_level 蚂蚁数控风险等级 
sql1='''-- 芝麻数控-交互安全风控
select
order_id, 
replace(case when JSON_VALID(risk_t0) THEN JSON_EXTRACT(risk_t0, '$.level') end,'"','') as zm_jh_t0_level 
from db_credit.risk_alipay_interactive_prod
;
'''

zm_jh_df=query(sql1)
zm_jh_df.shape

# 508 条无租后表现
# df_out_data = df_out_data.merge(zm_jh_df, on = 'order_id', how = 'outer')
df_out_data = df_out_data.merge(zm_jh_df, on = 'order_id', how = 'left')
df_out_data.shape
df_out_data[:1]

del zm_jh_df

(106324, 2)

(2302173, 131)

,order_id,order_number,order_trace_id,trace_id,risk_trace_id,bj_hm_score,rentOnlineRiskLevelOverall,rentOnlineRiskLevelMultiRent,rentOnlineRiskLevelFulfill,rentOnlineDetailLevelOverall,my_zl_score,my_zl_score2,br_ys_score,br_ra_score,model_score_2311,model_score_2401,model_score_yl,model_score_2501,is_model,bj_qy_score,grade,xy_qj_A22160001,xy_qj_A22160002,xy_qj_A22160003,xy_qj_A22160004,xy_qj_A22160005,xy_qj_A22160006,xy_qj_A22160007,xy_qj_A22160008,xy_qj_A22160009,xy_qj_A22160010,xy_qj_C22180001,xy_qj_C22180002,xy_qj_C22180003,xy_qj_C22180004,xy_qj_C22180005,xy_qj_C22180006,xy_qj_C22180007,xy_qj_C22180008,xy_qj_C22180009,xy_qj_C22180010,xy_qj_C22180011,xy_qj_C22180012,xy_qj_B22170001,xy_qj_B22170002,xy_qj_B22170003,xy_qj_B22170004,xy_qj_B22170005,xy_qj_B22170006,xy_qj_B22170007,xy_qj_B22170008,xy_qj_B22170009,xy_qj_B22170010,xy_qj_B22170011,xy_qj_B22170012,xy_qj_B22170013,xy_qj_B22170014,xy_qj_B22170015,xy_qj_B22170016,xy_qj_B22170017,xy_qj_B22170018,xy_qj_B22170019,xy_qj_B22170020,xy_qj_B22170021,xy_qj_B22170022,xy_qj_B22170023,xy_qj_B22170024,xy_qj_B22170025,xy_qj_B22170026,xy_qj_B22170027,xy_qj_B22170028,xy_qj_B22170029,xy_qj_B22170030,xy_qj_B22170031,xy_qj_B22170032,xy_qj_B22170033,xy_qj_B22170034,xy_qj_B22170035,xy_qj_B22170036,xy_qj_B22170037,xy_qj_B22170038,xy_qj_B22170039,xy_qj_B22170040,xy_qj_B22170041,xy_qj_B22170042,xy_qj_B22170043,xy_qj_B22170044,xy_qj_B22170045,xy_qj_B22170046,xy_qj_B22170047,xy_qj_B22170048,xy_qj_B22170049,xy_qj_B22170050,xy_qj_B22170051,xy_qj_B22170052,xy_qj_B22170053,xy_qj_B22170054,score_201048,score_202098,score_202100,jx_yl_UPPC011,jx_yl_UPPC036,jx_yl_UPPC098,jx_yl_UPPC134,jx_yl_UPPC173,jx_yl_UPPC196,jx_yl_UPPD002,jx_yl_UPPD004,jx_yl_UPPD005,jx_yl_UPPD007,jx_yl_UPPD009,jx_yl_UPPD028,jx_yl_UPPD029,jx_yl_UPPD068,jx_yl_UPPD069,jx_yl_UPPD075,jx_yl_UPPD091,jx_yl_UPPD097,jx_yl_UPPD098,jx_yl_UPPD100,jx_yl_UPPE061,jx_yl_UPPE074,jx_yl_UPPE077,jx_yl_UPPE079,jx_yl_UPPE082,jx_yl_UPPE086,jx_yl_UPPE090,jx_yl_UPPG046,jx_yl_UPPG050,jx_yl_UPPG054,zm_jh_t0_level
0,88929,A202301090005022,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,567.0,NaN,NaN,NaN,NaN,1.419,NaN,NaN,813.0,NaN,NaN,NaN,NaN,NaN,636.0,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## （5-4-2）极信-分数sfico 

In [111]:
# （5-4-2）极信-分数sfico 
sql1='''-- 风控-极信-分数sc32007   fico  
SELECT * 
from db_credit.risk_jixin_score_sc32007
;
'''

jx_fico_df =query(sql1)
jx_fico_df.shape

jx_fico_df['jx_fico_score'] = jx_fico_df['rs_decode'].apply(lambda x: json.loads(x).get('data', {}).get('resultData', {}).get('score', None))  
jx_fico_df['jx_fico_score'] = pd.to_numeric(jx_fico_df['jx_fico_score'].replace('"null"', np.nan), errors='coerce')
jx_fico_df = jx_fico_df[['trace_id', 'jx_fico_score']]


df_out_data = df_out_data.merge(jx_fico_df, on = 'trace_id', how = 'left')
df_out_data.shape
df_out_data[:1]

del jx_fico_df

(116334, 6)

(2302173, 132)

,order_id,order_number,order_trace_id,trace_id,risk_trace_id,bj_hm_score,rentOnlineRiskLevelOverall,rentOnlineRiskLevelMultiRent,rentOnlineRiskLevelFulfill,rentOnlineDetailLevelOverall,my_zl_score,my_zl_score2,br_ys_score,br_ra_score,model_score_2311,model_score_2401,model_score_yl,model_score_2501,is_model,bj_qy_score,grade,xy_qj_A22160001,xy_qj_A22160002,xy_qj_A22160003,xy_qj_A22160004,xy_qj_A22160005,xy_qj_A22160006,xy_qj_A22160007,xy_qj_A22160008,xy_qj_A22160009,xy_qj_A22160010,xy_qj_C22180001,xy_qj_C22180002,xy_qj_C22180003,xy_qj_C22180004,xy_qj_C22180005,xy_qj_C22180006,xy_qj_C22180007,xy_qj_C22180008,xy_qj_C22180009,xy_qj_C22180010,xy_qj_C22180011,xy_qj_C22180012,xy_qj_B22170001,xy_qj_B22170002,xy_qj_B22170003,xy_qj_B22170004,xy_qj_B22170005,xy_qj_B22170006,xy_qj_B22170007,xy_qj_B22170008,xy_qj_B22170009,xy_qj_B22170010,xy_qj_B22170011,xy_qj_B22170012,xy_qj_B22170013,xy_qj_B22170014,xy_qj_B22170015,xy_qj_B22170016,xy_qj_B22170017,xy_qj_B22170018,xy_qj_B22170019,xy_qj_B22170020,xy_qj_B22170021,xy_qj_B22170022,xy_qj_B22170023,xy_qj_B22170024,xy_qj_B22170025,xy_qj_B22170026,xy_qj_B22170027,xy_qj_B22170028,xy_qj_B22170029,xy_qj_B22170030,xy_qj_B22170031,xy_qj_B22170032,xy_qj_B22170033,xy_qj_B22170034,xy_qj_B22170035,xy_qj_B22170036,xy_qj_B22170037,xy_qj_B22170038,xy_qj_B22170039,xy_qj_B22170040,xy_qj_B22170041,xy_qj_B22170042,xy_qj_B22170043,xy_qj_B22170044,xy_qj_B22170045,xy_qj_B22170046,xy_qj_B22170047,xy_qj_B22170048,xy_qj_B22170049,xy_qj_B22170050,xy_qj_B22170051,xy_qj_B22170052,xy_qj_B22170053,xy_qj_B22170054,score_201048,score_202098,score_202100,jx_yl_UPPC011,jx_yl_UPPC036,jx_yl_UPPC098,jx_yl_UPPC134,jx_yl_UPPC173,jx_yl_UPPC196,jx_yl_UPPD002,jx_yl_UPPD004,jx_yl_UPPD005,jx_yl_UPPD007,jx_yl_UPPD009,jx_yl_UPPD028,jx_yl_UPPD029,jx_yl_UPPD068,jx_yl_UPPD069,jx_yl_UPPD075,jx_yl_UPPD091,jx_yl_UPPD097,jx_yl_UPPD098,jx_yl_UPPD100,jx_yl_UPPE061,jx_yl_UPPE074,jx_yl_UPPE077,jx_yl_UPPE079,jx_yl_UPPE082,jx_yl_UPPE086,jx_yl_UPPE090,jx_yl_UPPG046,jx_yl_UPPG050,jx_yl_UPPG054,zm_jh_t0_level,jx_fico_score
0,88929,A202301090005022,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,567.0,NaN,NaN,NaN,NaN,1.419,NaN,NaN,813.0,NaN,NaN,NaN,NaN,NaN,636.0,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## （5-4-3）极信-TD212

In [112]:
# （5-4-3）极信-TD212
sql1='''
SELECT *
# trace_id, 
# replace(case when JSON_VALID(rs_decode) THEN JSON_EXTRACT(rs_decode, '$.data.resultData.score') end,'"','') as td212_score 
from db_credit.risk_jixin_score_jx1000212
;
'''

jx_td_df = query(sql1)
jx_td_df['td212_score'] = jx_td_df['rs_decode'].apply(lambda x: json.loads(x).get('data', {}).get('resultData', {}).get('score', None)) 
jx_td_df = jx_td_df[['trace_id', 'td212_score']] 
jx_td_df['td212_score'] = jx_td_df['td212_score'].astype(float)
jx_td_df.shape
# jx_td_df[:5]

df_out_data = df_out_data.merge(jx_td_df,on = 'trace_id', how = 'left')
df_out_data.shape
df_out_data[:1]

del jx_td_df

(85521, 2)

(2302173, 133)

,order_id,order_number,order_trace_id,trace_id,risk_trace_id,bj_hm_score,rentOnlineRiskLevelOverall,rentOnlineRiskLevelMultiRent,rentOnlineRiskLevelFulfill,rentOnlineDetailLevelOverall,my_zl_score,my_zl_score2,br_ys_score,br_ra_score,model_score_2311,model_score_2401,model_score_yl,model_score_2501,is_model,bj_qy_score,grade,xy_qj_A22160001,xy_qj_A22160002,xy_qj_A22160003,xy_qj_A22160004,xy_qj_A22160005,xy_qj_A22160006,xy_qj_A22160007,xy_qj_A22160008,xy_qj_A22160009,xy_qj_A22160010,xy_qj_C22180001,xy_qj_C22180002,xy_qj_C22180003,xy_qj_C22180004,xy_qj_C22180005,xy_qj_C22180006,xy_qj_C22180007,xy_qj_C22180008,xy_qj_C22180009,xy_qj_C22180010,xy_qj_C22180011,xy_qj_C22180012,xy_qj_B22170001,xy_qj_B22170002,xy_qj_B22170003,xy_qj_B22170004,xy_qj_B22170005,xy_qj_B22170006,xy_qj_B22170007,xy_qj_B22170008,xy_qj_B22170009,xy_qj_B22170010,xy_qj_B22170011,xy_qj_B22170012,xy_qj_B22170013,xy_qj_B22170014,xy_qj_B22170015,xy_qj_B22170016,xy_qj_B22170017,xy_qj_B22170018,xy_qj_B22170019,xy_qj_B22170020,xy_qj_B22170021,xy_qj_B22170022,xy_qj_B22170023,xy_qj_B22170024,xy_qj_B22170025,xy_qj_B22170026,xy_qj_B22170027,xy_qj_B22170028,xy_qj_B22170029,xy_qj_B22170030,xy_qj_B22170031,xy_qj_B22170032,xy_qj_B22170033,xy_qj_B22170034,xy_qj_B22170035,xy_qj_B22170036,xy_qj_B22170037,xy_qj_B22170038,xy_qj_B22170039,xy_qj_B22170040,xy_qj_B22170041,xy_qj_B22170042,xy_qj_B22170043,xy_qj_B22170044,xy_qj_B22170045,xy_qj_B22170046,xy_qj_B22170047,xy_qj_B22170048,xy_qj_B22170049,xy_qj_B22170050,xy_qj_B22170051,xy_qj_B22170052,xy_qj_B22170053,xy_qj_B22170054,score_201048,score_202098,score_202100,jx_yl_UPPC011,jx_yl_UPPC036,jx_yl_UPPC098,jx_yl_UPPC134,jx_yl_UPPC173,jx_yl_UPPC196,jx_yl_UPPD002,jx_yl_UPPD004,jx_yl_UPPD005,jx_yl_UPPD007,jx_yl_UPPD009,jx_yl_UPPD028,jx_yl_UPPD029,jx_yl_UPPD068,jx_yl_UPPD069,jx_yl_UPPD075,jx_yl_UPPD091,jx_yl_UPPD097,jx_yl_UPPD098,jx_yl_UPPD100,jx_yl_UPPE061,jx_yl_UPPE074,jx_yl_UPPE077,jx_yl_UPPE079,jx_yl_UPPE082,jx_yl_UPPE086,jx_yl_UPPE090,jx_yl_UPPG046,jx_yl_UPPG050,jx_yl_UPPG054,zm_jh_t0_level,jx_fico_score,td212_score
0,88929,A202301090005022,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,567.0,NaN,NaN,NaN,NaN,1.419,NaN,NaN,813.0,NaN,NaN,NaN,NaN,NaN,636.0,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## （5-4-4）蚂蚁链-履约保护

In [113]:
# （5-4-4）蚂蚁链-履约保护
sql1='''
-- 122905	122905	122905
SELECT trace_id, nsf_rental_v1_3_hundred	
from db_credit.risk_antchain_risk_general
;
'''

my_zl_general_df=query(sql1)
my_zl_general_df.shape
# my_zl_general_df[:1]
# my_zl_general_df.time.max()
# Timestamp('2025-01-15 18:28:21')
# my_zl_general_df.time.min()
# Timestamp('2024-08-19 20:55:36')

df_out_data = df_out_data.merge(my_zl_general_df,on = 'trace_id', how = 'left')
df_out_data.shape
df_out_data[:1]

del my_zl_general_df  

(72834, 2)

(2302173, 134)

,order_id,order_number,order_trace_id,trace_id,risk_trace_id,bj_hm_score,rentOnlineRiskLevelOverall,rentOnlineRiskLevelMultiRent,rentOnlineRiskLevelFulfill,rentOnlineDetailLevelOverall,my_zl_score,my_zl_score2,br_ys_score,br_ra_score,model_score_2311,model_score_2401,model_score_yl,model_score_2501,is_model,bj_qy_score,grade,xy_qj_A22160001,xy_qj_A22160002,xy_qj_A22160003,xy_qj_A22160004,xy_qj_A22160005,xy_qj_A22160006,xy_qj_A22160007,xy_qj_A22160008,xy_qj_A22160009,xy_qj_A22160010,xy_qj_C22180001,xy_qj_C22180002,xy_qj_C22180003,xy_qj_C22180004,xy_qj_C22180005,xy_qj_C22180006,xy_qj_C22180007,xy_qj_C22180008,xy_qj_C22180009,xy_qj_C22180010,xy_qj_C22180011,xy_qj_C22180012,xy_qj_B22170001,xy_qj_B22170002,xy_qj_B22170003,xy_qj_B22170004,xy_qj_B22170005,xy_qj_B22170006,xy_qj_B22170007,xy_qj_B22170008,xy_qj_B22170009,xy_qj_B22170010,xy_qj_B22170011,xy_qj_B22170012,xy_qj_B22170013,xy_qj_B22170014,xy_qj_B22170015,xy_qj_B22170016,xy_qj_B22170017,xy_qj_B22170018,xy_qj_B22170019,xy_qj_B22170020,xy_qj_B22170021,xy_qj_B22170022,xy_qj_B22170023,xy_qj_B22170024,xy_qj_B22170025,xy_qj_B22170026,xy_qj_B22170027,xy_qj_B22170028,xy_qj_B22170029,xy_qj_B22170030,xy_qj_B22170031,xy_qj_B22170032,xy_qj_B22170033,xy_qj_B22170034,xy_qj_B22170035,xy_qj_B22170036,xy_qj_B22170037,xy_qj_B22170038,xy_qj_B22170039,xy_qj_B22170040,xy_qj_B22170041,xy_qj_B22170042,xy_qj_B22170043,xy_qj_B22170044,xy_qj_B22170045,xy_qj_B22170046,xy_qj_B22170047,xy_qj_B22170048,xy_qj_B22170049,xy_qj_B22170050,xy_qj_B22170051,xy_qj_B22170052,xy_qj_B22170053,xy_qj_B22170054,score_201048,score_202098,score_202100,jx_yl_UPPC011,jx_yl_UPPC036,jx_yl_UPPC098,jx_yl_UPPC134,jx_yl_UPPC173,jx_yl_UPPC196,jx_yl_UPPD002,jx_yl_UPPD004,jx_yl_UPPD005,jx_yl_UPPD007,jx_yl_UPPD009,jx_yl_UPPD028,jx_yl_UPPD029,jx_yl_UPPD068,jx_yl_UPPD069,jx_yl_UPPD075,jx_yl_UPPD091,jx_yl_UPPD097,jx_yl_UPPD098,jx_yl_UPPD100,jx_yl_UPPE061,jx_yl_UPPE074,jx_yl_UPPE077,jx_yl_UPPE079,jx_yl_UPPE082,jx_yl_UPPE086,jx_yl_UPPE090,jx_yl_UPPG046,jx_yl_UPPG050,jx_yl_UPPG054,zm_jh_t0_level,jx_fico_score,td212_score,nsf_rental_v1_3_hundred
0,88929,A202301090005022,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,9f6c77cf4c95bc3169456398da70cca7,567.0,NaN,NaN,NaN,NaN,1.419,NaN,NaN,813.0,NaN,NaN,NaN,NaN,NaN,636.0,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 6.宽表

In [114]:
# （6-1）进件+自变量
# df_out_data：申请/调用
# df_out_data 中 order_id、order_number 没有重复也没有空
# df_out_data.shape
# df_out_data.drop_duplicates(subset = 'order_id').shape
# df_out_data.drop_duplicates(subset = 'order_number').shape
# df_out_data[df_out_data.order_id.isnull()].shape
# df_out_data[df_out_data.order_number.isnull()].shape

# 285992
# df_jj：进件
# df_jj.shape
# df_jj.drop_duplicates(subset = 'order_id').shape
# df_jj.drop_duplicates(subset = 'order_number').shape


df_info_ori = df_jj.merge(df_out_data, on = 'order_number', how = 'left')
# Timestamp('2024-10-16 00:00:00')  ，690 条出库 在 df_out_data 中无记录，中间表往前回溯的时候少量数据侧漏，如 A202407222000361 1277221 ee3e7156005b1e040f7ca4a85455a796
# df_info_ori[(df_analy.order_id.isnull()) & (df_info_ori.下单日期 <= '2025-03-10') & (df_info_ori.审核状态 == '出库')].下单日期.max()
df_info_ori = df_info_ori.drop(columns = ['risk_trace_id'])
df_info_ori.shape
df_info_ori.tail(1) 
# (291100, 142)

(291605, 148)

,order_number,下单日期,search_time,归属渠道,是否免人审,审核状态,拒绝理由,是否前置拦截,是否机审强拒,是否出库前风控强拒,是否拒量,是否联合运营,联合运营命中风控,user_mobile,id_card_num,jd_user_level_risk,order_id,order_trace_id,trace_id,bj_hm_score,rentOnlineRiskLevelOverall,rentOnlineRiskLevelMultiRent,rentOnlineRiskLevelFulfill,rentOnlineDetailLevelOverall,my_zl_score,my_zl_score2,br_ys_score,br_ra_score,model_score_2311,model_score_2401,model_score_yl,model_score_2501,is_model,bj_qy_score,grade,xy_qj_A22160001,xy_qj_A22160002,xy_qj_A22160003,xy_qj_A22160004,xy_qj_A22160005,xy_qj_A22160006,xy_qj_A22160007,xy_qj_A22160008,xy_qj_A22160009,xy_qj_A22160010,xy_qj_C22180001,xy_qj_C22180002,xy_qj_C22180003,xy_qj_C22180004,xy_qj_C22180005,xy_qj_C22180006,xy_qj_C22180007,xy_qj_C22180008,xy_qj_C22180009,xy_qj_C22180010,xy_qj_C22180011,xy_qj_C22180012,xy_qj_B22170001,xy_qj_B22170002,xy_qj_B22170003,xy_qj_B22170004,xy_qj_B22170005,xy_qj_B22170006,xy_qj_B22170007,xy_qj_B22170008,xy_qj_B22170009,xy_qj_B22170010,xy_qj_B22170011,xy_qj_B22170012,xy_qj_B22170013,xy_qj_B22170014,xy_qj_B22170015,xy_qj_B22170016,xy_qj_B22170017,xy_qj_B22170018,xy_qj_B22170019,xy_qj_B22170020,xy_qj_B22170021,xy_qj_B22170022,xy_qj_B22170023,xy_qj_B22170024,xy_qj_B22170025,xy_qj_B22170026,xy_qj_B22170027,xy_qj_B22170028,xy_qj_B22170029,xy_qj_B22170030,xy_qj_B22170031,xy_qj_B22170032,xy_qj_B22170033,xy_qj_B22170034,xy_qj_B22170035,xy_qj_B22170036,xy_qj_B22170037,xy_qj_B22170038,xy_qj_B22170039,xy_qj_B22170040,xy_qj_B22170041,xy_qj_B22170042,xy_qj_B22170043,xy_qj_B22170044,xy_qj_B22170045,xy_qj_B22170046,xy_qj_B22170047,xy_qj_B22170048,xy_qj_B22170049,xy_qj_B22170050,xy_qj_B22170051,xy_qj_B22170052,xy_qj_B22170053,xy_qj_B22170054,score_201048,score_202098,score_202100,jx_yl_UPPC011,jx_yl_UPPC036,jx_yl_UPPC098,jx_yl_UPPC134,jx_yl_UPPC173,jx_yl_UPPC196,jx_yl_UPPD002,jx_yl_UPPD004,jx_yl_UPPD005,jx_yl_UPPD007,jx_yl_UPPD009,jx_yl_UPPD028,jx_yl_UPPD029,jx_yl_UPPD068,jx_yl_UPPD069,jx_yl_UPPD075,jx_yl_UPPD091,jx_yl_UPPD097,jx_yl_UPPD098,jx_yl_UPPD100,jx_yl_UPPE061,jx_yl_UPPE074,jx_yl_UPPE077,jx_yl_UPPE079,jx_yl_UPPE082,jx_yl_UPPE086,jx_yl_UPPE090,jx_yl_UPPG046,jx_yl_UPPG050,jx_yl_UPPG054,zm_jh_t0_level,jx_fico_score,td212_score,nsf_rental_v1_3_hundred
291604,A20250606174831588,2025-06-06,2025-06,搜索渠道,1,出库,[,0,0,0,0,None,None,15670210353,411526199611180741,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
# （6-2）关联租后表现
# df_info_ori ：进件订单的各方面数据
# FPD7+、f3PD7+、Mob9PD30+、
# 历史最大逾期天数：为租期历史最大逾期
# var_1 = ['order_number', '历史最大逾期天数', 'FPD', 'SPD', 'TPD', '第二期逾期天数', '第三期逾期天数','第四期逾期天数', '首期应还日期', 'overdue_type', 'f2pd7', 'f3pd7', 'y_fraud_fpd7','y_fraud_f2pd7', 'y_fraud_f3pd7', 'y_credit_mob3pd30', 'y_credit']
# var_1 = ['order_number', 'y_fraud_fpd7', 'y_fraud_mob3dpd7', 'y_credit_mob3dpd30', 'y_credit']
var_1 = ['order_number', 'y_fraud_fpd7', 'y_fraud_mob3dpd7', 'y_credit_mob3dpd30', 'y_credit_mob6dpd30', 'y_credit']
df_info_ori = df_info_ori.merge(df_analy_data[var_1], on = 'order_number', how = 'left')
# (33986, 31)
# df_info_ori = df_info_ori.merge(df_analy_data, on = 'order_number', how = 'inner')
df_info_ori.shape
df_info_ori[:1]
# (291100, 147)

(291618, 153)

,order_number,下单日期,search_time,归属渠道,是否免人审,审核状态,拒绝理由,是否前置拦截,是否机审强拒,是否出库前风控强拒,是否拒量,是否联合运营,联合运营命中风控,user_mobile,id_card_num,jd_user_level_risk,order_id,order_trace_id,trace_id,bj_hm_score,rentOnlineRiskLevelOverall,rentOnlineRiskLevelMultiRent,rentOnlineRiskLevelFulfill,rentOnlineDetailLevelOverall,my_zl_score,my_zl_score2,br_ys_score,br_ra_score,model_score_2311,model_score_2401,model_score_yl,model_score_2501,is_model,bj_qy_score,grade,xy_qj_A22160001,xy_qj_A22160002,xy_qj_A22160003,xy_qj_A22160004,xy_qj_A22160005,xy_qj_A22160006,xy_qj_A22160007,xy_qj_A22160008,xy_qj_A22160009,xy_qj_A22160010,xy_qj_C22180001,xy_qj_C22180002,xy_qj_C22180003,xy_qj_C22180004,xy_qj_C22180005,xy_qj_C22180006,xy_qj_C22180007,xy_qj_C22180008,xy_qj_C22180009,xy_qj_C22180010,xy_qj_C22180011,xy_qj_C22180012,xy_qj_B22170001,xy_qj_B22170002,xy_qj_B22170003,xy_qj_B22170004,xy_qj_B22170005,xy_qj_B22170006,xy_qj_B22170007,xy_qj_B22170008,xy_qj_B22170009,xy_qj_B22170010,xy_qj_B22170011,xy_qj_B22170012,xy_qj_B22170013,xy_qj_B22170014,xy_qj_B22170015,xy_qj_B22170016,xy_qj_B22170017,xy_qj_B22170018,xy_qj_B22170019,xy_qj_B22170020,xy_qj_B22170021,xy_qj_B22170022,xy_qj_B22170023,xy_qj_B22170024,xy_qj_B22170025,xy_qj_B22170026,xy_qj_B22170027,xy_qj_B22170028,xy_qj_B22170029,xy_qj_B22170030,xy_qj_B22170031,xy_qj_B22170032,xy_qj_B22170033,xy_qj_B22170034,xy_qj_B22170035,xy_qj_B22170036,xy_qj_B22170037,xy_qj_B22170038,xy_qj_B22170039,xy_qj_B22170040,xy_qj_B22170041,xy_qj_B22170042,xy_qj_B22170043,xy_qj_B22170044,xy_qj_B22170045,xy_qj_B22170046,xy_qj_B22170047,xy_qj_B22170048,xy_qj_B22170049,xy_qj_B22170050,xy_qj_B22170051,xy_qj_B22170052,xy_qj_B22170053,xy_qj_B22170054,score_201048,score_202098,score_202100,jx_yl_UPPC011,jx_yl_UPPC036,jx_yl_UPPC098,jx_yl_UPPC134,jx_yl_UPPC173,jx_yl_UPPC196,jx_yl_UPPD002,jx_yl_UPPD004,jx_yl_UPPD005,jx_yl_UPPD007,jx_yl_UPPD009,jx_yl_UPPD028,jx_yl_UPPD029,jx_yl_UPPD068,jx_yl_UPPD069,jx_yl_UPPD075,jx_yl_UPPD091,jx_yl_UPPD097,jx_yl_UPPD098,jx_yl_UPPD100,jx_yl_UPPE061,jx_yl_UPPE074,jx_yl_UPPE077,jx_yl_UPPE079,jx_yl_UPPE082,jx_yl_UPPE086,jx_yl_UPPE090,jx_yl_UPPG046,jx_yl_UPPG050,jx_yl_UPPG054,zm_jh_t0_level,jx_fico_score,td212_score,nsf_rental_v1_3_hundred,y_fraud_fpd7,y_fraud_mob3dpd7,y_credit_mob3dpd30,y_credit_mob6dpd30,y_credit
0,A2023100100113925,2023-10-01,2023-10,芝麻租物,0,机审强拒,"[命中云商分低于515强拒, 命中融安分低于680强拒, 评分等级低于D,命中拒绝]",0,1,0,0,None,None,15376116117,372923200009253510,None,656817.0,8962e07e36f809256441eb77b40f8f8c,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [116]:
# （6-3）规则命中处理--统计命中率的时候再加工
# 参考回捞分析

In [117]:
# （6-4）数据检查
# 1-
# df_info_ori['是否出库'] = np.where(df_info_ori.y_credit.isin([0,1]), 1, 0)
# df_info_ori.groupby(['归属渠道']).agg({'order_number': 'count'})
# df_info_ori[(df_info_ori.归属渠道 == '98租超')&(df_info_ori.search_time == '2025-03')].groupby(['下单日期']).agg({'order_number':'count', '是否出库':'sum'})
# df_info_ori[(df_info_ori.归属渠道 == '八派信息')&(df_info_ori.search_time == '2025-03')].groupby(['下单日期']).agg({'order_number':'count', '是否出库':'sum'})
# df_info_ori[(df_info_ori.归属渠道 == '九州信息')&(df_info_ori.search_time == '2025-03')].groupby(['下单日期']).agg({'order_number':'count', '是否出库':'sum'})

# 2-细化综合风险等级 免押
# tmp = df_info_ori[df_info_ori.下单日期 >= '2025-01-01'][['order_number','下单日期','拒绝理由','rentOnlineDetailLevelOverall']]

# pd.pivot_table(df_info_ori[df_info_ori.下单日期 >= '2025-03-15'],index='下单日期',columns='rentOnlineDetailLevelOverall',values='order_number',aggfunc='count')
# df_info_ori[(df_info_ori.下单日期 == '2025-03-30')&(df_info_ori.rentOnlineDetailLevelOverall.isin(['5','6']))][['order_number','下单日期','拒绝理由','rentOnlineDetailLevelOverall']]

# 3-火眸分和融安分的相关性
# df_info_ori[df_info_ori.下单日期 >= '2024-01-01'][['br_ra_score','bj_hm_score']].corr()
# df_analy_ks[df_analy_ks.下单日期 >= '2024-01-01'][['br_ra_score','bj_hm_score']].corr()

# （6-5）融360
# score_201048_Bin score_202098_Bin score_202100_Bin
# df_info_ori[df_info_ori.search_time == '2025-02'].score_201048_Bin.value_counts()
# df_info_ori[df_info_ori.search_time == '2025-02'].groupby(['score_202100_Bin']).agg({'order_number':'count'})
# df_info_ori[df_info_ori.search_time == '2025-03'].groupby(['score_202100_Bin']).agg({'order_number':'count'})
# df_info_ori[df_info_ori.search_time == '2025-04'].groupby(['score_202100_Bin']).agg({'order_number':'count'})
# df_info_ori[df_info_ori.search_time == '2025-05'].groupby(['score_202100_Bin']).agg({'order_number':'count'})

# df_info_ori[(df_info_ori.search_time == '2024-12') & (df_info_ori.y_credit.isin([0,1]))].groupby(['score_202100_Bin']).agg({'order_number':'count'})
# df_info_ori[(df_info_ori.search_time == '2025-01') & (df_info_ori.y_credit.isin([0,1]))].groupby(['score_202100_Bin']).agg({'order_number':'count'})
# df_info_ori[(df_info_ori.search_time == '2025-02') & (df_info_ori.y_credit.isin([0,1]))].groupby(['score_202100_Bin']).agg({'order_number':'count'})
# df_info_ori[(df_info_ori.search_time == '2025-03') & (df_info_ori.y_credit.isin([0,1]))].groupby(['score_202100_Bin']).agg({'order_number':'count'})
# df_info_ori[(df_info_ori.search_time == '2025-04') & (df_info_ori.y_credit.isin([0,1]))].groupby(['score_202100_Bin']).agg({'order_number':'count'})
# df_info_ori[(df_info_ori.search_time == '2025-05') & (df_info_ori.y_credit.isin([0,1]))].groupby(['score_202100_Bin']).agg({'order_number':'count'})

In [118]:
var_bin_qz = ['bj_hm_score']
df_risk_new = df_risk_new.merge(df_out_data[['order_number'] + var_bin_qz], on = 'order_number', how = 'left')


# 7.分箱-业务角度

## 7_1.前置

In [119]:
# （7-1）前置
# 数据驱动+业务实际应用 
# 冰鉴火眸分<500且搜索渠道、二手非抖音豁免
# df_info_ori['火眸分_Bin'] = df_info_ori.apply(lambda x: '1.(0,449]' if x.bj_hm_score >= 0 and x.bj_hm_score<= 449
#                                            else '2.(449,499]' if x.bj_hm_score > 449 and x.bj_hm_score <= 499
#                                            else '3.(499,530]' if x.bj_hm_score > 499 and x.bj_hm_score <= 530
#                                            else '4.(530,579]' if x.bj_hm_score > 530 and x.bj_hm_score <= 579
#                                            else '5.(579,657]' if x.bj_hm_score > 579 and x.bj_hm_score <= 657
#                                            else '6.(657,inf)' if x.bj_hm_score > 657
#                                            else '7.-999'
#                                            , axis = 1)

df_info_ori['火眸分_Bin'] = df_info_ori.apply(lambda x : '1.[0,449]'  if x.bj_hm_score >= 0 and x.bj_hm_score <= 449
                                                else '2.(449,475.0]' if x.bj_hm_score  > 449 and x.bj_hm_score  <= 475
                                                else '3.(475,499.0]' if x.bj_hm_score  > 475 and x.bj_hm_score  <= 499
                                                else '4.(499,520.0]' if x.bj_hm_score  > 499 and x.bj_hm_score  <= 520
                                                else '5.(520,538.0]' if x.bj_hm_score  > 520 and x.bj_hm_score  <= 538
                                                else '6.(538,556.0]' if x.bj_hm_score  > 538 and x.bj_hm_score  <= 556
                                                else '7.(556,575.0]' if x.bj_hm_score  > 556 and x.bj_hm_score  <= 575
                                                else '8.(575,596.0]' if x.bj_hm_score  > 575 and x.bj_hm_score  <= 596
                                                else '9.(596,625.0]' if x.bj_hm_score  > 596 and x.bj_hm_score  <= 625
                                                else '91.(625,inf)' if x.bj_hm_score  > 625
                                                else '92.-999', axis = 1)

df_risk_new['火眸分_Bin'] = df_risk_new.apply(lambda x : '1.[0,449]'  if x.bj_hm_score >= 0 and x.bj_hm_score <= 449
                                                else '2.(449,475.0]' if x.bj_hm_score  > 449 and x.bj_hm_score  <= 475
                                                else '3.(475,499.0]' if x.bj_hm_score  > 475 and x.bj_hm_score  <= 499
                                                else '4.(499,520.0]' if x.bj_hm_score  > 499 and x.bj_hm_score  <= 520
                                                else '5.(520,538.0]' if x.bj_hm_score  > 520 and x.bj_hm_score  <= 538
                                                else '6.(538,556.0]' if x.bj_hm_score  > 538 and x.bj_hm_score  <= 556
                                                else '7.(556,575.0]' if x.bj_hm_score  > 556 and x.bj_hm_score  <= 575
                                                else '8.(575,596.0]' if x.bj_hm_score  > 575 and x.bj_hm_score  <= 596
                                                else '9.(596,625.0]' if x.bj_hm_score  > 596 and x.bj_hm_score  <= 625
                                                else '91.(625,inf)' if x.bj_hm_score  > 625
                                                else '92.-999', axis = 1)

## 7_2_1.机审

In [120]:
# （7-2）机审
# 细化综合风险等级 rentOnlineDetailLevelOverall
# 综合风险等级 rentOnlineRiskLevelOverall
# 共租行为等级 rentOnlineRiskLevelMultiRent
# 履约历史等级 rentOnlineRiskLevelFulfill
df_info_ori['rentOnlineRiskLevelFulfill_Bin'] = df_info_ori.apply(lambda x:'1.1' if x.rentOnlineRiskLevelFulfill == '1'
                                                                else '2.0' if x.rentOnlineRiskLevelFulfill == '0'
                                                                else '3.-999', axis = 1)
df_info_ori['rentOnlineRiskLevelOverall_Bin'] = df_info_ori.apply(lambda x: '1.3' if x.rentOnlineRiskLevelOverall == '3'
                                                                   else '2.2' if x.rentOnlineRiskLevelOverall == '2'
                                                                   else '3.1' if x.rentOnlineRiskLevelOverall == '1'
                                                                   else '4.0' if x.rentOnlineRiskLevelOverall == '0'
                                                                   else '5.-999', axis = 1)
df_info_ori['rentOnlineDetailLevelOverall_Bin'] = df_info_ori.apply(lambda x: '1.6' if x.rentOnlineDetailLevelOverall == '6'
                                                                   else '2.5' if x.rentOnlineDetailLevelOverall == '5'
                                                                   else '3.4' if x.rentOnlineDetailLevelOverall == '4'
                                                                   else '4.3' if x.rentOnlineDetailLevelOverall == '3'
                                                                   else '5.2' if x.rentOnlineDetailLevelOverall == '2'
                                                                   else '6.1' if x.rentOnlineDetailLevelOverall == '1'
                                                                   else '7.0' if x.rentOnlineDetailLevelOverall == '0'
                                                                   else '8.-999', axis = 1)

# 规则17		蚁盾分>=80
# 规则18		蚁盾分>50且为搜索渠道
# 规则25		命中融安分低于680
# 规则26		命中融安分低于700且为搜索渠道

df_info_ori['my_zl_score'] = df_info_ori['my_zl_score'].astype(float)
df_info_ori['br_ra_score'] = df_info_ori['br_ra_score'].astype(float)
# df_info_ori['蚁盾分_Bin'] = df_info_ori.apply(lambda x: '7.[0,25]' if x.my_zl_score >=0 and x.my_zl_score <= 25
#                                            else '6.(25,50]' if x.my_zl_score > 25 and x.my_zl_score  <= 50
#                                            else '5.(50,60]' if x.my_zl_score > 50 and x.my_zl_score <= 60
#                                            else '4.(60,70]' if x.my_zl_score > 60 and x.my_zl_score  <= 70
#                                            else '3.(70,80)' if x.my_zl_score > 70 and x.my_zl_score  < 80
#                                             else '2.[80,90)' if x.my_zl_score >= 80 and x.my_zl_score  < 90
#                                             else '1.[90,inf)' if x.my_zl_score >= 90
#                                             else '8.-999' ,axis = 1)
df_info_ori['蚁盾分_Bin'] = df_info_ori.apply(lambda x: '91.[0,10]' if x.my_zl_score >=0 and x.my_zl_score <= 10
                                            else '9.(10,20]' if x.my_zl_score > 10 and x.my_zl_score  <= 20
                                            else '8.(20,30]' if x.my_zl_score > 20 and x.my_zl_score  <= 30
                                            else '7.(30,40]' if x.my_zl_score > 30 and x.my_zl_score  <= 40
                                            else '6.(40,50]' if x.my_zl_score > 40 and x.my_zl_score  <= 50
                                            else '5.(50,60]' if x.my_zl_score > 50 and x.my_zl_score  <= 60
                                            else '4.(60,70]' if x.my_zl_score > 60 and x.my_zl_score  <= 70
                                            else '3.(70,80]' if x.my_zl_score > 70 and x.my_zl_score  <= 80
                                            else '2.(80,90]' if x.my_zl_score > 80 and x.my_zl_score  <= 90
                                            else '1.(90,inf)' if x.my_zl_score > 90
                                            else '92.-999' ,axis = 1)

# df_info_ori['融安分_Bin'] = df_info_ori.apply(lambda x: '1.[0,600]' if x.br_ra_score >= 0 and x.br_ra_score <= 600
#                                            else '2.(600,650]' if x.br_ra_score > 600 and x.br_ra_score <= 650
#                                            else '3.(650,680]' if x.br_ra_score > 650 and x.br_ra_score <= 680
#                                            else '4.(680,700]' if x.br_ra_score > 680 and x.br_ra_score <= 700
#                                            else '5.(700,750]' if x.br_ra_score > 700 and x.br_ra_score <= 750
#                                            else '6.(750,800]' if x.br_ra_score > 750 and x.br_ra_score <= 800
#                                            else '7.(800,inf)' if x.br_ra_score > 800
#                                            else '8.-999', axis = 1)
df_info_ori['融安分_Bin'] = df_info_ori.apply(lambda x : '1.[0,680.0]'  if x.br_ra_score >= 0 and x.br_ra_score <= 680 
                                                else '2.(680,700.0]' if x.br_ra_score  > 680 and x.br_ra_score  <= 700
                                                else '3.(700,713.0]' if x.br_ra_score  > 700 and x.br_ra_score  <= 713
                                                else '4.(713,726.0]' if x.br_ra_score  > 713 and x.br_ra_score  <= 726
                                                else '5.(726,740.0]' if x.br_ra_score  > 726 and x.br_ra_score  <= 740
                                                else '6.(740,755.0]' if x.br_ra_score  > 740 and x.br_ra_score  <= 755
                                                else '7.(755,771.0]' if x.br_ra_score  > 755 and x.br_ra_score  <= 771
                                                else '8.(771,801.0]' if x.br_ra_score  > 771 and x.br_ra_score  <= 801
                                                else '9.(801,854.0]' if x.br_ra_score  > 801 and x.br_ra_score  <= 854
                                                else '91.(854,inf)' if x.br_ra_score  > 854
                                                else '92.-999', axis = 1)
# 蚁盾分2.0
# df_info_ori['my_zl_score2_Bin'] = df_info_ori.apply(lambda x: '7.[0,25]' if x.my_zl_score2 >=0 and x.my_zl_score2 <= 25
#                                                 else '6.(25,45]' if x.my_zl_score2 > 25 and x.my_zl_score2  <= 45
#                                                 else '5.(45,50]' if x.my_zl_score2 > 45 and x.my_zl_score2  <= 50
#                                                 else '4.(50,65)' if x.my_zl_score2 > 50 and x.my_zl_score2  < 65
#                                                 else '3.[65,70)' if x.my_zl_score2 >= 65 and x.my_zl_score2  < 70
#                                                 else '2.[70,75)' if x.my_zl_score2 >= 70 and x.my_zl_score2  < 75
#                                                 else '1.[75,inf)' if x.my_zl_score2 >= 75
#                                                 else '8.-999' 
#                                                 ,axis = 1)
df_info_ori['my_zl_score2_Bin'] = df_info_ori.apply(lambda x: '91.[0,50]' if x.my_zl_score2 >=0 and x.my_zl_score2 <= 50
                                            else '9.(50,53]' if x.my_zl_score2 > 50 and x.my_zl_score2  <= 53
                                            else '8.(53,56]' if x.my_zl_score2 > 53 and x.my_zl_score2  <= 56
                                            else '7.(56,58]' if x.my_zl_score2 > 56 and x.my_zl_score2  <= 58
                                            else '6.(58,60]' if x.my_zl_score2 > 58 and x.my_zl_score2  <= 60
                                            else '5.(60,62.5]' if x.my_zl_score2 > 60 and x.my_zl_score2  <= 62.5
                                            else '4.(62.5,65]' if x.my_zl_score2 > 62.5 and x.my_zl_score2  <= 65
                                            else '3.(65,70]' if x.my_zl_score2 > 65 and x.my_zl_score2  <= 70
                                            else '2.(70,80]' if x.my_zl_score2 > 70 and x.my_zl_score2  <= 80
                                            else '1.(80,inf)' if x.my_zl_score2 > 80
                                            else '92.-999' ,axis = 1)

# 云商分
# df_info_ori['br_ys_score_Bin'] = df_info_ori.apply(lambda x : '1.[0,450.0]'  if x.br_ys_score >= 0 and x.br_ys_score <= 450 
#                                                   else '2.(450,475.0]' if x.br_ys_score  > 450 and x.br_ys_score  <= 475
#                                                   else '3.(475,495.0]' if x.br_ys_score  > 475 and x.br_ys_score  <= 495
#                                                   else '4.(495,514.0]' if x.br_ys_score  > 495 and x.br_ys_score  <= 514
#                                                   else '5.(514,542.0]' if x.br_ys_score  > 514 and x.br_ys_score  <= 542
#                                                   else '6.(542,555.0]' if x.br_ys_score  > 542 and x.br_ys_score  <= 555
#                                                   else '7.(555,615.0]' if x.br_ys_score  > 555 and x.br_ys_score  <= 615
#                                                   else '8.(615,660.0]' if x.br_ys_score  > 615 and x.br_ys_score  <= 660
#                                                   else '9.(660,705.0]' if x.br_ys_score  > 660 and x.br_ys_score  <= 705
#                                                   else '91.(705,742.0]' if x.br_ys_score  > 705 and x.br_ys_score  <= 742
#                                                   else '92.(742,inf)' if x.br_ys_score  > 742
#                                                   else '93.-999', axis = 1)

df_info_ori['br_ys_score_Bin'] = df_info_ori.apply(lambda x : '1.[0,450.0]'  if x.br_ys_score >= 0 and x.br_ys_score <= 450 
                                                else '2.(450,475.0]' if x.br_ys_score  > 450 and x.br_ys_score  <= 475
                                                else '3.(475,495.0]' if x.br_ys_score  > 475 and x.br_ys_score  <= 495
                                                else '4.(495,514.0]' if x.br_ys_score  > 495 and x.br_ys_score  <= 514
                                                else '5.(514,540.0]' if x.br_ys_score  > 514 and x.br_ys_score  <= 540
                                                else '6.(540,565.0]' if x.br_ys_score  > 540 and x.br_ys_score  <= 565
                                                else '7.(565,620.0]' if x.br_ys_score  > 565 and x.br_ys_score  <= 620
                                                else '8.(620,685.0]' if x.br_ys_score  > 620 and x.br_ys_score  <= 685
                                                else '9.(685,742.0]' if x.br_ys_score  > 685 and x.br_ys_score  <= 742
                                                else '91.(742,inf)' if x.br_ys_score  > 742
                                                else '92.-999', axis = 1)
# 青云分
# df_info_ori['青云分_Bin'] = df_info_ori.apply(lambda x: '1.[0,449]' if x.bj_qy_score >= 0 and x.bj_qy_score <= 449
#                                             else '2.(449,499]' if x.bj_qy_score > 449 and x.bj_qy_score <= 499
#                                             else '3.(499,533.0]' if x.bj_qy_score > 499 and x.bj_qy_score <= 533
#                                             else '4.(533.0,575.0]' if x.bj_qy_score > 533 and x.bj_qy_score <= 575
#                                             else '5.(575.0,607.0]' if x.bj_qy_score > 575 and x.bj_qy_score <= 607
#                                             else '6.(607.0,652.0]' if x.bj_qy_score > 607 and x.bj_qy_score <= 652
#                                             else '7.(652, inf)' if x.bj_qy_score > 652
#                                             else '8.-999', axis = 1)
df_info_ori['青云分_Bin'] = df_info_ori.apply(lambda x : '1.[0,433.5]'  if x.bj_qy_score >= 0 and x.bj_qy_score <= 433.5 
                                                else '2.(433.5,477.0]' if x.bj_qy_score  > 433.5 and x.bj_qy_score  <= 477
                                                else '3.(477,499.0]' if x.bj_qy_score  > 477 and x.bj_qy_score  <= 499
                                                else '4.(499,516.0]' if x.bj_qy_score  > 499 and x.bj_qy_score  <= 516
                                                else '5.(516,533.0]' if x.bj_qy_score  > 516 and x.bj_qy_score  <= 533
                                                else '6.(533,551.0]' if x.bj_qy_score  > 533 and x.bj_qy_score  <= 551
                                                else '7.(551,570.0]' if x.bj_qy_score  > 551 and x.bj_qy_score  <= 570
                                                else '8.(570,594.0]' if x.bj_qy_score  > 570 and x.bj_qy_score  <= 594
                                                else '9.(594,627.0]' if x.bj_qy_score  > 594 and x.bj_qy_score  <= 627
                                                else '91.(627,inf)' if x.bj_qy_score  > 627
                                                else '92.-999', axis = 1)
# 按照之前分析的结果分组
# 0：没有使用模型
# 1：ascore2311
# 2：ascore_yl2311
# 3：ascore2401
conditions_1 = (df_info_ori.is_model == '1')
conditions_2 = (df_info_ori.is_model == '2')
conditions_3 = (df_info_ori.is_model == '3')
conditions_4 = (df_info_ori.is_model == '4')

# df_info_ori['model_score_2311_Bin'] = df_info_ori.apply(lambda x: '1.[0,12.9620]' if x.归属渠道 == '芝麻租物'  and x.是否拒量 ==0 and x.model_score_2311 >= 0  and x.model_score_2311 <= 12.9620
#                                                 else '2.(12.9620,15.2211]' if x.归属渠道 == '芝麻租物'  and x.是否拒量 ==0 and x.model_score_2311 > 12.9620  and x.model_score_2311 <= 15.2211
#                                                 else '3.(15.2211,18.321]' if x.归属渠道 == '芝麻租物'  and x.是否拒量 ==0 and x.model_score_2311 > 15.2211  and x.model_score_2311 <= 18.321
#                                                 else '4.(18.321, 23.297]' if x.归属渠道 == '芝麻租物'  and x.是否拒量 ==0 and x.model_score_2311 > 18.321 and x.model_score_2311 <= 23.297
#                                                 else '5.(23.297, inf)' if x.归属渠道 == '芝麻租物'  and x.是否拒量 ==0 and x.model_score_2311 > 23.297 

#                                                 else '6.[0,18.321]' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.归属渠道 != '抖音渠道' and x.归属渠道 != '广州易合顺' and x.归属渠道 != '深圳刘全兵号卡1' and x.是否拒量 ==0 and x.model_score_2311 >= 0  and x.model_score_2311 <= 18.321
#                                                 else '7.(18.321, inf)' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.归属渠道 != '抖音渠道' and x.归属渠道 != '广州易合顺' and x.归属渠道 != '深圳刘全兵号卡1' and x.是否拒量 ==0 and x.model_score_2311 > 18.321

#                                                 else '8.[0,16.6]' if x.归属渠道 == '搜索渠道'  and x.是否拒量 ==0 and x.model_score_2311 >= 0  and x.model_score_2311 <= 16.6
#                                                 else '9.(16.6, 21.435]' if x.归属渠道 == '搜索渠道'  and x.是否拒量 ==0 and x.model_score_2311 > 16.6 and x.model_score_2311 <= 21.435
#                                                 else '91.(21.435,inf)' if x.归属渠道 == '搜索渠道'  and x.是否拒量 ==0 and x.model_score_2311 > 21.435
#                                                 else 'other'
#                                                 , axis = 1)


# df_info_ori['model_score_yl_Bin'] = df_info_ori.apply(lambda x: '1.[0,13.5137]' if x.归属渠道 == '芝麻租物'  and x.是否拒量 ==0 and x.model_score_yl >= 0  and x.model_score_yl <= 13.5137
#                                                 else '2.(13.5137,14.9787]' if x.归属渠道 == '芝麻租物'  and x.是否拒量 ==0 and x.model_score_yl > 13.5137  and x.model_score_yl <= 14.9787
#                                                 else '3.(14.9787,17.55]' if x.归属渠道 == '芝麻租物'  and x.是否拒量 ==0 and x.model_score_yl > 14.9787  and x.model_score_yl <= 17.55
#                                                 else '4.(17.55,23.382]' if x.归属渠道 == '芝麻租物'  and x.是否拒量 ==0 and x.model_score_yl > 17.55 and x.model_score_yl <= 23.382
#                                                 else '5.(23.382, inf)' if x.归属渠道 == '芝麻租物'  and x.是否拒量 ==0 and x.model_score_yl > 23.382

#                                                 else '6.[0,17.55]' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.归属渠道 != '抖音渠道' and x.归属渠道 != '广州易合顺' and x.归属渠道 != '深圳刘全兵号卡1' and x.是否拒量 ==0 and x.model_score_yl >= 0  and x.model_score_yl <= 17.55
#                                                 else '7.(17.55, inf)' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.归属渠道 != '抖音渠道' and x.归属渠道 != '广州易合顺' and x.归属渠道 != '深圳刘全兵号卡1' and x.是否拒量 ==0 and x.model_score_yl > 17.55

#                                                 else '8.[0,10]' if x.归属渠道 == '搜索渠道'  and x.是否拒量 ==0 and x.model_score_yl >= 0  and x.model_score_yl <= 10
#                                                 else '9.(10,15.059]' if x.归属渠道 == '搜索渠道'  and x.是否拒量 ==0 and x.model_score_yl > 10  and x.model_score_yl <= 15.059
#                                                 else '91.(15.059, 21.661]' if x.归属渠道 == '搜索渠道'  and x.是否拒量 ==0 and x.model_score_yl > 15.059 and x.model_score_yl <= 21.661
#                                                 else '92.(21.661,inf)' if x.归属渠道 == '搜索渠道'  and x.是否拒量 ==0 and x.model_score_yl > 21.661
#                                                 else 'other'
#                                                 , axis = 1)

# df_info_ori['model_score_2401_Bin'] = df_info_ori.apply(lambda x: '1.[0,6.1907]' if x.归属渠道 == '芝麻租物'  and x.是否拒量 ==0 and x.model_score_2401 >= 0  and x.model_score_2401 <= 6.1907
#                                                 else '2.(6.1907,8.1386]' if x.归属渠道 == '芝麻租物'  and x.是否拒量 ==0 and x.model_score_2401 > 6.1907  and x.model_score_2401 <= 8.1386
#                                                 else '3.(8.1386,11.04]' if x.归属渠道 == '芝麻租物'  and x.是否拒量 ==0 and x.model_score_2401 > 8.1386  and x.model_score_2401 <= 11.04
#                                                 else '4.(11.04, 16]' if x.归属渠道 == '芝麻租物'  and x.是否拒量 ==0 and x.model_score_2401 > 11.04 and x.model_score_2401 <= 16 
#                                                 else '5.(16,inf)' if x.归属渠道 == '芝麻租物'  and x.是否拒量 ==0 and x.model_score_2401 > 16

#                                                 else '6.[0,11.04]' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.归属渠道 != '抖音渠道' and x.归属渠道 != '广州易合顺' and x.归属渠道 != '深圳刘全兵号卡1' and x.是否拒量 ==0 and x.model_score_2401 >= 0  and x.model_score_2401 <= 11.04
#                                                 else '7.(11.04,inf)' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.归属渠道 != '抖音渠道' and x.归属渠道 != '广州易合顺' and x.归属渠道 != '深圳刘全兵号卡1' and x.是否拒量 ==0 and x.model_score_2401 > 11.04 

#                                                 else '8.[0,9.35]' if x.归属渠道 == '搜索渠道'  and x.是否拒量 ==0 and x.model_score_2401 >= 0  and x.model_score_2401 <= 9.35
#                                                 else '9.(9.35, 14.93]' if x.归属渠道 == '搜索渠道'  and x.是否拒量 ==0 and x.model_score_2401 > 9.35 and x.model_score_2401 <= 14.93
#                                                 else '91.(14.93,inf)' if x.归属渠道 == '搜索渠道'  and x.是否拒量 ==0 and x.model_score_2401 > 14.93
#                                                 else 'other'
#                                                 , axis = 1)

# df_info_ori['model_score_2501_Bin'] = df_info_ori.apply(lambda x: '1.[0,10.047]' if x.归属渠道 == '芝麻租物'  and x.是否拒量 ==0 and x.model_score_2501 >= 0 and x.model_score_2501 <= 10.047
#                                                 else '2.(10.047,11.635]' if x.归属渠道 == '芝麻租物'  and x.是否拒量 ==0 and x.model_score_2501 > 10.047 and x.model_score_2501 <= 11.635
#                                                 else '3.(11.635,14.104]' if x.归属渠道 == '芝麻租物'  and x.是否拒量 ==0 and x.model_score_2501 > 11.635 and x.model_score_2501 <= 14.104
#                                                 else '4.(14.104, 18.132]' if x.归属渠道 == '芝麻租物'  and x.是否拒量 ==0 and x.model_score_2501 > 14.104 and x.model_score_2501 <= 18.132
#                                                 else '5.(18.132,inf)' if x.归属渠道 == '芝麻租物'  and x.是否拒量 ==0 and x.model_score_2501 > 18.132

#                                                 else '6.[0,10.047]' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.归属渠道 != '抖音渠道' and x.归属渠道 != '广州易合顺' and x.归属渠道 != '深圳刘全兵号卡1' and x.是否拒量 ==0 and x.model_score_2501 >= 0  and x.model_score_2501 <= 10.047
#                                                 else '7.(10.047, 13.79]' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.归属渠道 != '抖音渠道' and x.归属渠道 != '广州易合顺' and x.归属渠道 != '深圳刘全兵号卡1' and x.是否拒量 ==0 and x.model_score_2501 > 10.047 and x.model_score_2501 <= 13.79
#                                                 else '8.(13.79,18.098]' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.归属渠道 != '抖音渠道' and x.归属渠道 != '广州易合顺' and x.归属渠道 != '深圳刘全兵号卡1' and x.是否拒量 ==0 and x.model_score_2501 > 13.79 and x.model_score_2501 <= 18.098
#                                                 else '9.(18.098,inf)' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.归属渠道 != '抖音渠道' and x.归属渠道 != '广州易合顺' and x.归属渠道 != '深圳刘全兵号卡1' and x.是否拒量 ==0 and x.model_score_2501 > 18.098

#                                                 else '91.[0,10.047]' if x.归属渠道 == '搜索渠道'  and x.是否拒量 ==0 and x.model_score_2501 >= 0  and x.model_score_2501 <= 10.047
#                                                 else '92.(10.047, 13.79]' if x.归属渠道 == '搜索渠道'  and x.是否拒量 ==0 and x.model_score_2501 > 10.047 and x.model_score_2501 <= 13.79
#                                                 else '93.(13.79,18.098]' if x.归属渠道 == '搜索渠道'  and x.是否拒量 ==0 and x.model_score_2501 > 13.79 and x.model_score_2501 <= 18.098
#                                                 else '94.(18.098,inf)' if x.归属渠道 == '搜索渠道'  and x.是否拒量 ==0 and x.model_score_2501 > 18.098
#                                                 else 'other'
#                                                 , axis = 1)




# df_info_ori['model_score_2311_Bin'] = df_info_ori.apply(lambda x: '1.[0,12.9620]' if x.归属渠道 == '芝麻租物'  and x.model_score_2311 >= 0  and x.model_score_2311 <= 12.9620
#                                                 else '2.(12.9620,15.2211]' if x.归属渠道 == '芝麻租物'  and x.model_score_2311 > 12.9620  and x.model_score_2311 <= 15.2211
#                                                 else '3.(15.2211,18.321]' if x.归属渠道 == '芝麻租物'  and x.model_score_2311 > 15.2211  and x.model_score_2311 <= 18.321
#                                                 else '4.(18.321, 23.297]' if x.归属渠道 == '芝麻租物'  and x.model_score_2311 > 18.321 and x.model_score_2311 <= 23.297
#                                                 else '5.(23.297, inf)' if x.归属渠道 == '芝麻租物'  and x.model_score_2311 > 23.297 

#                                                 else '6.[0,18.321]' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.model_score_2311 >= 0  and x.model_score_2311 <= 18.321
#                                                 else '7.(18.321, inf)' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.model_score_2311 > 18.321

#                                                 else '8.[0,16.6]' if x.归属渠道 == '搜索渠道'  and x.model_score_2311 >= 0  and x.model_score_2311 <= 16.6
#                                                 else '9.(16.6, 21.435]' if x.归属渠道 == '搜索渠道'  and x.model_score_2311 > 16.6 and x.model_score_2311 <= 21.435
#                                                 else '91.(21.435,inf)' if x.归属渠道 == '搜索渠道'  and x.model_score_2311 > 21.435
#                                                 else 'other'
#                                                 , axis = 1)


# df_info_ori['model_score_yl_Bin'] = df_info_ori.apply(lambda x: '1.[0,13.5137]' if x.归属渠道 == '芝麻租物'  and x.model_score_yl >= 0  and x.model_score_yl <= 13.5137
#                                                 else '2.(13.5137,14.9787]' if x.归属渠道 == '芝麻租物'  and x.model_score_yl > 13.5137  and x.model_score_yl <= 14.9787
#                                                 else '3.(14.9787,17.55]' if x.归属渠道 == '芝麻租物'  and x.model_score_yl > 14.9787  and x.model_score_yl <= 17.55
#                                                 else '4.(17.55,23.382]' if x.归属渠道 == '芝麻租物'  and x.model_score_yl > 17.55 and x.model_score_yl <= 23.382
#                                                 else '5.(23.382, inf)' if x.归属渠道 == '芝麻租物'  and x.model_score_yl > 23.382

#                                                 else '6.[0,17.55]' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.model_score_yl >= 0  and x.model_score_yl <= 17.55
#                                                 else '7.(17.55, inf)' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.model_score_yl > 17.55

#                                                 else '8.[0,10]' if x.归属渠道 == '搜索渠道'  and x.model_score_yl >= 0  and x.model_score_yl <= 10
#                                                 else '9.(10,15.059]' if x.归属渠道 == '搜索渠道'  and x.model_score_yl > 10  and x.model_score_yl <= 15.059
#                                                 else '91.(15.059, 21.661]' if x.归属渠道 == '搜索渠道'  and x.model_score_yl > 15.059 and x.model_score_yl <= 21.661
#                                                 else '92.(21.661,inf)' if x.归属渠道 == '搜索渠道'  and x.model_score_yl > 21.661
#                                                 else 'other'
#                                                 , axis = 1)

# df_info_ori['model_score_2401_Bin'] = df_info_ori.apply(lambda x: '1.[0,6.1907]' if x.归属渠道 == '芝麻租物'  and x.model_score_2401 >= 0  and x.model_score_2401 <= 6.1907
#                                                 else '2.(6.1907,8.1386]' if x.归属渠道 == '芝麻租物'  and x.model_score_2401 > 6.1907  and x.model_score_2401 <= 8.1386
#                                                 else '3.(8.1386,11.04]' if x.归属渠道 == '芝麻租物'  and x.model_score_2401 > 8.1386  and x.model_score_2401 <= 11.04
#                                                 else '4.(11.04, 16]' if x.归属渠道 == '芝麻租物'  and x.model_score_2401 > 11.04 and x.model_score_2401 <= 16 
#                                                 else '5.(16,inf)' if x.归属渠道 == '芝麻租物'  and x.model_score_2401 > 16

#                                                 else '6.[0,11.04]' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.model_score_2401 >= 0  and x.model_score_2401 <= 11.04
#                                                 else '7.(11.04,inf)' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.model_score_2401 > 11.04 

#                                                 else '8.[0,9.35]' if x.归属渠道 == '搜索渠道'  and x.model_score_2401 >= 0  and x.model_score_2401 <= 9.35
#                                                 else '9.(9.35, 14.93]' if x.归属渠道 == '搜索渠道'  and x.model_score_2401 > 9.35 and x.model_score_2401 <= 14.93
#                                                 else '91.(14.93,inf)' if x.归属渠道 == '搜索渠道'  and x.model_score_2401 > 14.93
#                                                 else 'other'
#                                                 , axis = 1)

# df_info_ori['model_score_2501_Bin'] = df_info_ori.apply(lambda x: '1.[0,10.047]' if x.归属渠道 == '芝麻租物'  and x.model_score_2501 >= 0 and x.model_score_2501 <= 10.047
#                                                 else '2.(10.047,11.635]' if x.归属渠道 == '芝麻租物'  and x.model_score_2501 > 10.047 and x.model_score_2501 <= 11.635
#                                                 else '3.(11.635,14.104]' if x.归属渠道 == '芝麻租物'  and x.model_score_2501 > 11.635 and x.model_score_2501 <= 14.104
#                                                 else '4.(14.104, 18.132]' if x.归属渠道 == '芝麻租物'  and x.model_score_2501 > 14.104 and x.model_score_2501 <= 18.132
#                                                 else '5.(18.132,inf)' if x.归属渠道 == '芝麻租物'  and x.model_score_2501 > 18.132

#                                                 else '6.[0,10.047]' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.model_score_2501 >= 0  and x.model_score_2501 <= 10.047
#                                                 else '7.(10.047, 13.79]' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.model_score_2501 > 10.047 and x.model_score_2501 <= 13.79
#                                                 else '8.(13.79,18.098]' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.model_score_2501 > 13.79 and x.model_score_2501 <= 18.098
#                                                 else '9.(18.098,inf)' if x.归属渠道 != '芝麻租物' and x.归属渠道 != '搜索渠道' and x.model_score_2501 > 18.098

#                                                 else '91.[0,10.047]' if x.归属渠道 == '搜索渠道'  and x.model_score_2501 >= 0  and x.model_score_2501 <= 10.047
#                                                 else '92.(10.047, 13.79]' if x.归属渠道 == '搜索渠道'  and x.model_score_2501 > 10.047 and x.model_score_2501 <= 13.79
#                                                 else '93.(13.79,18.098]' if x.归属渠道 == '搜索渠道'  and x.model_score_2501 > 13.79 and x.model_score_2501 <= 18.098
#                                                 else '94.(18.098,inf)' if x.归属渠道 == '搜索渠道'  and x.model_score_2501 > 18.098
#                                                 else 'other'
#                                                 , axis = 1)

# 评分等级
df_info_ori['grade_Bin'] = df_info_ori.apply(lambda x: '1.F' if x.grade == 'F'
                                             else '2.E' if x.grade == 'E'
                                             else '3.D' if x.grade == 'D'
                                             else '4.C' if x.grade == 'C'
                                             else '5.B' if x.grade == 'B'
                                             else '6.A' if x.grade == 'A'
                                             else '7.S' if x.grade == 'S'
                                             else '8.-999'
                                             , axis = 1)
# 策略240703_4
df_info_ori['strategy_240703_4_Bin'] = df_info_ori.apply(lambda x: '1.客群1' if x.model_score_2401 <= 14.27667 and x.jx_yl_UPPC134 > 1.5 and x.jx_yl_UPPC134 <= 3.5 and x.jx_yl_UPPD007 > 47
                                                         else '2.客群2' if x.model_score_2401 > 14.27667
                                                         else '3.客群3' if x.model_score_2401 <= 14.27667 and x.jx_yl_UPPC134 <= 1.5
                                                         else '4.客群4' if x.model_score_2401 <= 14.27667 and x.jx_yl_UPPC134 > 1.5 and x.jx_yl_UPPC134 <= 3.5 and x.jx_yl_UPPD007 <= 47
                                                         else '5.客群5' if x.model_score_2401 <= 14.27667 and x.jx_yl_UPPC134 > 3.5
                                                         else '6.-999', axis = 1)
# 策略240703_10
df_info_ori['strategy_240703_10_Bin'] = df_info_ori.apply(lambda x: '1.客群1' if x.model_score_2401 > 14.27667 and x.xy_qj_A22160001 <= 623.5 and x.xy_qj_B22170036 > 8.5
                                                          else '2.客群2' if x.model_score_2401 > 14.27667 and x.xy_qj_A22160001 <= 623.5 and x.xy_qj_B22170036 <= 8.5
                                                          else '3.客群3' if x.model_score_2401 > 14.27667 and x.xy_qj_A22160001 > 623.5
                                                          else '4.客群4' if x.model_score_2401 >= 0 and x.model_score_2401 <= 14.27667
                                                          else '5.-999', axis = 1)

# 策略240703_14
df_info_ori['strategy_240703_14_Bin'] = df_info_ori.apply(lambda x: '1.客群1' if x.model_score_2401 > 14.27667 and x.jx_yl_UPPE074 > 1.345 and x.my_zl_score > 18.324
                                                          else '2.客群2' if x.model_score_2401 > 14.27667 and x.jx_yl_UPPE074 > 1.345 and x.my_zl_score <= 18.324
                                                          else '3.客群3' if x.model_score_2401 > 14.27667 and x.jx_yl_UPPE074 <= 1.345
                                                          else '4.客群4' if x.model_score_2401 >= 0 and x.model_score_2401 <= 14.27667
                                                          else '5.-999', axis = 1)

# 融360 
df_info_ori['score_201048'] = df_info_ori['score_201048'].astype(float)
df_info_ori['score_202098'] = df_info_ori['score_202098'].astype(float)
df_info_ori['score_202100'] = df_info_ori['score_202100'].astype(float)

# df_info_ori['score_201048_Bin'] = df_info_ori.apply(lambda x: '1.[0,409]' if x.score_201048 >= 0 and x.score_201048 <= 409
#                                                     else '2.(409,424]' if x.score_201048 > 409 and x.score_201048 <= 424
#                                                     else '3.(424,517]' if x.score_201048 > 424 and x.score_201048 <= 517
#                                                     else '4.(517,543]' if x.score_201048 > 517 and x.score_201048 <= 543
#                                                     else '5.(543,588]' if x.score_201048 > 543 and x.score_201048 <= 588
#                                                     else '6.(588,inf)' if x.score_201048 > 588
#                                                     else '7.[-999,-999]', axis = 1)
df_info_ori['score_201048_Bin'] = df_info_ori.apply(lambda x : '1.[0,465]' if x.score_201048 >= 0 and x.score_201048 <= 465
                                                else '2.(465,491]' if x.score_201048 > 465 and x.score_201048 <= 491
                                                else '3.(491,510]' if x.score_201048 > 491 and x.score_201048 <= 510
                                                else '4.(510,526]' if x.score_201048 > 510 and x.score_201048 <= 526
                                                else '5.(526,540]' if x.score_201048 > 526 and x.score_201048 <= 540
                                                else '6.(540,555]' if x.score_201048 > 540 and x.score_201048 <= 555
                                                else '7.(555,571]' if x.score_201048 > 555 and x.score_201048 <= 571
                                                else '8.(571,589]' if x.score_201048 > 571 and x.score_201048 <= 589
                                                else '9.(589,613]' if x.score_201048 > 589 and x.score_201048 <= 613
                                                else '91.(613,inf)' if x.score_201048 > 613
                                                else '92.-999', axis = 1)

# df_info_ori['score_202098_Bin'] = df_info_ori.apply(lambda x: '1.[0,417]' if x.score_202098 >= 0 and x.score_202098 <= 417
#                                                     else '2.(417,470]' if x.score_202098 > 417 and x.score_202098 <= 470
#                                                     else '3.(470,511]' if x.score_202098 > 470 and x.score_202098 <= 511
#                                                     else '4.(511,546]' if x.score_202098 > 511 and x.score_202098 <= 546
#                                                     else '5.(546,602)' if x.score_202098 > 546 and x.score_202098 <= 602
#                                                     else '6.(602,inf)' if x.score_202098 > 602
#                                                     else '7.[-999,-999]', axis = 1)
df_info_ori['score_202098_Bin'] = df_info_ori.apply(lambda x : '1.[0,431]' if x.score_202098 >= 0 and x.score_202098 <= 431
                                                else '2.(431,456]' if x.score_202098 > 431 and x.score_202098 <= 456
                                                else '3.(456,474]' if x.score_202098 > 456 and x.score_202098 <= 474
                                                else '4.(474,490]' if x.score_202098 > 474 and x.score_202098 <= 490
                                                else '5.(490,506]' if x.score_202098 > 490 and x.score_202098 <= 506
                                                else '6.(506,523]' if x.score_202098 > 506 and x.score_202098 <= 523
                                                else '7.(523,542]' if x.score_202098 > 523 and x.score_202098 <= 542
                                                else '8.(542,564]' if x.score_202098 > 542 and x.score_202098 <= 564
                                                else '9.(564,595]' if x.score_202098 > 564 and x.score_202098 <= 595
                                                else '91.(595,inf)' if x.score_202098 > 595
                                                else '92.-999', axis = 1)

# df_info_ori['score_202100_Bin'] = df_info_ori.apply(lambda x: '1.[0,387]' if x.score_202100 >= 0 and x.score_202100 <= 387
#                                                     else '2.(387,405]' if x.score_202100 > 387 and x.score_202100 <= 405
#                                                     else '3.(405,440]' if x.score_202100 > 405 and x.score_202100 <= 440
#                                                     else '4.(440,477]' if x.score_202100 > 440 and x.score_202100 <= 477
#                                                     else '5.(477,524]' if x.score_202100 > 477 and x.score_202100 <= 524
#                                                     else '6.(524,inf)' if x.score_202100 > 524
#                                                     else '7.[-999,-999]', axis = 1)
df_info_ori['score_202100_Bin'] = df_info_ori.apply(lambda x : '1.[0,442]' if x.score_202100 >= 0 and x.score_202100 <= 442
                                                else '2.(442,466]' if x.score_202100 > 442 and x.score_202100 <= 466
                                                else '3.(466,484]' if x.score_202100 > 466 and x.score_202100 <= 484
                                                else '4.(484,500]' if x.score_202100 > 484 and x.score_202100 <= 500
                                                else '5.(500,515]' if x.score_202100 > 500 and x.score_202100 <= 515
                                                else '6.(515,531]' if x.score_202100 > 515 and x.score_202100 <= 531
                                                else '7.(531,548]' if x.score_202100 > 531 and x.score_202100 <= 548
                                                else '8.(548,568]' if x.score_202100 > 548 and x.score_202100 <= 568
                                                else '9.(568,597]' if x.score_202100 > 568 and x.score_202100 <= 597
                                                else '91.(597,inf)' if x.score_202100 > 597
                                                else '92.-999', axis = 1)

In [121]:
# jx_yl_UPPC134   近1个月出账类交易频度
df_info_ori['jx_yl_UPPC134_Bin'] = df_info_ori.apply(lambda x : '1.[0,1.5]'  if x.jx_yl_UPPC134 >= 0 and x.jx_yl_UPPC134 <= 1.5
                                                  else '2.(1.5,3.5]' if x.jx_yl_UPPC134  > 1.5 and x.jx_yl_UPPC134  <= 3.5
                                                  else '3.(3.5,6.5]' if x.jx_yl_UPPC134  > 3.5 and x.jx_yl_UPPC134  <= 6.5
                                                  else '4.(6.5,10.0]' if x.jx_yl_UPPC134  > 6.5 and x.jx_yl_UPPC134  <= 10
                                                  else '5.(10,15.0]' if x.jx_yl_UPPC134  > 10 and x.jx_yl_UPPC134  <= 15
                                                  else '6.(15,21.0]' if x.jx_yl_UPPC134  > 15 and x.jx_yl_UPPC134  <= 21
                                                  else '7.(21,28.0]' if x.jx_yl_UPPC134  > 21 and x.jx_yl_UPPC134  <= 28
                                                  else '8.(28,38.0]' if x.jx_yl_UPPC134  > 28 and x.jx_yl_UPPC134  <= 38
                                                  else '9.(38,56.0]' if x.jx_yl_UPPC134  > 38 and x.jx_yl_UPPC134  <= 56
                                                  else '91.(56,inf)' if x.jx_yl_UPPC134  > 56
                                                  else '92.-999', axis = 1)
# jx_yl_UPPD007	近6个月失败交易频度
df_info_ori['jx_yl_UPPD007_Bin'] = df_info_ori.apply(lambda x : '91.[0,5]'  if x.jx_yl_UPPD007 >= 0 and x.jx_yl_UPPD007 <= 5
                                                  else '9.(5,11]' if x.jx_yl_UPPD007  > 5 and x.jx_yl_UPPD007  <= 11
                                                  else '8.(11,17]' if x.jx_yl_UPPD007  > 11 and x.jx_yl_UPPD007  <= 17
                                                  else '7.(17,26.0]' if x.jx_yl_UPPD007  > 17 and x.jx_yl_UPPD007  <= 26
                                                  else '6.(26,38.0]' if x.jx_yl_UPPD007  > 26 and x.jx_yl_UPPD007  <= 38
                                                  else '5.(38,47.0]' if x.jx_yl_UPPD007  > 38 and x.jx_yl_UPPD007  <= 47
                                                  else '4.(47,77.0]' if x.jx_yl_UPPD007  > 47 and x.jx_yl_UPPD007  <= 77
                                                  else '3.(77,125.0]' if x.jx_yl_UPPD007  > 77 and x.jx_yl_UPPD007  <= 125
                                                  else '2.(125,236.0]' if x.jx_yl_UPPD007  > 125 and x.jx_yl_UPPD007  <= 236
                                                  else '1.(236,inf)' if x.jx_yl_UPPD007  > 236
                                                  else '92.-999', axis = 1)

# jx_yl_UPPE074	近12个月平均代扣失败天数
df_info_ori['jx_yl_UPPE074_Bin'] = df_info_ori.apply(lambda x : '6.[0,0.82]'  if x.jx_yl_UPPE074 >= 0 and x.jx_yl_UPPE074 <= 0.82
                                                  else '5.(0.82,1)' if x.jx_yl_UPPE074 > 0.82 and x.jx_yl_UPPE074 < 1
                                                  else '4.1' if x.jx_yl_UPPE074 == 1
                                                  else '3.(1,1.345]' if x.jx_yl_UPPE074 > 1 and x.jx_yl_UPPE074 <= 1.345
                                                  else '2.(1.345,2.31]' if x.jx_yl_UPPE074 > 1.345 and x.jx_yl_UPPE074 <= 2.31
                                                  else '1.(2.31,inf)' if x.jx_yl_UPPE074 > 2.31
                                                  else '8.-999', axis = 1)

# jx_yl_UPPE082	近12个月成功扣款频度占比
df_info_ori['jx_yl_UPPE082_Bin'] = df_info_ori.apply(lambda x : '1.[0,0.15]'  if x.jx_yl_UPPE082 >= 0 and x.jx_yl_UPPE082 <= 0.15
                                                  else '2.(0.15,0.3]' if x.jx_yl_UPPE082  > 0.15 and x.jx_yl_UPPE082  <= 0.3
                                                  else '3.(0.3,0.45]' if x.jx_yl_UPPE082  > 0.3 and x.jx_yl_UPPE082  <= 0.45
                                                  else '4.(0.45,0.55]' if x.jx_yl_UPPE082  > 0.45 and x.jx_yl_UPPE082  <= 0.55
                                                  else '5.(0.55,0.655]' if x.jx_yl_UPPE082  > 0.55 and x.jx_yl_UPPE082  <= 0.655
                                                  else '6.(0.655,0.81]' if x.jx_yl_UPPE082  > 0.655 and x.jx_yl_UPPE082  <= 0.81
                                                  else '7.(0.81,0.95]' if x.jx_yl_UPPE082  > 0.81 and x.jx_yl_UPPE082  <= 0.95
                                                  else '8.(0.95,inf)' if x.jx_yl_UPPE082  > 0.95
                                                  else '9.-999', axis = 1)

# xy_qj_C22180008	消⾦贷款类产品数
df_info_ori['xy_qj_C22180008_Bin'] = df_info_ori.apply(lambda x : '7.[0,2]' if x.xy_qj_C22180008 >= 0 and x.xy_qj_C22180008 <= 2
                                                  else '6.(2,5]' if x.xy_qj_C22180008 > 2 and x.xy_qj_C22180008 <= 5
                                                  else '5.(5,10.5]' if x.xy_qj_C22180008 > 5 and x.xy_qj_C22180008 <= 10.5
                                                  else '4.(10.5,15]' if x.xy_qj_C22180008 > 10.5 and x.xy_qj_C22180008 <= 15
                                                  else '3.(15,23]' if x.xy_qj_C22180008 > 15 and x.xy_qj_C22180008 <= 23
                                                  else '2.(23,36]' if x.xy_qj_C22180008 > 23 and x.xy_qj_C22180008 <= 36
                                                  else '1.(36,inf)' if x.xy_qj_C22180008 > 36
                                                  else '8.-999', axis = 1)

# xy_qj_B22170035	近1个⽉失败扣款笔数
df_info_ori['xy_qj_B22170035_Bin'] = df_info_ori.apply(lambda x : '6.[0,1]' if x.xy_qj_B22170035 >= 0 and x.xy_qj_B22170035 <= 1
                                                  else '5.(1,2]' if x.xy_qj_B22170035 > 1 and x.xy_qj_B22170035 <= 2
                                                  else '4.(2,4.5]' if x.xy_qj_B22170035 > 2 and x.xy_qj_B22170035 <= 4.5
                                                  else '3.(4.5,7]' if x.xy_qj_B22170035 > 4.5 and x.xy_qj_B22170035 <= 7
                                                  else '2.(7,15]' if x.xy_qj_B22170035 > 7 and x.xy_qj_B22170035 <= 15
                                                  else '1.(15,inf)' if x.xy_qj_B22170035 > 15
                                                  else '7.-999', axis = 1)

# xy_qj_B22170036	近3个⽉失败扣款笔数
df_info_ori['xy_qj_B22170036_Bin'] = df_info_ori.apply(lambda x : '7.[0,1]' if x.xy_qj_B22170036 >= 0 and x.xy_qj_B22170036 <= 1
                                                  else '6.(1,2]' if x.xy_qj_B22170036 > 1 and x.xy_qj_B22170036 <= 2
                                                  else '5.(2,4]' if x.xy_qj_B22170036 > 2 and x.xy_qj_B22170036 <= 4
                                                  else '4.(4,8.5]' if x.xy_qj_B22170036 > 4 and x.xy_qj_B22170036 <= 8.5
                                                  else '3.(8.5,16]' if x.xy_qj_B22170036 > 8.5 and x.xy_qj_B22170036 <= 16
                                                  else '2.(16,31]' if x.xy_qj_B22170036 > 16 and x.xy_qj_B22170036 <= 31
                                                  else '1.(31,inf)' if x.xy_qj_B22170036 > 31
                                                  else '91.-999', axis = 1)
# xy_qj_A22160001	申请准⼊分
df_info_ori['xy_qj_A22160001_Bin'] = df_info_ori.apply(lambda x : '1.[0,477]' if x.xy_qj_A22160001 >= 0 and x.xy_qj_A22160001 <= 477
                                                  else '2.(477,501]' if x.xy_qj_A22160001 > 477 and x.xy_qj_A22160001 <= 501
                                                  else '3.(501,520]' if x.xy_qj_A22160001 > 501 and x.xy_qj_A22160001 <= 520
                                                  else '4.(520,534]' if x.xy_qj_A22160001 > 520 and x.xy_qj_A22160001 <= 534
                                                  else '5.(534,543]' if x.xy_qj_A22160001 > 534 and x.xy_qj_A22160001 <= 543
                                                  else '6.(543,580]' if x.xy_qj_A22160001 > 543 and x.xy_qj_A22160001 <= 580
                                                  else '7.(580,621]' if x.xy_qj_A22160001 > 580 and x.xy_qj_A22160001 <= 621
                                                  else '8.(621,622]' if x.xy_qj_A22160001 > 621 and x.xy_qj_A22160001 <= 622
                                                  else '9.(622,623.5]' if x.xy_qj_A22160001 > 622 and x.xy_qj_A22160001 <= 623.5
                                                  else '91.(623.5,inf)' if x.xy_qj_A22160001 > 623.5
                                                  else '92.-999', axis = 1)

## 7_2_2.机审2

In [122]:
# 策略240801
# df_info_ori['jx_fico_score_Bin'] = df_info_ori.apply(lambda x : '1.[0,489.5]' if x.jx_fico_score >= 0 and x.jx_fico_score <= 489.5 
#                                                 else '2.(489.5,537]' if x.jx_fico_score > 489.5  and x.jx_fico_score <= 537
#                                                 else '3.(537,559]' if x.jx_fico_score > 537 and x.jx_fico_score <= 559
#                                                 else '4.(559,588]' if x.jx_fico_score > 559 and x.jx_fico_score <= 588
#                                                 else '5.(588,604.5]' if x.jx_fico_score > 588 and x.jx_fico_score <= 604.5
#                                                 else '6.(604.5,622]' if x.jx_fico_score > 604.5 and x.jx_fico_score <= 622
#                                                 else '7.(622,642]' if x.jx_fico_score > 622 and x.jx_fico_score <= 642
#                                                 else '8.(642,664]' if x.jx_fico_score > 642 and x.jx_fico_score <= 664
#                                                 else '9.(664,692]' if x.jx_fico_score > 664 and x.jx_fico_score <= 692
#                                                 else '91.(692,inf)' if x.jx_fico_score > 692
#                                                 else '92.-999', axis = 1)
# reject
df_info_ori['strategy_240801_1_Bin'] = df_info_ori.apply(lambda x: '1.客群1' if x.jx_fico_score <= 561.5 and x.model_score_2311 > 17.684
                                                          else '2.客群2' if x.jx_fico_score <= 561.5 and x.model_score_2311 <= 17.684
                                                          else '3.客群3' if x.jx_fico_score > 561.5
                                                          else '4.-999', axis = 1)

df_info_ori['strategy_240801_2_Bin'] = df_info_ori.apply(lambda x: '1.客群1' if x.jx_fico_score <= 561.5 and x.model_score_yl > 21.6
                                                          else '2.客群2' if x.jx_fico_score <= 561.5 and x.model_score_yl <= 21.6
                                                          else '3.客群3' if x.jx_fico_score > 561.5
                                                          else '4.-999', axis = 1)

df_info_ori['strategy_240801_3_Bin'] = df_info_ori.apply(lambda x: '1.客群1' if x.jx_fico_score <= 561.5 and x.model_score_2401 > 14.326
                                                          else '2.客群2' if x.jx_fico_score <= 561.5 and x.model_score_2401 <= 14.326
                                                          else '3.客群3' if x.jx_fico_score > 561.5
                                                          else '4.-999', axis = 1)

df_info_ori['strategy_240801_4_Bin'] = df_info_ori.apply(lambda x: '1.客群1' if x.jx_fico_score <= 561.5 and x.my_zl_score > 20.885
                                                          else '2.客群2' if x.jx_fico_score <= 561.5 and x.my_zl_score <= 20.885
                                                          else '3.客群3' if x.jx_fico_score > 561.5
                                                          else '4.-999', axis = 1)

df_info_ori['strategy_240801_5_Bin'] = df_info_ori.apply(lambda x: '1.客群1' if x.xy_qj_C22180008 <= 10.5 and x.xy_qj_B22170035 > 4.5
                                                          else '2.客群2' if x.xy_qj_C22180008 <= 10.5 and x.xy_qj_B22170035 <= 4.5
                                                          else '3.客群3' if x.xy_qj_C22180008 > 10.5
                                                          else '4.-999', axis = 1)

# pass
df_info_ori['strategy_240801_7_Bin'] = df_info_ori.apply(lambda x: '3.客群3' if x.jx_fico_score > 604.5 and x.jx_yl_UPPE082 > 0.655
                                                          else '2.客群2' if x.jx_fico_score > 604.5 and x.jx_yl_UPPE082 <= 0.655
                                                          else '1.客群1' if x.jx_fico_score <= 604.5
                                                          else '4.-999', axis = 1)

df_info_ori['strategy_240801_8_Bin'] = df_info_ori.apply(lambda x: '3.客群3' if x.jx_fico_score > 604.5 and x.model_score_2401 <= 8.093
                                                          else '2.客群2' if x.jx_fico_score > 604.5 and x.model_score_2401 > 8.093
                                                          else '1.客群1' if x.jx_fico_score <= 604.5
                                                          else '4.-999', axis = 1)

df_info_ori['strategy_240801_9_Bin'] = df_info_ori.apply(lambda x: '3.客群3' if x.jx_fico_score > 561.5 and x.model_score_2401 <= 9.849
                                                          else '2.客群2' if x.jx_fico_score > 561.5 and x.model_score_2401 > 9.849
                                                          else '1.客群1' if x.jx_fico_score <= 561.5
                                                          else '4.-999', axis = 1)

df_info_ori['strategy_240801_10_Bin'] = df_info_ori.apply(lambda x: '3.客群3' if x.jx_fico_score > 561.5 and x.bj_qy_score > 433.5
                                                          else '2.客群2' if x.jx_fico_score > 561.5 and x.bj_qy_score <= 433.5
                                                          else '1.客群1' if x.jx_fico_score <= 561.5
                                                          else '4.-999', axis = 1)

df_info_ori['strategy_240801_11_Bin'] = df_info_ori.apply(lambda x: '3.客群3' if x.model_score_2401 <= 14.326 and x.xy_qj_C22180008 > 10.5
                                                          else '2.客群2' if x.model_score_2401 <= 14.326 and x.xy_qj_C22180008 <= 10.5
                                                          else '1.客群1' if x.model_score_2401 > 14.326
                                                          else '4.-999', axis = 1)

## 7_3.出库前强拒

In [123]:
# （7-3）出库前强拒
# 规则37		蚂蚁数控风险等级=998强拒
# 规则38		蚂蚁数控风险等级=996强拒且非芝麻租物
# 规则39		蚂蚁数控风险等级=979强拒且非芝麻租物非搜索渠道
# 规则40		fico_S50c067信用分<537
# 规则41		非单人话且非扫一扫渠道：命中fico_S50c067信用分 > 489.5 且 fico_S50c067信用分 <= 537 且青云分 > 450 随机40% 豁免
# 规则42		银联模型ascore_yl2311分>17.55且fico_S50c067信用分 >537 且 fico_S50c067信用分 <=559，命中强拒  但青云分 > 450的话 随机40% 豁免该规则
# 规则43	   strategy_240927	"芝麻租物渠道： 1. 腾云212>98拒绝；
                                        # 2. 履约保护>26拒绝；（关系为“或”，命中其中一个拒绝）

                                # 搜索渠道： 1. 腾云212>98拒绝；
                                       # 2. 履约保护>26拒绝；（关系为“或”，命中其中一个拒绝）

                                # 其他渠道的： 1. 腾云212>95拒绝;
                                # 2. 履约保护>25拒绝; （关系为“或”，命中其中一个拒绝）"

# 0 为青云分豁免或回捞的单（有青云分<450的，但集中在2024.4及之前）,1表示强拒，2 表示命中但40%随机豁免，3 表示通过，4 表示通过且 jx_fico_score > 559，5 为jx_fico_score缺失
df_info_ori['规则42'] = df_info_ori.apply(lambda x: 0 if x.jx_fico_score >= 0 and x.jx_fico_score <= 537
                                        else 1 if x.jx_fico_score > 537 and x.jx_fico_score <= 559 and x.model_score_yl > 17.55 and  x.bj_qy_score <= 450
                                        else 2 if x.jx_fico_score > 537 and x.jx_fico_score <= 559 and x.model_score_yl > 17.55 and x.bj_qy_score > 450
                                        else 3 if x.jx_fico_score > 537 and x.jx_fico_score <= 559 and x.model_score_yl <= 17.55
                                        else 4 if x.jx_fico_score > 559
                                        else 5, axis = 1)
# 根据应用建议分组， nsf_rental_v1_3_hundred_Bin 为根据数据表现分组
# df_info_ori['履约保护_Bin'] = df_info_ori.apply(lambda x: '91.[0,1]' if x.nsf_rental_v1_3_hundred >= 0 and x.nsf_rental_v1_3_hundred <= 1
#                                             else '9.2' if x.nsf_rental_v1_3_hundred == 2
#                                             else '8.3' if x.nsf_rental_v1_3_hundred == 3
#                                             else '7.[4,7]' if x.nsf_rental_v1_3_hundred >= 4 and x.nsf_rental_v1_3_hundred <= 7
#                                             else '6.[8,9]' if x.nsf_rental_v1_3_hundred >= 8 and x.nsf_rental_v1_3_hundred <= 9
#                                             else '5.[10,25]' if x.nsf_rental_v1_3_hundred >= 10 and x.nsf_rental_v1_3_hundred <= 25
#                                             else '4.[26,30]' if x.nsf_rental_v1_3_hundred >= 26 and x.nsf_rental_v1_3_hundred <= 30
#                                             else '3.[31,95]' if x.nsf_rental_v1_3_hundred >= 31 and x.nsf_rental_v1_3_hundred <= 95
#                                             else '2.[96,98]' if x.nsf_rental_v1_3_hundred >= 96 and x.nsf_rental_v1_3_hundred <= 98
#                                             else '1.(98,inf)' if x.nsf_rental_v1_3_hundred > 98
#                                             else '92.-999', axis = 1)
df_info_ori['履约保护_Bin'] = df_info_ori.apply(lambda x : '91.[0,7]' if x.nsf_rental_v1_3_hundred >= 0 and x.nsf_rental_v1_3_hundred <= 7
                                                  else '9.(7,8]' if x.nsf_rental_v1_3_hundred > 7  and x.nsf_rental_v1_3_hundred <= 8
                                                  else '8.(8,9]' if x.nsf_rental_v1_3_hundred > 8 and x.nsf_rental_v1_3_hundred <= 9
                                                  else '7.(9,10]' if x.nsf_rental_v1_3_hundred > 9 and x.nsf_rental_v1_3_hundred <= 10
                                                  else '6.(10,11]' if x.nsf_rental_v1_3_hundred > 10 and x.nsf_rental_v1_3_hundred <= 11
                                                  else '5.(11,12]' if x.nsf_rental_v1_3_hundred > 11 and x.nsf_rental_v1_3_hundred <= 12
                                                  else '4.(12,14]' if x.nsf_rental_v1_3_hundred > 12 and x.nsf_rental_v1_3_hundred <= 14
                                                  else '3.(14,16]' if x.nsf_rental_v1_3_hundred > 14 and x.nsf_rental_v1_3_hundred <= 16
                                                  else '2.(16,25]' if x.nsf_rental_v1_3_hundred > 16 and x.nsf_rental_v1_3_hundred <= 25
                                                  else '1.(25,inf)' if x.nsf_rental_v1_3_hundred > 25
                                                  else '92.-999', axis = 1)

df_info_ori['td212_score'] = df_info_ori['td212_score'].astype(float)
# df_info_ori['td212_Bin'] = df_info_ori.apply(lambda x: '1.(98,inf)' if x.td212_score > 98
#                                              else '2.(95,98]' if x.td212_score > 95 and x.td212_score <= 98
#                                              else '3.(90,95]' if x.td212_score > 90 and x.td212_score <= 95
#                                              else '4.(50,90]' if x.td212_score > 50 and x.td212_score <= 90
#                                              else '5.[0,50]' if x.td212_score >= 0 and x.td212_score <= 50
#                                              else '6.-999', axis = 1)
df_info_ori['td212_Bin'] = df_info_ori.apply(lambda x : '91.[0,14.24]' if x.td212_score >= 0 and x.td212_score <= 14.24
                                                  else '9.(14.24 ,21.34]' if x.td212_score > 14.24  and x.td212_score <= 21.34
                                                  else '8.(21.34,26.34]' if x.td212_score > 21.34 and x.td212_score <= 26.34
                                                  else '7.(26.34,30.65]' if x.td212_score > 26.34 and x.td212_score <= 30.65
                                                  else '6.(30.65,34.46]' if x.td212_score > 30.65 and x.td212_score <= 34.46
                                                  else '5.(34.46,38.19]' if x.td212_score > 34.46 and x.td212_score <= 38.19
                                                  else '4.(38.19,41.99]' if x.td212_score > 38.19 and x.td212_score <= 41.99
                                                  else '3.(41.99,48.83]' if x.td212_score > 41.99 and x.td212_score <= 48.83
                                                  else '2.(48.83,95]' if x.td212_score > 48.83 and x.td212_score <= 95
                                                  else '1.(95,inf)' if x.td212_score > 95
                                                  else '92.-999', axis = 1)

df_info_ori['zm_jh_t0_level_Bin'] = df_info_ori.apply(lambda x: '1.998' if x.zm_jh_t0_level == '998'
                                                      else '2.996' if x.zm_jh_t0_level == '996'
                                                      else '3.979' if x.zm_jh_t0_level == '979'
                                                      else '4.975' if x.zm_jh_t0_level == '975'
                                                      else '5.969' if x.zm_jh_t0_level == '969'
                                                      else '6.989' if x.zm_jh_t0_level == '989'
                                                      else '7.-999'
                                                      , axis = 1)

# df_info_ori['FICO_Bin'] = df_info_ori.apply(lambda x: '1.[0,489]' if x.jx_fico_score >= 0  and x.jx_fico_score <= 489
#                                             else '2.(489,536]' if x.jx_fico_score > 489 and x.jx_fico_score <= 536
#                                             else '3.(536,563]' if x.jx_fico_score > 536 and x.jx_fico_score <= 563
#                                             else '4.(563,587]' if x.jx_fico_score > 563 and x.jx_fico_score <= 587
#                                             else '5.(587,617]' if x.jx_fico_score > 587 and x.jx_fico_score <= 617
#                                             else '6.(617,632]' if x.jx_fico_score > 617 and x.jx_fico_score <= 632
#                                             else '7.(632,672]' if x.jx_fico_score > 632 and x.jx_fico_score <= 672
#                                             else '8.(672,inf)' if x.jx_fico_score > 672
#                                             else '9.-999', axis=1)
df_info_ori['FICO_Bin'] = df_info_ori.apply(lambda x : '1.[0,489.5]' if x.jx_fico_score >= 0 and x.jx_fico_score <= 489.5 
                                                else '2.(489.5,537]' if x.jx_fico_score > 489.5  and x.jx_fico_score <= 537
                                                else '3.(537,559]' if x.jx_fico_score > 537 and x.jx_fico_score <= 559
                                                else '4.(559,588]' if x.jx_fico_score > 559 and x.jx_fico_score <= 588
                                                else '5.(588,604.5]' if x.jx_fico_score > 588 and x.jx_fico_score <= 604.5
                                                else '6.(604.5,622]' if x.jx_fico_score > 604.5 and x.jx_fico_score <= 622
                                                else '7.(622,642]' if x.jx_fico_score > 622 and x.jx_fico_score <= 642
                                                else '8.(642,664]' if x.jx_fico_score > 642 and x.jx_fico_score <= 664
                                                else '9.(664,692]' if x.jx_fico_score > 664 and x.jx_fico_score <= 692
                                                else '91.(692,inf)' if x.jx_fico_score > 692
                                                else '92.-999', axis = 1)

# # td212_score
# df_info_ori['td212_score_Bin'] = df_info_ori.apply(lambda x : '91.[0,14.24]' if x.td212_score >= 0 and x.td212_score <= 14.24
#                                                   else '9.(14.24 ,21.34]' if x.td212_score > 14.24  and x.td212_score <= 21.34
#                                                   else '8.(21.34,26.34]' if x.td212_score > 21.34 and x.td212_score <= 26.34
#                                                   else '7.(26.34,30.65]' if x.td212_score > 26.34 and x.td212_score <= 30.65
#                                                   else '6.(30.65,34.46]' if x.td212_score > 30.65 and x.td212_score <= 34.46
#                                                   else '5.(34.46,38.19]' if x.td212_score > 34.46 and x.td212_score <= 38.19
#                                                   else '4.(38.19,41.99]' if x.td212_score > 38.19 and x.td212_score <= 41.99
#                                                   else '3.(41.99,48.83]' if x.td212_score > 41.99 and x.td212_score <= 48.83
#                                                   else '2.(48.83,95]' if x.td212_score > 48.83 and x.td212_score <= 95
#                                                   else '1.(95,inf)' if x.td212_score > 95
#                                                   else '92.-999', axis = 1)

# # 履约保护
# df_info_ori['nsf_rental_v1_3_hundred_Bin'] = df_info_ori.apply(lambda x : '91.[0,7]' if x.nsf_rental_v1_3_hundred >= 0 and x.nsf_rental_v1_3_hundred <= 7
#                                                   else '9.(7,8]' if x.nsf_rental_v1_3_hundred > 7  and x.nsf_rental_v1_3_hundred <= 8
#                                                   else '8.(8,9]' if x.nsf_rental_v1_3_hundred > 8 and x.nsf_rental_v1_3_hundred <= 9
#                                                   else '7.(9,10]' if x.nsf_rental_v1_3_hundred > 9 and x.nsf_rental_v1_3_hundred <= 10
#                                                   else '6.(10,11]' if x.nsf_rental_v1_3_hundred > 10 and x.nsf_rental_v1_3_hundred <= 11
#                                                   else '5.(11,12]' if x.nsf_rental_v1_3_hundred > 11 and x.nsf_rental_v1_3_hundred <= 12
#                                                   else '4.(12,14]' if x.nsf_rental_v1_3_hundred > 12 and x.nsf_rental_v1_3_hundred <= 14
#                                                   else '3.(14,16]' if x.nsf_rental_v1_3_hundred > 14 and x.nsf_rental_v1_3_hundred <= 16
#                                                   else '2.(16,25]' if x.nsf_rental_v1_3_hundred > 16 and x.nsf_rental_v1_3_hundred <= 25
#                                                   else '1.(25,inf)' if x.nsf_rental_v1_3_hundred > 25
#                                                   else '92.-999', axis = 1)

## 模型监控

In [124]:
df_info_ori['model_score_2311_Bin'] = df_info_ori.apply(lambda x : '91.[0,14.374]' if x.model_score_2311 >= 0 and x.model_score_2311 <= 14.374 
                                                else '9.(14.374,16.299]' if x.model_score_2311 > 14.374  and x.model_score_2311 <= 16.299
                                                else '8.(16.299,17.669]' if x.model_score_2311 > 16.299 and x.model_score_2311 <= 17.669
                                                else '7.(17.669,19.989]' if x.model_score_2311 > 17.669 and x.model_score_2311 <= 19.989
                                                else '6.(19.989,20.787]' if x.model_score_2311 > 19.989 and x.model_score_2311 <= 20.787
                                                else '5.(20.787,22.754]' if x.model_score_2311 > 20.787 and x.model_score_2311 <= 22.754
                                                else '4.(22.754,24.404]' if x.model_score_2311 > 22.754 and x.model_score_2311 <= 24.404
                                                else '3.(24.404,25.065]' if x.model_score_2311 > 24.404 and x.model_score_2311 <= 25.065
                                                else '2.(25.065,27.295]' if x.model_score_2311 > 25.065 and x.model_score_2311 <= 27.295
                                                else '1.(27.295,inf)' if x.model_score_2311 > 27.295
                                                else '92.-999', axis = 1)

df_info_ori['model_score_yl_Bin'] = df_info_ori.apply(lambda x : '91.[0,12.142]' if x.model_score_yl >= 0 and x.model_score_yl <= 12.142 
                                                else '9.(12.142,14.524]' if x.model_score_yl > 12.142  and x.model_score_yl <= 14.524
                                                else '8.(14.524,16.515]' if x.model_score_yl > 14.524 and x.model_score_yl <= 16.515
                                                else '7.(16.515,18.653]' if x.model_score_yl > 16.515 and x.model_score_yl <= 18.653
                                                else '6.(18.653,20.508]' if x.model_score_yl > 18.653 and x.model_score_yl <= 20.508
                                                else '5.(20.508,22.376]' if x.model_score_yl > 20.508 and x.model_score_yl <= 22.376
                                                else '4.(22.376,24.748]' if x.model_score_yl > 22.376 and x.model_score_yl <= 24.748
                                                else '3.(24.748,27.666]' if x.model_score_yl > 24.748 and x.model_score_yl <= 27.666
                                                else '2.(27.666,30.907]' if x.model_score_yl > 27.666 and x.model_score_yl <= 30.907
                                                else '1.(30.907,inf)' if x.model_score_yl > 30.907
                                                else '92.-999', axis = 1)

df_info_ori['model_score_2401_Bin'] = df_info_ori.apply(lambda x : '91.[0,6.899]' if x.model_score_2401 >= 0 and x.model_score_2401 <= 6.899 
                                                else '9.(6.899,8.786]' if x.model_score_2401 > 6.899  and x.model_score_2401 <= 8.786
                                                else '8.(8.786,10.028]' if x.model_score_2401 > 8.786 and x.model_score_2401 <= 10.028
                                                else '7.(10.028,12.388]' if x.model_score_2401 > 10.028 and x.model_score_2401 <= 12.388
                                                else '6.(12.388,13.485]' if x.model_score_2401 > 12.388 and x.model_score_2401 <= 13.485
                                                else '5.(13.485,15.787]' if x.model_score_2401 > 13.485 and x.model_score_2401 <= 15.787
                                                else '4.(15.787,17.466]' if x.model_score_2401 > 15.787 and x.model_score_2401 <= 17.466
                                                else '3.(17.466,18.845]' if x.model_score_2401 > 17.466 and x.model_score_2401 <= 18.845
                                                else '2.(18.845,21.449]' if x.model_score_2401 > 18.845 and x.model_score_2401 <= 21.449
                                                else '1.(21.449,inf)' if x.model_score_2401 > 21.449
                                                else '92.-999', axis = 1)

df_info_ori['model_score_2501_Bin'] = df_info_ori.apply(lambda x : '91.[0,10.375]' if x.model_score_2501 >= 0 and x.model_score_2501 <= 10.375 
                                                else '9.(10.375,11.932]' if x.model_score_2501 > 10.375  and x.model_score_2501 <= 11.932
                                                else '8.(11.932,13.268]' if x.model_score_2501 > 11.932 and x.model_score_2501 <= 13.268
                                                else '7.(13.268,14.536]' if x.model_score_2501 > 13.268 and x.model_score_2501 <= 14.536
                                                else '6.(14.536,15.896]' if x.model_score_2501 > 14.536 and x.model_score_2501 <= 15.896
                                                else '5.(15.896,17.425]' if x.model_score_2501 > 15.896 and x.model_score_2501 <= 17.425
                                                else '4.(17.425,19.216]' if x.model_score_2501 > 17.425 and x.model_score_2501 <= 19.216
                                                else '3.(19.216,21.648]' if x.model_score_2501 > 19.216 and x.model_score_2501 <= 21.648
                                                else '2.(21.648,25.345]' if x.model_score_2501 > 21.648 and x.model_score_2501 <= 25.345
                                                else '1.(25.345,inf)' if x.model_score_2501 > 25.345
                                                else '92.-999', axis = 1)

# 8.分箱-数据驱动

In [125]:
# （8-1-1）宽表-区分度
df_analy_ks = df_info_ori[df_info_ori.y_credit.isin([0,1])]
df_analy_ks.shape

# 字符型
# xy_qj_A22160007 ： 最近⼀次查询时间
# xy_qj_B22170054 ： 最近⼀次贷款放款时间
# xy_qj_B22170007	近1个⽉贷款总⾦额
# xy_qj_B22170008	近3个⽉贷款总⾦额
# xy_qj_B22170009	近6个⽉贷款总⾦额
# xy_qj_B22170010	近12个⽉贷款总⾦额
# xy_qj_B22170011	近24个⽉贷款总⾦额
# xy_qj_B22170031	近6个⽉累计逾期⾦额参考⾦额区间
# xy_qj_B22170032	近12个⽉累计逾期⾦额参考⾦额区间
# xy_qj_B22170033	近24个⽉累计逾期⾦额参考⾦额区间
# xy_qj_B22170040	近1个⽉履约贷款总⾦额参考⾦额区间	object
# xy_qj_B22170041	近3个⽉履约贷款总⾦额参考⾦额区间	object
# xy_qj_B22170042	近6个⽉履约贷款总⾦额参考⾦额区间	object
# xy_qj_B22170043	近12个⽉履约贷款总⾦额参考⾦额区间	object
# xy_qj_B22170044	近24个⽉履约贷款总⾦额参考⾦额区间	object
# xy_qj_B22170050	最近⼀次履约距今天数参考天数区间


# var_info = ['order_number','下单日期','search_time','归属渠道','y_fraud_fpd7','y_fraud_f3pd7','y_credit_mob3pd30','y_credit']
var_info = ['order_number','下单日期','search_time','归属渠道','y_fraud_fpd7', 'y_fraud_mob3dpd7', 'y_credit_mob3dpd30', 'y_credit_mob6dpd30', 'y_credit']
var_1 = ['bj_hm_score','my_zl_score','my_zl_score2','br_ys_score','br_ra_score','model_score_2311','model_score_2401','model_score_yl','model_score_2501'
         ,'bj_qy_score','jx_fico_score','td212_score','nsf_rental_v1_3_hundred']
var_2 = list(jx_yl_df.columns)
var_2.remove('trace_id')
var_3 = list(xy_risk_qjld_df.columns)
var_3.remove('trace_id')


# df_analy_ks = df_analy_ks[var_info + var_1 + var_2 + var_3]
var_3.remove('xy_qj_A22160007')
var_3.remove('xy_qj_B22170054')
var_3.remove('xy_qj_B22170007')
var_3.remove('xy_qj_B22170008')
var_3.remove('xy_qj_B22170009')
var_3.remove('xy_qj_B22170010')
var_3.remove('xy_qj_B22170011')
var_3.remove('xy_qj_B22170031')
var_3.remove('xy_qj_B22170032')
var_3.remove('xy_qj_B22170033')
var_3.remove('xy_qj_B22170040')
var_3.remove('xy_qj_B22170041')
var_3.remove('xy_qj_B22170042')
var_3.remove('xy_qj_B22170043')
var_3.remove('xy_qj_B22170044')
var_3.remove('xy_qj_B22170050')

var_all = var_1 + var_2 + var_3
len(var_all)

for var in var_all:
    # print (var)
    df_analy_ks[var] = df_analy_ks[var].astype(float)
    df_analy_ks[var] = np.where(df_analy_ks[var] < 0, -999, df_analy_ks[var])
    df_analy_ks[var] = df_analy_ks[var].fillna(-999)
    
df_analy_ks.shape
df_analy_ks.tail(1)

(40324, 196)

103

(40324, 196)

,order_number,下单日期,search_time,归属渠道,是否免人审,审核状态,拒绝理由,是否前置拦截,是否机审强拒,是否出库前风控强拒,是否拒量,是否联合运营,联合运营命中风控,user_mobile,id_card_num,jd_user_level_risk,order_id,order_trace_id,trace_id,bj_hm_score,rentOnlineRiskLevelOverall,rentOnlineRiskLevelMultiRent,rentOnlineRiskLevelFulfill,rentOnlineDetailLevelOverall,my_zl_score,my_zl_score2,br_ys_score,br_ra_score,model_score_2311,model_score_2401,model_score_yl,model_score_2501,is_model,bj_qy_score,grade,xy_qj_A22160001,xy_qj_A22160002,xy_qj_A22160003,xy_qj_A22160004,xy_qj_A22160005,xy_qj_A22160006,xy_qj_A22160007,xy_qj_A22160008,xy_qj_A22160009,xy_qj_A22160010,xy_qj_C22180001,xy_qj_C22180002,xy_qj_C22180003,xy_qj_C22180004,xy_qj_C22180005,xy_qj_C22180006,xy_qj_C22180007,xy_qj_C22180008,xy_qj_C22180009,xy_qj_C22180010,xy_qj_C22180011,xy_qj_C22180012,xy_qj_B22170001,xy_qj_B22170002,xy_qj_B22170003,xy_qj_B22170004,xy_qj_B22170005,xy_qj_B22170006,xy_qj_B22170007,xy_qj_B22170008,xy_qj_B22170009,xy_qj_B22170010,xy_qj_B22170011,xy_qj_B22170012,xy_qj_B22170013,xy_qj_B22170014,xy_qj_B22170015,xy_qj_B22170016,xy_qj_B22170017,xy_qj_B22170018,xy_qj_B22170019,xy_qj_B22170020,xy_qj_B22170021,xy_qj_B22170022,xy_qj_B22170023,xy_qj_B22170024,xy_qj_B22170025,xy_qj_B22170026,xy_qj_B22170027,xy_qj_B22170028,xy_qj_B22170029,xy_qj_B22170030,xy_qj_B22170031,xy_qj_B22170032,xy_qj_B22170033,xy_qj_B22170034,xy_qj_B22170035,xy_qj_B22170036,xy_qj_B22170037,xy_qj_B22170038,xy_qj_B22170039,xy_qj_B22170040,xy_qj_B22170041,xy_qj_B22170042,xy_qj_B22170043,xy_qj_B22170044,xy_qj_B22170045,xy_qj_B22170046,xy_qj_B22170047,xy_qj_B22170048,xy_qj_B22170049,xy_qj_B22170050,xy_qj_B22170051,xy_qj_B22170052,xy_qj_B22170053,xy_qj_B22170054,score_201048,score_202098,score_202100,jx_yl_UPPC011,jx_yl_UPPC036,jx_yl_UPPC098,jx_yl_UPPC134,jx_yl_UPPC173,jx_yl_UPPC196,jx_yl_UPPD002,jx_yl_UPPD004,jx_yl_UPPD005,jx_yl_UPPD007,jx_yl_UPPD009,jx_yl_UPPD028,jx_yl_UPPD029,jx_yl_UPPD068,jx_yl_UPPD069,jx_yl_UPPD075,jx_yl_UPPD091,jx_yl_UPPD097,jx_yl_UPPD098,jx_yl_UPPD100,jx_yl_UPPE061,jx_yl_UPPE074,jx_yl_UPPE077,jx_yl_UPPE079,jx_yl_UPPE082,jx_yl_UPPE086,jx_yl_UPPE090,jx_yl_UPPG046,jx_yl_UPPG050,jx_yl_UPPG054,zm_jh_t0_level,jx_fico_score,td212_score,nsf_rental_v1_3_hundred,y_fraud_fpd7,y_fraud_mob3dpd7,y_credit_mob3dpd30,y_credit_mob6dpd30,y_credit,火眸分_Bin,rentOnlineRiskLevelFulfill_Bin,rentOnlineRiskLevelOverall_Bin,rentOnlineDetailLevelOverall_Bin,蚁盾分_Bin,融安分_Bin,my_zl_score2_Bin,br_ys_score_Bin,青云分_Bin,grade_Bin,strategy_240703_4_Bin,strategy_240703_10_Bin,strategy_240703_14_Bin,score_201048_Bin,score_202098_Bin,score_202100_Bin,jx_yl_UPPC134_Bin,jx_yl_UPPD007_Bin,jx_yl_UPPE074_Bin,jx_yl_UPPE082_Bin,xy_qj_C22180008_Bin,xy_qj_B22170035_Bin,xy_qj_B22170036_Bin,xy_qj_A22160001_Bin,strategy_240801_1_Bin,strategy_240801_2_Bin,strategy_240801_3_Bin,strategy_240801_4_Bin,strategy_240801_5_Bin,strategy_240801_7_Bin,strategy_240801_8_Bin,strategy_240801_9_Bin,strategy_240801_10_Bin,strategy_240801_11_Bin,规则42,履约保护_Bin,td212_Bin,zm_jh_t0_level_Bin,FICO_Bin,model_score_2311_Bin,model_score_yl_Bin,model_score_2401_Bin,model_score_2501_Bin
290620,A20250605095151278,2025-06-05,2025-06,芝麻租物,1,出库,[,0,0,0,0,None,None,13631814742,440782199211135649,None,2817425.0,f99ac681efe3f1bb5ec41e96ae1bc525,e2647786b452e5b21c0ba9afe3e99027,501.0,0,0,0,1,30.3195,51.8901,538.0,709.0,13.739502,11.693036,13.764214,11.654716,1,510.0,D,607.0,72.0,23.0,12.0,3.0,44.0,2025-05,2.0,12.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,4200.0,2600.0,2600.0,85.0,583.0,0.0,1.0,1.0,2.0,2.0,0,"[1000,2000)","[1000,2000)","[3000,5000)","[3000,5000)",0.0,1.0,1.0,0.0,0.0,1.0,1.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.5,0.0,1.0,1.0,1.0,1.0,0,0,0,"[500,1000)","[500,1000)",0.0,0.0,0.0,1.0,1.0,"(180,360]",85.0,1.0,212.0,2025-04,579.0,584.0,544.0,97.0,146.0,97.0,31.0,48.0,46.0,3900.0,10300.0,7.0,8.0,4.0,7300.0,7.0,4.0,5.0,3.0,7.0,7300.0,3.0,4.0,2.0,1.0,0.62,0.5,0.6,1.0,1.0,3000.0,1.0,1.0,979,670.0,31.52,9.0,0.0,0.0,0.0,0.0,0.0,"4.(499,520.0]",2.0,4.0,6.1,"7.(30,40]","3.(700,713.0]","9.(50,53]","5.(514,540

In [126]:
# （8-1-2）其他
# （1）S量策略开发的宽表
# var_5 = list(df_out_data.columns)
# var_5.remove('order_number')
# var_5.remove('order_id')
# var_5.remove('order_trace_id')
# var_5.remove('trace_id')
# var_5.remove('risk_trace_id')

# 租后表现
# var_6 = ['order_number','order_id','下单日期','search_time','归属渠道','y_fraud_fpd7','y_fraud_f3pd7','y_credit_mob3pd30','y_credit']
# df_analy_ks[var_6 + var_5].to_excel('data_s_20250417.xlsx', index = False)

# # 申请
# tmp1 = df_risk_new[['order_number','下单日期','归属渠道','id_card_num']].merge(df_out_data[['order_number','bj_hm_score']], on = 'order_number', how = 'left')
# conditions_1 = ['租机宝','八派信息','CPS直播','派金花','富士康租机','九州信息','勉丫租','哈银','租瓜直播','美仑美奂','兔丫租赁','分期乐1','分期乐2','98租超','邦客惠乘车码','推一推','硬派抖音']
# 租机宝、富士康租机、兔丫租赁、邦客惠乘车码  -- 不算S量
# conditions_2 = ((tmp1.下单日期 >= '2025-04-01')&(tmp1.下单日期 <= '2025-04-15'))
# conditions_3 = ((tmp1.bj_hm_score >= 0)&(tmp1.bj_hm_score <= 499))
# # 申请数
# # (5652, 5)
# tmp1[tmp1.归属渠道.isin(conditions_1) & conditions_2].shape
# (1036, 5)
# tmp1[tmp1.归属渠道.isin(conditions_1) & conditions_2 & conditions_3].shape
# 申请-去重订单数-身份证+申请日期维度去重
# (5435, 5)
# tmp1[tmp1.归属渠道.isin(conditions_1) & conditions_2].drop_duplicates(subset=['id_card_num','下单日期']).shape
# tmp2 = tmp1[tmp1.归属渠道.isin(conditions_1) & conditions_2].drop_duplicates(subset=['id_card_num','下单日期'])
# tmp2.groupby(['下单日期']).agg({'order_number': 'count'})
# (858, 5)
# tmp1[tmp1.归属渠道.isin(conditions_1) & conditions_2 & conditions_3].drop_duplicates(subset=['id_card_num','下单日期']).shape
# tmp3 = tmp1[tmp1.归属渠道.isin(conditions_1) & conditions_2 & conditions_3].drop_duplicates(subset=['id_card_num','下单日期'])
# tmp3.groupby(['下单日期']).agg({'order_number': 'count'})


# 进件-非免押
# tmp1 = df_jj[['order_number','拒绝理由','归属渠道','下单日期','id_card_num','审核状态']]
# tmp1['tmp'] = np.where(tmp1['拒绝理由'].str.contains('非免押'), 1, 0)

# # 890
# tmp1[tmp1.归属渠道.isin(conditions_1) & conditions_2].drop_duplicates(subset=['id_card_num','下单日期']).shape
# # 49
# tmp1[tmp1.归属渠道.isin(conditions_1) & conditions_2 & (tmp1.tmp == 1)].drop_duplicates(subset=['id_card_num','下单日期']).shape
# tmp1[(tmp1.tmp == 1)&(tmp1.下单日期>='2025-04-01')].拒绝理由.value_counts()
# 审核状态为出库的为小蚂蚁出库 A202504161000356
# tmp1[(tmp1.tmp == 1)&(tmp1.下单日期>='2025-04-01')].审核状态.value_counts()
# [命中非免押用户拒绝]
# 命中非免押用户拒绝

# # 进件
# conditions_1 = ['租机宝','八派信息','CPS直播','派金花','富士康租机','九州信息','勉丫租','哈银','租瓜直播','美仑美奂','兔丫租赁','分期乐1','分期乐2','98租超','邦客惠乘车码','推一推','硬派抖音']
# conditions_2 = ((df_info_ori.下单日期 >= '2025-04-01')&(df_info_ori.下单日期 <= '2025-04-25'))
# df_info_ori[df_info_ori.归属渠道.isin(conditions_1) & conditions_2].shape
# df_info_ori[df_info_ori.归属渠道.isin(conditions_1) & conditions_2].drop_duplicates(subset=['id_card_num','下单日期']).shape
# df_info_ori[df_info_ori.归属渠道.isin(conditions_1) & conditions_2].to_excel('data_s_jj_20250426.xlsx', index = False)


# # （2）取消订单分析
# # 2025.3进件-取消的样本
# path = 'C:/Users/Administrator/Desktop/mine/4_专题分析/11_一元发货/'
# df_qx_1 = pd.read_excel(path + '取消原因订单_20250403.xlsx')
# df_qx_1.shape
# df_qx_1 = df_qx_1.merge(df_info_ori, on = 'order_number', how = 'left')
# df_qx_1.shape
# # df_qx_1.to_excel('tmp.xlsx', index = False)
# # 细化综合风险等级 rentOnlineDetailLevelOverall
# # 蚁盾分2.0 my_zl_score2_Bin
# # 评分等级 grade
# # 自有模型 model_score_2311_Bin model_score_yl_Bin model_score_2401_Bin
# # df_qx_1.groupby(['rentOnlineDetailLevelOverall_Bin']).agg({'order_number':'count'})
# # df_qx_1.groupby(['my_zl_score2_Bin']).agg({'order_number':'count'})
# # df_qx_1.groupby(['grade_Bin']).agg({'order_number':'count'})
# df_qx_1[(df_qx_1.归属渠道 == '芝麻租物')].shape
# df_qx_1[(df_qx_1.归属渠道 == '芝麻租物')&(df_qx_1.rentOnlineDetailLevelOverall == '0')|(df_qx_1.grade == 'S')].shape


# # 历史的一元发货的订单-2023.3-2023.7
# # df_qx_2 = pd.read_excel(path + '1元发货订单.xlsx')
# # df_qx_2 = df_qx_2[['订单号','型号']]
# # df_qx_2 = df_qx_2.rename(columns = {'订单号':'order_number'})
# # df_qx_2.shape
# # df_qx_2['是否二手'] = np.where(df_qx_2['型号'].str.contains('95新')|df_qx_2['型号'].str.contains('99新')|df_qx_2['型号'].str.contains('准新'),"二手","全新")
# # df_qx_2['是否带锁']  = np.where(df_qx_2['型号'].str.contains(r'带锁|有锁'), '带锁', '未带锁')
# # df_qx_2['是否二手'].value_counts()
# # df_qx_2['是否带锁'].value_counts()

# # df_qx_2 = df_qx_2.merge(df_analy_data[['order_number','search_time','归属渠道','y_fraud_fpd7','y_fraud_f3pd7','y_credit_mob3pd30','y_credit','历史最大逾期天数']], on = 'order_number', how = 'left')
# # df_qx_2['归属渠道_new'] = np.where((~df_qx_2.归属渠道.isin(['芝麻租物','搜索渠道'])), '其他', df_qx_2.归属渠道)
# # df_qx_2['归属渠道_new'].value_counts()
# # df_qx_2.shape
# # df_qx_2[:2]

# # pd.pivot_table(df_qx_2[df_qx_2.归属渠道_new == '芝麻租物'],index='是否带锁',columns='y_fraud_fpd7',values='order_number',aggfunc='count')
# # pd.pivot_table(df_qx_2[df_qx_2.归属渠道_new == '芝麻租物'],index='是否带锁',columns='y_fraud_f3pd7',values='order_number',aggfunc='count')
# # pd.pivot_table(df_qx_2[df_qx_2.归属渠道_new == '芝麻租物'],index='是否带锁',columns='y_credit_mob3pd30',values='order_number',aggfunc='count')
# # pd.pivot_table(df_qx_2[df_qx_2.归属渠道_new == '芝麻租物'],index='是否带锁',columns='y_credit',values='order_number',aggfunc='count')

# （3）新旧渠道对比
# df_analy_ks.shape
# df_analy_ks[['order_number','归属渠道']].to_excel('tmp.xlsx', index = False)

# （4）抖音
# temp = df_info_ori[(df_info_ori.归属渠道 == '抖音渠道') & (df_info_ori.下单日期 >= '2025-03-01')]
# pd.pivot_table(temp,index='下单日期',columns='grade',values='order_number',aggfunc='count')
# pd.pivot_table(temp[temp.审核状态 == '出库'],index='下单日期',columns='grade',values='order_number',aggfunc='count')

# df_analy_ks[df_analy_ks.归属渠道 == '抖音渠道'][['order_number','search_time','y_fraud_fpd7','y_fraud_f3pd7','y_credit_mob3pd30','y_credit']].to_excel('抖音_20250506.xlsx', index = False)

# （5）免押用户归属渠道
# df_info_ori[(~df_info_ori.拒绝理由.isin(['[命中非免押用户拒绝]','命中非免押用户拒绝']))&(df_info_ori.下单日期 >= '2023-01-01')][['order_number','归属渠道','下单日期']].to_excel('免押用户归属渠道_20250423.xlsx', index = False)

# 联合运营租后表现
# df_analy_ks[df_analy_ks.是否联合运营 == '1'][['order_number','下单日期','y_fraud_fpd7','y_credit']]
# df_analy_ks[(df_analy_ks.是否联合运营 == '1')&(df_analy_ks.下单日期 >= '2025-03-25')&(df_analy_ks.下单日期 <= '2025-04-02')][['order_number','下单日期','y_fraud_fpd7','y_credit']]

# # 联合运营进件客群-R0、R1
# df_info_ori.是否联合运营.value_counts()
# df_info_ori[df_info_ori.是否联合运营 == '1'].rentOnlineDetailLevelOverall.value_counts()
# df_info_ori[(df_info_ori.是否联合运营 == '1')&(df_info_ori.rentOnlineDetailLevelOverall.isin(['1','2','3','4','5','6']))][['order_number','下单日期','rentOnlineDetailLevelOverall']]


# （6）周报-统计cpd、hpd时用最新的渠道，月更新即可
# df_analy_ks[['order_number','归属渠道','是否拒量']].to_excel('df_cpd_hpd_20250427.xlsx', index = False)

# （7）拒量-联合项目
# 进件-非免押
# conditions_1 = ((df_jj.归属渠道.isin(['芝麻租物','搜索渠道'])) & (df_jj.下单日期 >= '2024-01-01') & (df_jj.下单日期 <= '2025-02-28'))
# tmp1 = df_jj[conditions_1][['order_number','id_card_num','user_mobile','下单日期','拒绝理由','审核状态','是否拒量']]
# tmp1['tmp'] = np.where(tmp1['拒绝理由'].str.contains('非免押'), 1, 0)
# tmp1 = tmp1[tmp1.tmp == 0]
# tmp1 = tmp1.drop(columns = ['tmp','拒绝理由'])
# tmp1.shape
# tmp1[:1]

# tmp1.to_excel('df_jl_no_label.xlsx', index = False)

# # （8）拒量调优宽表
# tmp = df_risk_new[df_risk_new['是否拒量'] == 1][['下单日期','order_number','tips','审核状态']]
# tmp['是否出库'] = np.where(tmp.审核状态 == '出库', 1, 0)
# tmp['策略命中等级'] = tmp['tips'].str.extract(r'(策略241205命中\(\d+\)?|策略241212命中?|命中自有模型回捞策略|回捞策略250330命中?)')[0]
# tmp = tmp[['order_number','策略命中等级','是否出库']]
# tmp.shape

# df_analy_jl = tmp.merge(df_info_ori, on = 'order_number', how = 'left')
# df_analy_jl.shape

# var_info = ['order_number','下单日期','search_time','归属渠道','策略命中等级','是否出库','y_fraud_fpd7','y_fraud_f3pd7','y_credit_mob3pd30','y_credit']
# # 产品-数值
# var_1_1 = ['bj_hm_score','rentOnlineDetailLevelOverall','rentOnlineRiskLevelOverall','rentOnlineRiskLevelMultiRent','rentOnlineRiskLevelFulfill'
#          ,'my_zl_score','my_zl_score2','br_ys_score','br_ra_score','model_score_2311','model_score_2401','model_score_yl','model_score_2501'
#          ,'bj_qy_score','score_201048','score_202098','score_202100','jx_fico_score','td212_score','nsf_rental_v1_3_hundred']
# # 产品-字符
# var_1_2 = ['grade','zm_jh_t0_level']
# var_2 = list(jx_yl_df.columns)
# var_2.remove('trace_id')
# var_3 = list(xy_risk_qjld_df.columns)
# var_3.remove('trace_id')


# # 字符型-16
# # xy_qj_A22160007 ： 最近⼀次查询时间
# # xy_qj_B22170054 ： 最近⼀次贷款放款时间
# # xy_qj_B22170007	近1个⽉贷款总⾦额
# # xy_qj_B22170008	近3个⽉贷款总⾦额
# # xy_qj_B22170009	近6个⽉贷款总⾦额
# # xy_qj_B22170010	近12个⽉贷款总⾦额
# # xy_qj_B22170011	近24个⽉贷款总⾦额
# # xy_qj_B22170031	近6个⽉累计逾期⾦额参考⾦额区间
# # xy_qj_B22170032	近12个⽉累计逾期⾦额参考⾦额区间
# # xy_qj_B22170033	近24个⽉累计逾期⾦额参考⾦额区间
# # xy_qj_B22170040	近1个⽉履约贷款总⾦额参考⾦额区间	object
# # xy_qj_B22170041	近3个⽉履约贷款总⾦额参考⾦额区间	object
# # xy_qj_B22170042	近6个⽉履约贷款总⾦额参考⾦额区间	object
# # xy_qj_B22170043	近12个⽉履约贷款总⾦额参考⾦额区间	object
# # xy_qj_B22170044	近24个⽉履约贷款总⾦额参考⾦额区间	object
# # xy_qj_B22170050	最近⼀次履约距今天数参考天数区间

# var_3.remove('xy_qj_A22160007')
# var_3.remove('xy_qj_B22170054')
# var_3.remove('xy_qj_B22170007')
# var_3.remove('xy_qj_B22170008')
# var_3.remove('xy_qj_B22170009')
# var_3.remove('xy_qj_B22170010')
# var_3.remove('xy_qj_B22170011')
# var_3.remove('xy_qj_B22170031')
# var_3.remove('xy_qj_B22170032')
# var_3.remove('xy_qj_B22170033')
# var_3.remove('xy_qj_B22170040')
# var_3.remove('xy_qj_B22170041')
# var_3.remove('xy_qj_B22170042')
# var_3.remove('xy_qj_B22170043')
# var_3.remove('xy_qj_B22170044')
# var_3.remove('xy_qj_B22170050')

# # 变量-字符
# var_3_2 = ['xy_qj_A22160007','xy_qj_B22170054','xy_qj_B22170007','xy_qj_B22170008','xy_qj_B22170009','xy_qj_B22170010','xy_qj_B22170011','xy_qj_B22170031','xy_qj_B22170032'
#            ,'xy_qj_B22170033','xy_qj_B22170040','xy_qj_B22170041','xy_qj_B22170042','xy_qj_B22170043','xy_qj_B22170044','xy_qj_B22170050']

# var_all = var_1_1 + var_2 + var_3
# len(var_all)

# for var in var_all:
#     df_analy_jl[var] = np.where(df_analy_jl[var] == 'DEFAULT', -999, df_analy_jl[var])
#     df_analy_jl[var] = df_analy_jl[var].astype(float)
#     df_analy_jl[var] = np.where(df_analy_jl[var] < 0, -999, df_analy_jl[var])
#     df_analy_jl[var] = df_analy_jl[var].fillna(-999)
    
# df_analy_jl.shape
# df_analy_jl = df_analy_jl[var_info + var_all + var_3_2 + var_1_2]
# df_analy_jl.shape
# df_analy_jl.tail(1)

# # df_analy_jl.to_excel('拒量数据_20250507.xlsx', index = False)

# # （9）京享租宽表
# # bj_hm_score 冰鉴-火眸分
# # my_zl_score2 蚁盾分2.0
# # bj_qy_score 冰鉴-青云分
# # grade 评分等级
# # br_ra_score 百融-融安分
# # score_201048 融360
# # score_202098 融360
# # score_202100 融360
# # td212_score 极信-td212
# # my_zl_score 蚁盾分1.0
# # jx_yl_UPPE082 近12个月成功扣款频度占比
# # jx_yl_UPPD029 近1个月资金不足交易频度
# # jx_fico_score  极信-fico
# # jx_yl_UPPC196 近1个月消费频度
# # xy_qj_A22160004 申请命中消⾦类机构数

# var_jxz = ['jd_user_level_risk','bj_hm_score','my_zl_score2','bj_qy_score','grade','br_ra_score','score_201048','score_202098','score_202100','td212_score','my_zl_score',
#            'jx_yl_UPPE082','jx_yl_UPPD029','jx_fico_score','jx_yl_UPPC196','xy_qj_A22160004']
# var_info = ['order_number','下单日期','归属渠道','审核状态','拒绝理由']
# conditions_1 = (df_info_ori.下单日期 == '2025-05-20')
# conditions_2 = (df_info_ori.归属渠道 == '京东渠道')
# # df_info_ori[conditions_1].归属渠道.value_counts()
# # df_info_ori[:1]
# df_jxz = df_info_ori[conditions_1 & conditions_2][var_info + var_jxz]
# df_jxz.to_excel('data_jxz.xlsx', index = False)


In [127]:
# （8-3）样本切分-机审
# 融安分&蚁盾分1.0
# 融安分2023.1.1就开始使用，蚁盾分1.0 2023.2开始使用
# br_ra_score、 my_zl_score
numeric_desc_var_21 = ['br_ra_score','my_zl_score']
df_train_21 = df_analy_ks[(df_analy_ks.下单日期 >= '2024-01-01') & (df_analy_ks.下单日期 <= '2024-06-30')]
df_test_21 = df_analy_ks[(df_analy_ks.下单日期 >= '2024-07-01')]
df_train_21['y'] = df_train_21['y_credit_mob3dpd30']
df_test_21['y'] = df_test_21['y_credit_mob3dpd30']
df_train_21.shape
df_test_21.shape

(6364, 197)

(30218, 197)

In [128]:
# （8-4-1）样本切分-出库前强拒
# 训练集和测试集
# 不同产品的训练集和测试集可能不一样
# 3表示出库前强拒
# jx_fico_score ：极信-2024.2

numeric_desc_var_31 = ['jx_fico_score']
df_train_31 = df_analy_ks[(df_analy_ks.下单日期 >= '2024-01-01') & (df_analy_ks.下单日期 <= '2024-06-30')]
df_test_31 = df_analy_ks[(df_analy_ks.下单日期 >= '2024-07-01')]
df_train_31['y'] = df_train_31['y_credit_mob3dpd30']
df_test_31['y'] = df_test_31['y_credit_mob3dpd30']
df_train_31.shape
df_test_31.shape

(6364, 197)

(30218, 197)

In [129]:
# （8-4-2）样本切分-出库前强拒
# td212_score ：腾云-2024.9-反欺诈评分-对坏客户定义是逾期超过30天,2024.9.8开始调用
# nsf_rental_v1_3_hundred ：履约保护-2024.8-mob3&dpd30+,2024.8.14开始调用

numeric_desc_var_32 = ['td212_score','nsf_rental_v1_3_hundred']
df_train_32 = df_analy_ks[(df_analy_ks.下单日期 >= '2024-09-01') & (df_analy_ks.下单日期 <= '2024-10-31')]
df_test_32 = df_analy_ks[(df_analy_ks.下单日期 >= '2024-11-01')]
df_train_32['y'] = df_train_32['y_credit_mob3dpd30']
df_test_32['y'] = df_test_32['y_credit_mob3dpd30']
df_train_32.shape
df_test_32.shape

(4202, 197)

(21716, 197)

In [130]:
# # （8-5）分箱
# %run F:/Project_Business/风控/2.model_func.ipynb
# continous_merged_dict = {}
# var_bin_list = []

# # trainData_kf_Bin = df_train_31.copy()
# # numeric_desc_var = numeric_desc_var_31

# # trainData_kf_Bin = df_train_32.copy()
# # numeric_desc_var = numeric_desc_var_32

# # trainData_kf_Bin = df_train_21.copy()
# # numeric_desc_var = numeric_desc_var_21

# trainData_kf_Bin = df_train_11.copy()
# numeric_desc_var = numeric_desc_var_11

# for col in numeric_desc_var:
#     print ("{} is in processing".format(col))
#     if -1111 not in set(trainData_kf_Bin[col]) and -999 not in set(trainData_kf_Bin[col]): 
#         max_interval = 5 
#         cutOff = ChiMerge(trainData_kf_Bin, col, 'y', max_interval=max_interval,special_attribute=[],minBinPcnt=0)
#         trainData_kf_Bin[col+'_Bin'] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff,special_attribute=[]))
#         monotone = BadRateMonotone(trainData_kf_Bin, col+'_Bin', 'y')
#         while(not monotone):
#             max_interval -= 1
#             cutOff = ChiMerge(trainData_kf_Bin, col, 'y', max_interval=max_interval, special_attribute = [],
#                                           minBinPcnt = 0)
#             trainData_kf_Bin[col + '_Bin'] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff, special_attribute = []))
#             if max_interval == 2:
#                 break
#             monotone = BadRateMonotone(trainData_kf_Bin, col + '_Bin', 'y')
#         newVar = col + '_Bin'
#         trainData_kf_Bin[newVar] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff, special_attribute = [])) 
#         del trainData_kf_Bin[col]
#         var_bin_list.append(newVar)
#     else:
#         max_interval = 7 #正常取值最多分5箱
#         cutOff = ChiMerge(trainData_kf_Bin, col, 'y', max_interval = max_interval, special_attribute = [-1111, -999],
#                                       minBinPcnt = 0)
#         trainData_kf_Bin[col + '_Bin'] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff, special_attribute=[-1111, -999]))
#         monotone = BadRateMonotone(trainData_kf_Bin, col + '_Bin', 'y',['Bin -1', 'Bin -2'])
#         while (not monotone):
#             max_interval -= 1
#             cutOff = ChiMerge(trainData_kf_Bin, col, 'y', max_interval=max_interval, special_attribute=[-1111, -999],
#                                           minBinPcnt = 0)
#             trainData_kf_Bin[col + '_Bin'] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff, special_attribute = [-1111, -999]))
#             if max_interval == 3:
#                 break
# #             if max_interval == 3 and -1111 in set(trainData_kf_Bin[col]) and -999 not in set(trainData_kf_Bin[col]):
# #                 break
# #             if max_interval == 3 and -1111 not in set(trainData_kf_Bin[col]) and -999 in set(trainData_kf_Bin[col]):
# #                 break
# #             if max_interval == 4 and -1111 in set(trainData_kf_Bin[col]) and -999 in set(trainData_kf_Bin[col]):
# #                 break
#             monotone = BadRateMonotone(trainData_kf_Bin, col + '_Bin', 'y',['Bin -1', 'Bin -2']) 
#         newVar = col + '_Bin'
#         trainData_kf_Bin[newVar] = trainData_kf_Bin[col].map(lambda x: AssignBin(x, cutOff, special_attribute = [-1111, -999]))  
#         del trainData_kf_Bin[col]
#         var_bin_list.append(newVar)
#     continous_merged_dict[col] = cutOff

In [131]:
# # （8-6）改变映射方式
# # trainData_kf_Bin_1 = df_train_31.copy()
# # trainData_kf_Bin_1 = df_train_32.copy()
# # trainData_kf_Bin_1 = df_train_21.copy()
# trainData_kf_Bin_1 = df_train_11.copy()

# # start_time = time.time()
# for col in numeric_desc_var:
#     if -1111 not in set(trainData_kf_Bin_1[col]) and -999 not in set(trainData_kf_Bin_1[col]):
#         trainData_kf_Bin_1[col+'_Bin'] = trainData_kf_Bin_1[col].map(lambda x: AssignBin_me(x, continous_merged_dict[col]))
#         # del trainData_kf_Bin_1[col]
#     else:
#         trainData_kf_Bin_1[col + '_Bin'] = trainData_kf_Bin_1[col].map(lambda x: AssignBin_me(x, continous_merged_dict[col],[-1111, -999]))
#         # del trainData_kf_Bin_1[col]
        

In [132]:
# # （8-7）映射到test
# # test = df_test_31.copy()
# # test = df_test_32.copy()
# test = df_test_21.copy()
# short_list_ori = numeric_desc_var

# for var in short_list_ori:
#     if -1111 not in set(test[var]) and -999 not in set(test[var]):
#         test[var + '_Bin'] = test[var].map(lambda x: AssignBin_me(x, continous_merged_dict[var]))
#         # del test[var]
#     else:
#         test[var + '_Bin'] = test[var].map(lambda x: AssignBin_me(x, continous_merged_dict[var],[-1111, -999]))
#         # del test[var]

# test.shape

# 10 自动写入

## 10_11.数据整理

In [256]:
# 监控
# 对pd.pivot_table的数据进行拓展补充
def lift_ks(df, x, y, key=None):
    if 1.0 not in df.columns.to_list():
        df.loc[:, 1] = np.nan
    df = df.fillna(0).rename(columns={0.0: x, 1.0: y})
    df.loc['合计', :] = df.sum(axis=0)
    df.loc[:, '合计'] = df.sum(axis=1)
    df.loc[:, '组内占比'] = (df.合计/df.loc['合计', '合计']).map(lambda x: format(x, '.2%'))
    df.loc[:, '占比'] = df[y]/df['合计']
    df.loc[:, 'lift'] = round(df.占比/df.loc['合计', '占比'], 2)
    df.loc[:, '占比'] = df.占比.map(lambda x: format(x, '.2%'))
    df.loc[:, 'KS'] = round(df[y].cumsum()/df.loc['合计',y]-df[x].cumsum()/df.loc['合计',x], 2)
    df.loc['合计','KS'] = df.KS.max()
    if key is None:
        df.loc[:, 'nan'] = np.nan
    return df.reset_index()

def add_data(df, df_analy_ks, key):
    # 对行数不足的数据进行补充
    key_list = df_analy_ks[f'{key}'].drop_duplicates().to_list()
    for kl in key_list:
        if kl not in df.index.to_list():
            df.loc[kl, :] = np.nan
    df = df.sort_values(by=f'{key}')
    return df


# 对不同月份的数据和不同的mob数据进行拼接
def pivot_table_self(df, conditions_1, key, m_list):
    df_concat = pd.DataFrame()
    conditions_2 = ''
    for idx, m in enumerate(m_list):
        # 判断月份
        if m=='2023.10-2023.12':
            conditions_2 = ((df.下单日期 >= '2023-10-01') & (df.下单日期 <= '2023-12-31'))
        elif m=='2024.1-2024.6':
            conditions_2 = ((df.下单日期 >= '2024-01-01') & (df.下单日期 <= '2024-06-30'))
        elif m=='2024.2-2024.6':
            conditions_2 = ((df.下单日期 >= '2024-02-01') & (df.下单日期 <= '2024-06-30'))
        elif m=='2024.4-2024.6':
            conditions_2 = ((df.下单日期 >= '2024-04-01') & (df.下单日期 <= '2024-06-30'))
        else:
            conditions_2 = (df.search_time == m)
        # 不同mob或fpd的数据透视表
        df_fpd7 = add_data(pd.pivot_table(df[conditions_1 & conditions_2],index=key,columns='y_fraud_fpd7',values='order_number',aggfunc='count'), df, key)
        df_mob3fpd7 = add_data(pd.pivot_table(df[conditions_1 & conditions_2],index=key,columns='y_fraud_mob3dpd7',values='order_number',aggfunc='count'), df, key)
        df_mob3dpd30 = add_data(pd.pivot_table(df[conditions_1 & conditions_2],index=key,columns='y_credit_mob3dpd30',values='order_number',aggfunc='count'), df, key)
        df_mob6dpd30 = add_data(pd.pivot_table(df[conditions_1 & conditions_2],index=key,columns='y_credit_mob6dpd30',values='order_number',aggfunc='count'), df, key)
        df_credit = add_data(pd.pivot_table(df[conditions_1 & conditions_2],index=key,columns='y_credit',values='order_number',aggfunc='count'), df, key)
        # 获取每个时间内的数据
        y_fraud_fpd7 = lift_ks(df_fpd7, 'fpd<=7', 'fpd7+')
        y_fraud_mob3dpd7 = lift_ks(df_mob3fpd7, 'mob3dpd<=7','mob3dpd7+')
        y_credit_mob3dpd30 = lift_ks(df_mob3dpd30, 'mob3dpd<=30','mob3dpd30+')
        y_credit_mob6dpd30 = lift_ks(df_mob6dpd30, 'mob6dpd<=30','mob6dpd30+')
        y_credit = lift_ks(df_credit, 'dpd<=30', 'dpd30+', key=1)
        df_concat_new = pd.concat([y_fraud_fpd7, y_fraud_mob3dpd7, y_credit_mob3dpd30, y_credit_mob6dpd30, y_credit], axis=1)
        # 从第二个开始在最后一行添加表头
        if idx>0:
            last_row = {
            key: m.replace('-', '.'), 'fpd<=7':'fpd<=7', 'fpd7+':'fpd7+', '合计':'合计', '组内占比':'组内占比', '占比':'占比', 'lift':'lift', 'KS':'KS',
            key:m.replace('-', '.'),'mob3dpd<=7':'mob3dpd<=7','mob3dpd7+':'mob3dpd7+','合计':'合计', '组内占比':'组内占比', '占比':'占比', 'lift':'lift', 'KS':'KS',
            key:m.replace('-', '.'),'mob3dpd<=30':'mob3dpd<=30','mob3dpd30+':'mob3dpd30+','合计':'合计', '组内占比':'组内占比', '占比':'占比', 'lift':'lift', 'KS':'KS',
            key:m.replace('-', '.'),'mob6dpd<=30':'mob6dpd<=30','mob6dpd30+':'mob6dpd30+','合计':'合计', '组内占比':'组内占比', '占比':'占比', 'lift':'lift', 'KS':'KS',
            key:m.replace('-', '.'),'dpd<=30':'dpd<=30','dpd30+':'dpd30+','合计':'合计', '组内占比':'组内占比', '占比':'占比', 'lift':'lift', 'KS':'KS'
            }
            df_concat.loc[len(df_concat)] = last_row
        # 拼接每个月的数据并在最后一行添加空行
        df_concat = pd.concat([df_concat, df_concat_new], axis=0)
        df_concat.loc[len(df_concat)] = np.nan
    df_concat = df_concat.rename(columns={f'{key}': m_list[0]})
    return df_concat

In [342]:
# psi
def math_psi(df, model=None):
    '''
    df：传入的已经透视好的数据
    model：判断是否需要计算psi
    '''
    df.loc['合计', :] = df.sum(axis=0)
    # 获取表列名为列表
    col_list = df.columns.to_list()
    # 计算组内占比
    for col in col_list:
        df.loc[:, f'{col}_rate'] = df[f'{col}']/df.loc['合计', f'{col}']
    if model is None:
        # 计算以固定月为基底的psi
        for idx, col in enumerate(col_list):
            if idx>0:
                df.loc[:, f'psi_{col}_1'] = df.apply(lambda x: round((x[f'{col}_rate']-x[f'{col_list[0]}_rate'])*math.log((x[f'{col}_rate']/x[f'{col_list[0]}_rate'])), 4), axis=1)
        # 计算以上个月为基底的psi
        for idx, col in enumerate(col_list):
            if idx>1:
                df.loc[:, f'psi_{col}_2'] = df.apply(lambda x: round((x[f'{col}_rate']-x[f'{col_list[idx-1]}_rate'])*math.log((x[f'{col}_rate']/x[f'{col_list[idx-1]}_rate'])), 4), axis=1)
    df.loc['合计', :] = df.sum(axis=0)
    df.loc[np.nan, :] = np.nan
    return df

def psi_data(df, conditions, key, date, conditions1=None, model=None):
    '''
    df：数据底表
    conditions：筛选条件
    key：模型的分组字段
    date：需要筛选的日期
    conditions1：筛选条件，当conditions1为None时没有此筛选条件
    model：判断是否需要计算psi
    '''
    # 筛选固定时间内的数据
    df = df[(df.search_time>=date)&(df.search_time!=datetime.datetime.now().strftime('%Y-%m'))]
    # 判断进件是否使用条件
    # 计算conditions为条件的数据
    if conditions1 is None:
        df_jj = add_data(pd.pivot_table(df, index=key, columns='search_time', values='order_number',aggfunc='count'), df, key)
    else:
        df_jj = add_data(pd.pivot_table(df[conditions1], index=key, columns='search_time', values='order_number',aggfunc='count'), df, key)
    # 计算conditions1为条件的数据
    df_ck = add_data(pd.pivot_table(df[conditions], index=key, columns='search_time', values='order_number',aggfunc='count'), df, key)
    # 判断是否计算psi，model为None则计算psi，不为None则不计算psi
    if model is None:
        df_jj_psi = math_psi(df_jj).reset_index()
        df_ck_psi = math_psi(df_ck).reset_index()
        df_jj_psi.loc[:, 'nan'] = np.nan
        df_all = pd.concat([df_jj_psi, df_ck_psi], axis=1)
        return df_all
    else:
        df_jj_psi = math_psi(df_jj, model).reset_index()
        df_ck_psi = math_psi(df_ck, model).reset_index()
        return df_jj_psi, df_ck_psi
# 计算组内占比
def rate(df, df_tg, conditions1, conditions2, key, date):
    '''
    df：传入的底表数据
    df_tg：已经计算好的模型通过的数据
    conditions1：作为使用模型的进件筛选条件
    conditions2：作为使用模型的通过筛选条件
    key：模型的分组字段
    '''
    df = df[(df.search_time>=date)&(df.search_time!=datetime.datetime.now().strftime('%Y-%m'))]
    # 使用该模型的进件，并计算合计
    df1 = pd.pivot_table(df[conditions1], index=key, columns='search_time', values='order_number',aggfunc='count')
    df1.loc['合计', :] = df1.fillna(0).sum(axis=0)
    # 使用该模型的通过，并计算合计
    df2 = pd.pivot_table(df[conditions2], index=key, columns='search_time', values='order_number',aggfunc='count')
    df2.loc['合计', :] = df2.fillna(0).sum(axis=0)
    # 计算通过占比
    df_rate = df2/df1
    df_tg = pd.concat([df_tg.set_index(key), df_rate], axis=1)
    df_tg = df_tg.reset_index()
    return df_tg

## 10_12.写入数据

In [350]:
# def Open_Excel2(df, path,  sheet_name, start_row, start_col):
#     '''
#     对日报数据进行自动更新
#     :param df: 日报各个渠道的df
#     :param path: 日报的excel文件路径
#     :param sheet_name: excel的sheet页名称
#     :param start_row: 开始的行数
#     :param start_col: 开始的列数
#     :return:
#     '''
#     # 定义文件路径和密码
#     file_path = path
#     # 使用 xlwings 打开加密的 Excel 文件
#     app = xw.App(visible=False)
#     try:
#         wb = xw.Book(file_path)
#         # 获取工作表
#         sheet = wb.sheets[sheet_name]
        
#         # 获取上一个df的长度并相加
#         row_lens = start_row + df.shape[0]
#         # 写入 DataFrame 的数据
#         # for row_num, (_, row_data) in enumerate(df.iterrows(), start=start_row):
#         #     for new_row, value in enumerate(row_data, start=start_col):
#         #         sheet.cells(row_num, new_row).value = value
#         # 定位写入区域左上角单元格（行, 列）
#         target_range = sheet.range((start_row, start_col))
#         # 一次性写入整个DataFrame数据
#         target_range.options(index=False, header=False).value = df
        
#         # 保存并重新加密文件
#         wb.save()
#         return row_lens
#     finally:
#         # 确保关闭工作簿和应用程序实例
#         wb.close()
#         app.quit()

def Open_Excel2(df_list, path, start_row, start_col, key=None):
    '''
    对日报数据进行自动更新
    :param df: 日报各个渠道的df
    :param path: 日报的excel文件路径
    :param sheet_name: excel的sheet页名称
    :param start_row: 开始的行数
    :param start_col: 开始的列数
    :return:
    '''
    # 定义文件路径和密码
    file_path = path
    # 使用 xlwings 打开加密的 Excel 文件
    app = xw.App(visible=False)
    try:
        wb = xw.Book(file_path)
        # 获取excel文件的sheet页名称
        sheet_names = wb.sheet_names
        # 获取工作表
        # 赋值开始行初始值
        row_lens = start_row
        # 循环df大列表
        for idx1, df_lis in enumerate(df_list):
            # 获取对应sheet页
            if key=='风控监控':
                sheet = wb.sheets[sheet_names[idx1+1]]
            else:
                sheet = wb.sheets[sheet_names[idx1]]
            # 循环df小列表
            for df in df_lis:
                # 定位写入区域左上角单元格（行, 列）
                target_range = sheet.range((row_lens, start_col))
                # 一次性写入整个DataFrame数据
                target_range.options(index=False).value = df
                # 获取上一个df的长度并相加
                row_lens += df.shape[0]+2
            # 写入完成一个sheet页后开始行重新赋为初始值
            row_lens = start_row
        # 保存并重新加密文件
        wb.save()
        return row_lens
    finally:
        # 确保关闭工作簿和应用程序实例
        wb.close()
        app.quit()

## 10_13.数据整合

In [282]:
path = 'F:/租后表分析数据_ld/风控周报/1_风控监控_效果_auto - 副本.xlsx'
# 区分渠道
conditions_11 = ((df_analy_ks.归属渠道 == '芝麻租物') & (df_analy_ks.是否拒量 == 0))
conditions_12 = ((df_analy_ks.归属渠道 == '搜索渠道') & (df_analy_ks.是否拒量 == 0))
conditions_13 = ((~df_analy_ks.归属渠道.isin(['芝麻租物','搜索渠道','抖音渠道','广州易合顺','深圳刘全兵号卡1'])) & (df_analy_ks.是否拒量 == 0))

conditions_model1 = ((df_analy_ks.is_model == '1') & ((~df_analy_ks.归属渠道.isin(['抖音渠道','广州易合顺','深圳刘全兵号卡1'])) & (df_analy_ks.是否拒量 == 0)))
conditions_model2 = ((df_analy_ks.is_model == '2') & ((~df_analy_ks.归属渠道.isin(['抖音渠道','广州易合顺','深圳刘全兵号卡1'])) & (df_analy_ks.是否拒量 == 0)))
conditions_model3 = ((df_analy_ks.is_model == '3') & ((~df_analy_ks.归属渠道.isin(['抖音渠道','广州易合顺','深圳刘全兵号卡1'])) & (df_analy_ks.是否拒量 == 0)))
conditions_model4 = ((df_analy_ks.is_model == '4') & ((~df_analy_ks.归属渠道.isin(['抖音渠道','广州易合顺','深圳刘全兵号卡1'])) & (df_analy_ks.是否拒量 == 0)))

### 10_13_11 数据监控

In [324]:
# 获取所需数据
view_m_list = ['2023.10-2023.12', '2024.1-2024.6', '2024-07', '2024-08', '2024-09', '2024-10', '2024-11', '2024-12', '2025-01', '2025-02', '2025-03', '2025-04']
view_m_list_cl240703 = ['2024.2-2024.6', '2024-07', '2024-08', '2024-09', '2024-10', '2024-11', '2024-12', '2025-01', '2025-02', '2025-03', '2025-04']
view_m_list_model = ['2024-03', '2024-04', '2024-05', '2024-06', '2024-07', '2024-08', '2024-09', '2024-10', '2024-11', '2024-12', '2025-01', '2025-02', '2025-03', '2025-04']
view_m_list_model2 = ['2025-03', '2025-04'] # 
view_m_list_cl240801 = ['2024.4-2024.6', '2024-07', '2024-08', '2024-09', '2024-10', '2024-11', '2024-12', '2025-01', '2025-02', '2025-03', '2025-04']
view_m_list_rule = ['2024-07','2024-08', '2024-09', '2024-10', '2024-11', '2024-12', '2025-01', '2025-02', '2025-03', '2025-04']
view_m_list_ly = ['2024-08', '2024-09', '2024-10', '2024-11', '2024-12', '2025-01', '2025-02', '2025-03', '2025-04']
view_m_list_ydf2 = ['2024-09', '2024-10', '2024-11', '2024-12', '2025-01', '2025-02', '2025-03', '2025-04']
# 火眸分
df_hmf_zm = pivot_table_self(df_analy_ks, conditions_11, '火眸分_Bin', view_m_list)
df_hmf_ss = pivot_table_self(df_analy_ks, conditions_12, '火眸分_Bin', view_m_list)
df_hmf_qt = pivot_table_self(df_analy_ks, conditions_13, '火眸分_Bin', view_m_list)
# 履约历史等级
df_ly_zm = pivot_table_self(df_analy_ks, conditions_11, 'rentOnlineRiskLevelFulfill_Bin', view_m_list)
df_ly_ss = pivot_table_self(df_analy_ks, conditions_12, 'rentOnlineRiskLevelFulfill_Bin', view_m_list)
df_ly_qt = pivot_table_self(df_analy_ks, conditions_13, 'rentOnlineRiskLevelFulfill_Bin', view_m_list)
# 综合风险等级
df_zhfx_zm = pivot_table_self(df_analy_ks, conditions_11, 'rentOnlineRiskLevelOverall_Bin', view_m_list)
df_zhfx_ss = pivot_table_self(df_analy_ks, conditions_12, 'rentOnlineRiskLevelOverall_Bin', view_m_list)
df_zhfx_qt = pivot_table_self(df_analy_ks, conditions_13, 'rentOnlineRiskLevelOverall_Bin', view_m_list)
# 细分综合风险等级
df_xf_zm = pivot_table_self(df_analy_ks, conditions_11, 'rentOnlineDetailLevelOverall_Bin', view_m_list)
df_xf_ss = pivot_table_self(df_analy_ks, conditions_12, 'rentOnlineDetailLevelOverall_Bin', view_m_list)
df_xf_qt = pivot_table_self(df_analy_ks, conditions_13, 'rentOnlineDetailLevelOverall_Bin', view_m_list)
# 蚁盾分1.0
df_ydf1_zm = pivot_table_self(df_analy_ks, conditions_11, '蚁盾分_Bin', view_m_list)
df_ydf1_ss = pivot_table_self(df_analy_ks, conditions_12, '蚁盾分_Bin', view_m_list)
df_ydf1_qt = pivot_table_self(df_analy_ks, conditions_13, '蚁盾分_Bin', view_m_list)
# 蚁盾分2.0
df_ydf2_zm = pivot_table_self(df_analy_ks, conditions_11, 'my_zl_score2_Bin', view_m_list_ydf2)
df_ydf2_ss = pivot_table_self(df_analy_ks, conditions_12, 'my_zl_score2_Bin', view_m_list_ydf2)
df_ydf2_qt = pivot_table_self(df_analy_ks, conditions_13, 'my_zl_score2_Bin', view_m_list_ydf2)
# 云商分
df_ysf_zm = pivot_table_self(df_analy_ks, conditions_11, 'br_ys_score_Bin', view_m_list)
df_ysf_ss = pivot_table_self(df_analy_ks, conditions_12, 'br_ys_score_Bin', view_m_list)
df_ysf_qt = pivot_table_self(df_analy_ks, conditions_13, 'br_ys_score_Bin', view_m_list)
# 融安分
df_raf_zm = pivot_table_self(df_analy_ks, conditions_11, '融安分_Bin', view_m_list)
df_raf_ss = pivot_table_self(df_analy_ks, conditions_12, '融安分_Bin', view_m_list)
df_raf_qt = pivot_table_self(df_analy_ks, conditions_13, '融安分_Bin', view_m_list)
# 青云分
df_qyf_zm = pivot_table_self(df_analy_ks, conditions_11, '青云分_Bin', view_m_list)
df_qyf_ss = pivot_table_self(df_analy_ks, conditions_12, '青云分_Bin', view_m_list)
df_qyf_qt = pivot_table_self(df_analy_ks, conditions_13, '青云分_Bin', view_m_list)
# 评分等级
df_pf_zm = pivot_table_self(df_analy_ks, conditions_11, 'grade_Bin', view_m_list)
df_pf_ss = pivot_table_self(df_analy_ks, conditions_12, 'grade_Bin', view_m_list)
df_pf_qt = pivot_table_self(df_analy_ks, conditions_13, 'grade_Bin', view_m_list)
# 策略240703_04
df_cl4_zm = pivot_table_self(df_analy_ks, conditions_11, 'strategy_240703_4_Bin', view_m_list_cl240703)
df_cl4_ss = pivot_table_self(df_analy_ks, conditions_12, 'strategy_240703_4_Bin', view_m_list_cl240703)
df_cl4_qt = pivot_table_self(df_analy_ks, conditions_13, 'strategy_240703_4_Bin', view_m_list_cl240703)
# 策略240703_10
df_cl10_zm = pivot_table_self(df_analy_ks, conditions_11, 'strategy_240703_10_Bin', view_m_list_cl240703)
df_cl10_ss = pivot_table_self(df_analy_ks, conditions_12, 'strategy_240703_10_Bin', view_m_list_cl240703)
df_cl10_qt = pivot_table_self(df_analy_ks, conditions_13, 'strategy_240703_10_Bin', view_m_list_cl240703)
# 策略240703_14
df_cl14_zm = pivot_table_self(df_analy_ks, conditions_11, 'strategy_240703_14_Bin', view_m_list_cl240703)
df_cl14_ss = pivot_table_self(df_analy_ks, conditions_12, 'strategy_240703_14_Bin', view_m_list_cl240703)
df_cl14_qt = pivot_table_self(df_analy_ks, conditions_13, 'strategy_240703_14_Bin', view_m_list_cl240703)
# strategy_240801_1
df_s1_zm = pivot_table_self(df_analy_ks, conditions_11, 'strategy_240801_1_Bin', view_m_list_cl240801)
df_s1_ss = pivot_table_self(df_analy_ks, conditions_12, 'strategy_240801_1_Bin', view_m_list_cl240801)
df_s1_qt = pivot_table_self(df_analy_ks, conditions_13, 'strategy_240801_1_Bin', view_m_list_cl240801)
# strategy_240801_2
df_s2_zm = pivot_table_self(df_analy_ks, conditions_11, 'strategy_240801_2_Bin', view_m_list_cl240801) 
df_s2_ss = pivot_table_self(df_analy_ks, conditions_12, 'strategy_240801_2_Bin', view_m_list_cl240801)
df_s2_qt = pivot_table_self(df_analy_ks, conditions_13, 'strategy_240801_2_Bin', view_m_list_cl240801)
# strategy_240801_3
df_s3_zm = pivot_table_self(df_analy_ks, conditions_11, 'strategy_240801_3_Bin', view_m_list_cl240801)
df_s3_ss = pivot_table_self(df_analy_ks, conditions_12, 'strategy_240801_3_Bin', view_m_list_cl240801)
df_s3_qt = pivot_table_self(df_analy_ks, conditions_13, 'strategy_240801_3_Bin', view_m_list_cl240801)
# strategy_240801_4
df_s4_zm = pivot_table_self(df_analy_ks, conditions_11, 'strategy_240801_4_Bin', view_m_list_cl240801)
df_s4_ss = pivot_table_self(df_analy_ks, conditions_12, 'strategy_240801_4_Bin', view_m_list_cl240801)
df_s4_qt = pivot_table_self(df_analy_ks, conditions_13, 'strategy_240801_4_Bin', view_m_list_cl240801)
# strategy_240801_5
df_s5_zm = pivot_table_self(df_analy_ks, conditions_11, 'strategy_240801_5_Bin', view_m_list_cl240801)
df_s5_ss = pivot_table_self(df_analy_ks, conditions_12, 'strategy_240801_5_Bin', view_m_list_cl240801)
df_s5_qt = pivot_table_self(df_analy_ks, conditions_13, 'strategy_240801_5_Bin', view_m_list_cl240801)
# strategy_240801_7
df_s7_zm = pivot_table_self(df_analy_ks, conditions_11, 'strategy_240801_7_Bin', view_m_list_cl240801)
df_s7_ss = pivot_table_self(df_analy_ks, conditions_12, 'strategy_240801_7_Bin', view_m_list_cl240801)
df_s7_qt = pivot_table_self(df_analy_ks, conditions_13, 'strategy_240801_7_Bin', view_m_list_cl240801)
# strategy_240801_8
df_s8_zm = pivot_table_self(df_analy_ks, conditions_11, 'strategy_240801_8_Bin', view_m_list_cl240801)
df_s8_ss = pivot_table_self(df_analy_ks, conditions_12, 'strategy_240801_8_Bin', view_m_list_cl240801)
df_s8_qt = pivot_table_self(df_analy_ks, conditions_13, 'strategy_240801_8_Bin', view_m_list_cl240801)
# strategy_240801_9
df_s9_zm = pivot_table_self(df_analy_ks, conditions_11, 'strategy_240801_9_Bin', view_m_list_cl240801)
df_s9_ss = pivot_table_self(df_analy_ks, conditions_12, 'strategy_240801_9_Bin', view_m_list_cl240801)
df_s9_qt = pivot_table_self(df_analy_ks, conditions_13, 'strategy_240801_9_Bin', view_m_list_cl240801)
# strategy_240801_10
df_s10_zm = pivot_table_self(df_analy_ks, conditions_11, 'strategy_240801_10_Bin', view_m_list_cl240801)
df_s10_ss = pivot_table_self(df_analy_ks, conditions_12, 'strategy_240801_10_Bin', view_m_list_cl240801)
df_s10_qt = pivot_table_self(df_analy_ks, conditions_13, 'strategy_240801_10_Bin', view_m_list_cl240801)
# strategy_240801_11
df_s11_zm = pivot_table_self(df_analy_ks, conditions_11, 'strategy_240801_11_Bin', view_m_list_cl240801)
df_s11_ss = pivot_table_self(df_analy_ks, conditions_12, 'strategy_240801_11_Bin', view_m_list_cl240801)
df_s11_qt = pivot_table_self(df_analy_ks, conditions_13, 'strategy_240801_11_Bin', view_m_list_cl240801)
# score_201048
df_sc48_zm = pivot_table_self(df_analy_ks, conditions_11, 'score_201048_Bin', view_m_list)
df_sc48_ss = pivot_table_self(df_analy_ks, conditions_12, 'score_201048_Bin', view_m_list)
df_sc48_qt = pivot_table_self(df_analy_ks, conditions_13, 'score_201048_Bin', view_m_list)
# score_202098
df_sc98_zm = pivot_table_self(df_analy_ks, conditions_11, 'score_202098_Bin', view_m_list)
df_sc98_ss = pivot_table_self(df_analy_ks, conditions_12, 'score_202098_Bin', view_m_list)
df_sc98_qt = pivot_table_self(df_analy_ks, conditions_13, 'score_202098_Bin', view_m_list)
# score_202100
df_sc00_zm = pivot_table_self(df_analy_ks, conditions_11, 'score_202100_Bin', view_m_list)
df_sc00_ss = pivot_table_self(df_analy_ks, conditions_12, 'score_202100_Bin', view_m_list)
df_sc00_qt = pivot_table_self(df_analy_ks, conditions_13, 'score_202100_Bin', view_m_list)
# 317_极信FICO分
df_fico_zm = pivot_table_self(df_analy_ks, conditions_11, 'FICO_Bin', view_m_list_rule)
df_fico_ss = pivot_table_self(df_analy_ks, conditions_12, 'FICO_Bin', view_m_list_rule)
df_fico_qt = pivot_table_self(df_analy_ks, conditions_13, 'FICO_Bin', view_m_list_rule)
# 规则42
df_rule42_zm = pivot_table_self(df_analy_ks, conditions_11, '规则42', view_m_list_rule)
df_rule42_ss = pivot_table_self(df_analy_ks, conditions_12, '规则42', view_m_list_rule)
df_rule42_qt = pivot_table_self(df_analy_ks, conditions_13, '规则42', view_m_list_rule)
# 履约保护
df_lybh_zm = pivot_table_self(df_analy_ks, conditions_11, '履约保护_Bin', view_m_list_ly)
df_lybh_ss = pivot_table_self(df_analy_ks, conditions_12, '履约保护_Bin', view_m_list_ly)
df_lybh_qt = pivot_table_self(df_analy_ks, conditions_13, '履约保护_Bin', view_m_list_ly)
# TD212
df_td212_zm = pivot_table_self(df_analy_ks, conditions_11, 'td212_Bin', view_m_list_ydf2)
df_td212_ss = pivot_table_self(df_analy_ks, conditions_12, 'td212_Bin', view_m_list_ydf2)
df_td212_qt = pivot_table_self(df_analy_ks, conditions_13, 'td212_Bin', view_m_list_ydf2)
# 数控风险等级
df_sk_zm = pivot_table_self(df_analy_ks, conditions_11, 'zm_jh_t0_level_Bin', view_m_list)
df_sk_ss = pivot_table_self(df_analy_ks, conditions_12, 'zm_jh_t0_level_Bin', view_m_list)
df_sk_qt = pivot_table_self(df_analy_ks, conditions_13, 'zm_jh_t0_level_Bin', view_m_list)
# 模型 2311
df_model_2311 = pivot_table_self(df_analy_ks, conditions_model1, 'model_score_2311_Bin', view_m_list_model)
df_model_2311_zm = pivot_table_self(df_analy_ks, (conditions_model1&conditions_11), 'model_score_2311_Bin', view_m_list_model)
df_model_2311_ss = pivot_table_self(df_analy_ks, (conditions_model1&conditions_12), 'model_score_2311_Bin', view_m_list_model)
df_model_2311_qt = pivot_table_self(df_analy_ks, (conditions_model1&conditions_13), 'model_score_2311_Bin', view_m_list_model)
# 模型 yl
df_model_yl = pivot_table_self(df_analy_ks, conditions_model2, 'model_score_yl_Bin', view_m_list_model)
df_model_yl_zm = pivot_table_self(df_analy_ks, (conditions_model2&conditions_11), 'model_score_yl_Bin', view_m_list_model)
df_model_yl_ss = pivot_table_self(df_analy_ks, (conditions_model2&conditions_12), 'model_score_yl_Bin', view_m_list_model)
df_model_yl_qt = pivot_table_self(df_analy_ks, (conditions_model2&conditions_13), 'model_score_yl_Bin', view_m_list_model)
# 模型 2401
df_model_2401 = pivot_table_self(df_analy_ks, conditions_model3, 'model_score_2401_Bin', view_m_list_model)
df_model_2401_zm = pivot_table_self(df_analy_ks, (conditions_model3&conditions_11), 'model_score_2401_Bin', view_m_list_model)
df_model_2401_ss = pivot_table_self(df_analy_ks, (conditions_model3&conditions_12), 'model_score_2401_Bin', view_m_list_model)
df_model_2401_qt = pivot_table_self(df_analy_ks, (conditions_model3&conditions_13), 'model_score_2401_Bin', view_m_list_model)
# 模型 2501
df_model_2501 = pivot_table_self(df_analy_ks, conditions_model4, 'model_score_2501_Bin', view_m_list_model2)
df_model_2501_zm = pivot_table_self(df_analy_ks, (conditions_model4&conditions_11), 'model_score_2501_Bin', view_m_list_model2)
df_model_2501_ss = pivot_table_self(df_analy_ks, (conditions_model4&conditions_12), 'model_score_2501_Bin', view_m_list_model2)
df_model_2501_qt = pivot_table_self(df_analy_ks, (conditions_model4&conditions_13), 'model_score_2501_Bin', view_m_list_model2)

### 10_13_12 PSI监控

In [188]:
df_analy_ks['是否出库'] = 1
df_risk_new = df_risk_new.merge(df_analy_ks[['order_number','是否出库']], on = 'order_number', how = 'left')
df_info_ori = df_info_ori.merge(df_analy_ks[['order_number','是否出库']], on = 'order_number', how = 'left')

conditions_2 = (df_risk_new.是否出库 == 1)
conditions_21 = (~df_info_ori.拒绝理由.isin(['[命中非免押用户拒绝]','命中非免押用户拒绝']))
conditions_3 = ((~df_risk_new.归属渠道.isin(['抖音渠道','广州易合顺','深圳刘全兵号卡1'])) & (df_risk_new.是否拒量 == 0))
conditions_31 = (df_info_ori.是否出库 == 1)
conditions_32 = ((df_info_ori.审核状态 != '机审强拒') & (df_info_ori.是否拒量 == 0))
conditions_4 = ((~df_info_ori.归属渠道.isin(['抖音渠道','广州易合顺','深圳刘全兵号卡1'])) & (df_info_ori.是否拒量 == 0))
conditions_51 = (df_info_ori.is_model == '1')
conditions_52 = (df_info_ori.is_model == '2')
conditions_53 = (df_info_ori.is_model == '3')
conditions_54 = (df_info_ori.is_model == '4')
conditions_6 = ['单人聊天会话中的小程序消息卡片（分享）','八派信息','CPS直播','派金花','九州信息','勉丫租','哈银','租瓜直播','美仑美奂','分期乐1','分期乐2','98租超','推一推','硬派抖音']


conditions_all = conditions_21 & conditions_31 & conditions_4
conditions_all1 = conditions_21 & conditions_31 & conditions_4
conditions_hmf = conditions_2 & conditions_3
conditions_ckq = conditions_21 & conditions_31 & conditions_32 & conditions_4
conditions_ckq1 = conditions_21 & conditions_32

In [310]:
# 模型2311
df_info_ori['model_score_2311_result']= df_info_ori.apply(lambda x: '1.accept' if x.归属渠道 != '搜索渠道'  and x.model_score_2311 >= 0  and x.model_score_2311 <= 18.321
                                                else '2.review' if x.归属渠道 != '搜索渠道'  and x.model_score_2311 > 18.321 and x.model_score_2311 <= 23.297
                                                else '3.reject' if x.归属渠道 != '搜索渠道'  and x.model_score_2311 > 23.297 
                                                else '1.accept' if x.归属渠道 == '搜索渠道'  and x.model_score_2311 >= 0  and x.model_score_2311 <= 16.6
                                                else '2.review' if x.归属渠道 == '搜索渠道'  and x.model_score_2311 > 16.6 and x.model_score_2311 <= 21.435
                                                else '3.reject' if x.归属渠道 == '搜索渠道'  and x.model_score_2311 > 21.435
                                                else 'other'
                                                , axis = 1)
df_info_ori['model_score_2311_result'] = np.where((df_info_ori.归属渠道.isin(conditions_6)) & (df_info_ori.model_score_2311_result == '2.review'), '3.reject', df_info_ori.model_score_2311_result)
df_info_ori['model_score_2311_result'] = np.where((df_info_ori.归属渠道.isin(conditions_6)) & (df_info_ori.model_score_2311_result == '1.accept'), '2.review', df_info_ori.model_score_2311_result)
df_info_ori['model_score_2311_result_f'] = np.where(df_info_ori.model_score_2311_result.isin(['1.accept','2.review']), '模型通过', '模型拒绝')
# 模型yl
df_info_ori['model_score_yl_result']= df_info_ori.apply(lambda x: '1.accept' if x.归属渠道 != '搜索渠道'  and x.model_score_yl >= 0  and x.model_score_yl <= 17.55
                                                else '2.review' if x.归属渠道 != '搜索渠道'  and x.model_score_yl > 17.55 and x.model_score_yl <= 23.382
                                                else '3.reject' if x.归属渠道 != '搜索渠道'  and x.model_score_yl > 23.382
                                                else '1.accept' if x.归属渠道 == '搜索渠道'  and x.model_score_yl >= 0  and x.model_score_yl <= 15.059
                                                else '2.review' if x.归属渠道 == '搜索渠道'  and x.model_score_yl > 15.059 and x.model_score_yl <= 21.661
                                                else '3.reject' if x.归属渠道 == '搜索渠道'  and x.model_score_yl > 21.661
                                                else 'other'
                                                , axis = 1)
df_info_ori['model_score_yl_result'] = np.where((df_info_ori.归属渠道.isin(conditions_6)) & (df_info_ori.model_score_yl_result == '2.review'), '3.reject', df_info_ori.model_score_yl_result)
df_info_ori['model_score_yl_result'] = np.where((df_info_ori.归属渠道.isin(conditions_6)) & (df_info_ori.model_score_yl_result == '1.accept'), '2.review', df_info_ori.model_score_yl_result)
df_info_ori['model_score_yl_result'] = np.where((df_info_ori.下单日期 >= '2025-03-28') & (df_info_ori.model_score_yl_result == '3.reject') & (df_info_ori.model_score_yl <= 24.5) & (df_info_ori.归属渠道.isin(['芝麻租物','搜索渠道'])), '2.review', df_info_ori.model_score_yl_result)
df_info_ori['model_score_yl_result_f'] = np.where(df_info_ori.model_score_yl_result.isin(['1.accept','2.review']), '模型通过', '模型拒绝')
# 模型2401
df_info_ori['model_score_2401_result']= df_info_ori.apply(lambda x: '1.accept' if x.归属渠道 != '搜索渠道'  and x.model_score_2401 >= 0  and x.model_score_2401 <= 11.04
                                                else '2.review' if x.归属渠道 != '搜索渠道'  and x.model_score_2401 > 11.04 and x.model_score_2401 <= 16
                                                else '3.reject' if x.归属渠道 != '搜索渠道'  and x.model_score_2401 > 16
                                                else '1.accept' if x.归属渠道 == '搜索渠道'  and x.model_score_2401 >= 0  and x.model_score_2401 <= 9.35
                                                else '2.review' if x.归属渠道 == '搜索渠道'  and x.model_score_2401 > 9.35 and x.model_score_2401 <= 14.93
                                                else '3.reject' if x.归属渠道 == '搜索渠道'  and x.model_score_2401 > 14.93
                                                else 'other'
                                                , axis = 1)
df_info_ori['model_score_2401_result'] = np.where((df_info_ori.归属渠道.isin(conditions_6)) & (df_info_ori.model_score_2401_result == '2.review'), '3.reject', df_info_ori.model_score_2401_result)
df_info_ori['model_score_2401_result'] = np.where((df_info_ori.归属渠道.isin(conditions_6)) & (df_info_ori.model_score_2401_result == '1.accept'), '2.review', df_info_ori.model_score_2401_result)
df_info_ori['model_score_2401_result_f'] = np.where(df_info_ori.model_score_2401_result.isin(['1.accept','2.review']), '模型通过', '模型拒绝')
# 模型2501
df_info_ori['model_score_2501_result']= df_info_ori.apply(lambda x: '1.accept' if x.归属渠道 == '芝麻租物'  and x.model_score_2501 >= 0  and x.model_score_2501 <= 14.104
                                                else '2.review' if x.归属渠道 == '芝麻租物'  and x.model_score_2501 > 14.104 and x.model_score_2501 <= 18.1312
                                                else '3.reject' if x.归属渠道 == '芝麻租物'  and x.model_score_2501 > 18.1312
                                                else '1.accept' if x.归属渠道 != '芝麻租物'  and x.model_score_2501 >= 0  and x.model_score_2501 <= 13.79
                                                else '2.review' if x.归属渠道 != '芝麻租物'  and x.model_score_2501 > 13.79 and x.model_score_2501 <= 18.098
                                                else '3.reject' if x.归属渠道 != '芝麻租物'  and x.model_score_2501 > 18.098
                                                else 'other'
                                                , axis = 1)
df_info_ori['model_score_2501_result'] = np.where((df_info_ori.归属渠道.isin(conditions_6)) & (df_info_ori.model_score_2501_result == '2.review'), '3.reject', df_info_ori.model_score_2501_result)
df_info_ori['model_score_2501_result'] = np.where((df_info_ori.归属渠道.isin(conditions_6)) & (df_info_ori.model_score_2501_result == '1.accept'), '2.review', df_info_ori.model_score_2501_result)
df_info_ori['model_score_2501_result_f'] = np.where(df_info_ori.model_score_2501_result.isin(['1.accept','2.review']), '模型通过', '模型拒绝')


In [354]:
# 履约历史等级
df_lydj = psi_data(df_info_ori, conditions_all, 'rentOnlineRiskLevelFulfill_Bin', '2024-10', conditions_21)
# 综合风险等级 rentOnlineRiskLevelOverall
df_zhdj = psi_data(df_info_ori, conditions_all, 'rentOnlineRiskLevelOverall_Bin','2024-10', conditions_21)
# 细化综合风险等级 rentOnlineDetailLevelOverall
df_xhdj = psi_data(df_info_ori, conditions_all, 'rentOnlineDetailLevelOverall_Bin','2025-01', conditions_21)
# 蚁盾分1.0 '蚁盾分_Bin'
df_ydf1 = psi_data(df_info_ori, conditions_all, '蚁盾分_Bin','2024-10', conditions_21)
# 蚁盾分2.0 'my_zl_score2_Bin'
df_ydf2 = psi_data(df_info_ori, conditions_all, 'my_zl_score2_Bin','2025-03', conditions_21)
# 履约保护 '履约保护_Bin'
df_lybh = psi_data(df_info_ori, conditions_ckq, '履约保护_Bin','2024-10', conditions_ckq1)
# 数控风险等级 'zm_jh_t0_level_Bin'
df_skfx = psi_data(df_info_ori, conditions_ckq, 'zm_jh_t0_level_Bin','2024-10', conditions_ckq1)

# 火眸分
df_hmf = psi_data(df_risk_new, conditions_hmf, '火眸分_Bin', '2024-10')
# 青云分
df_qyf = psi_data(df_info_ori, conditions_all, '青云分_Bin','2024-10', conditions_21)
# 云商分
df_ysf = psi_data(df_info_ori, conditions_all, 'br_ys_score_Bin','2024-10', conditions_21)
# 融安分
df_raf = psi_data(df_info_ori, conditions_all, '融安分_Bin','2024-10', conditions_21)

# 极信FICO分
df_fico = psi_data(df_info_ori, conditions_all, 'FICO_Bin','2024-10', conditions_21)
# TD212
df_td212 = psi_data(df_info_ori, conditions_ckq, 'td212_Bin','2024-10', conditions_ckq1)

# jx_yl_UPPC134_Bin
df_yl134 = psi_data(df_info_ori, conditions_all, 'jx_yl_UPPC134_Bin','2024-10', conditions_21)
# jx_yl_UPPD007_Bin
df_yl007 = psi_data(df_info_ori, conditions_all, 'jx_yl_UPPD007_Bin','2024-10', conditions_21)
# jx_yl_UPPE074_Bin
df_yl074 = psi_data(df_info_ori, conditions_all, 'jx_yl_UPPE074_Bin','2024-10', conditions_21)
# jx_yl_UPPE082_Bin
df_yl082 = psi_data(df_info_ori, conditions_all, 'jx_yl_UPPE082_Bin','2024-10', conditions_21)

# xy_qj_C22180008_Bin
df_qj008 = psi_data(df_info_ori, conditions_all, 'xy_qj_C22180008_Bin','2024-10', conditions_21)
# xy_qj_B22170035_Bin
df_qj035 = psi_data(df_info_ori, conditions_all, 'xy_qj_B22170035_Bin','2024-10', conditions_21)
# xy_qj_B22170036_Bin
df_qj036 = psi_data(df_info_ori, conditions_all, 'xy_qj_B22170036_Bin','2024-10', conditions_21)
# xy_qj_A22160001_Bin
df_qj001 = psi_data(df_info_ori, conditions_all, 'xy_qj_A22160001_Bin','2024-10', conditions_21)

# score_201048_Bin
df_201048 = psi_data(df_info_ori, conditions_all1, 'score_201048_Bin','2025-03', conditions_21)
# score_202098_Bin
df_202098 = psi_data(df_info_ori, conditions_all1, 'score_202098_Bin','2025-03', conditions_21)
# score_202100_Bin
df_202100 = psi_data(df_info_ori, conditions_all1, 'score_202100_Bin','2025-03', conditions_21)

# 评分等级
df_grade = psi_data(df_info_ori, conditions_all, 'grade_Bin','2024-10', conditions_21)

# 模型 2311 进件
df_model_2311_p, df_model_2311_m = psi_data(df_info_ori, (conditions_51&conditions_21), 'model_score_2311_Bin', '2024-03', conditions_21, '2311')
# 2311通过和出库
df_model_2311_mt, df_model_2311_mc = psi_data(df_info_ori, (conditions_31&conditions_4&conditions_21), 'model_score_2311_Bin', '2024-03', (conditions_51 & (df_info_ori.model_score_2311_result_f == '模型通过')&conditions_21), '2311')
# 获取并关联通过占比
df_model_2311_mt = rate(df_info_ori, df_model_2311_mt, (conditions_51&conditions_21), (conditions_51 & (df_info_ori.model_score_2311_result_f == '模型通过')&conditions_21), 'model_score_2311_Bin', '2024-03')
# 模型 yl 进件
df_model_yl_p, df_model_yl_m = psi_data(df_info_ori, (conditions_52&conditions_21), 'model_score_yl_Bin', '2024-03', conditions_21, 'yl')
# yl通过和出库
df_model_yl_mt, df_model_yl_mc = psi_data(df_info_ori, (conditions_31&conditions_4&conditions_21), 'model_score_yl_Bin', '2024-03', (conditions_52 & (df_info_ori.model_score_yl_result_f == '模型通过')&conditions_21), 'yl')
# 获取并关联通过占比
df_model_yl_mt = rate(df_info_ori, df_model_yl_mt, (conditions_52&conditions_21), (conditions_52 & (df_info_ori.model_score_yl_result_f == '模型通过')&conditions_21), 'model_score_yl_Bin', '2024-03')
# 模型 2401 进件
df_model_2401_p, df_model_2401_m = psi_data(df_info_ori, (conditions_53&conditions_21), 'model_score_2401_Bin', '2024-03', conditions_21, '2401')
# 2401通过和出库
df_model_2401_mt, df_model_2401_mc = psi_data(df_info_ori, (conditions_31&conditions_4&conditions_21), 'model_score_2401_Bin', '2024-03', (conditions_53 & (df_info_ori.model_score_2401_result_f == '模型通过')&conditions_21), '2401')
# 获取并关联通过占比
df_model_2401_mt = rate(df_info_ori, df_model_2401_mt, (conditions_53&conditions_21), (conditions_53 & (df_info_ori.model_score_2401_result_f == '模型通过')&conditions_21), 'model_score_2401_Bin', '2024-03')
# 模型 2501 进件
df_model_2501_p, df_model_2501_m = psi_data(df_info_ori, (conditions_54&conditions_21), 'model_score_2501_Bin', '2025-03', conditions_21, '2401')
# 2501通过和出库
df_model_2501_mt, df_model_2501_mc = psi_data(df_info_ori, (conditions_31&conditions_4&conditions_21), 'model_score_2501_Bin', '2025-03', (conditions_54 & (df_info_ori.model_score_2501_result_f == '模型通过')&conditions_21), '2501')
# 获取并关联通过占比
df_model_2501_mt = rate(df_info_ori, df_model_2501_mt, (conditions_54&conditions_21), (conditions_54 & (df_info_ori.model_score_2501_result_f == '模型通过')&conditions_21), 'model_score_2501_Bin', '2025-03')


In [1]:
df_model_2311_mt

NameError: name 'df_model_2311_mt' is not defined

### 数据写入

In [357]:
# 风控模型
# df_list = [[df_hmf_zm, df_hmf_ss,df_hmf_qt],[df_ly_zm,df_ly_ss,df_ly_qt], [df_zhfx_zm,df_zhfx_ss,df_zhfx_qt], [df_xf_zm,df_xf_ss,df_xf_qt], [df_ydf1_zm,df_ydf1_ss,df_ydf1_qt], 
#         [df_ydf2_zm,df_ydf2_ss,df_ydf2_qt], [df_ysf_zm,df_ysf_ss,df_ysf_qt], [df_raf_zm,df_raf_ss,df_raf_qt], [df_qyf_zm,df_qyf_ss,df_qyf_qt], [df_pf_zm,df_pf_ss,df_pf_qt]
#         , [df_cl4_zm,df_cl4_ss,df_cl4_qt], [df_cl10_zm,df_cl10_ss,df_cl10_qt], [df_cl14_zm,df_cl14_ss,df_cl14_qt], [df_s1_zm,df_s1_ss,df_s1_qt], [df_s2_zm,df_s2_ss,df_s2_qt]
#         , [df_s3_zm,df_s3_ss,df_s3_qt], [df_s4_zm,df_s4_ss,df_s4_qt], [df_s5_zm,df_s5_ss,df_s5_qt], [df_s7_zm,df_s7_ss,df_s7_qt], [df_s8_zm,df_s8_ss,df_s8_qt]
#         , [df_s9_zm,df_s9_ss,df_s9_qt], [df_s10_zm,df_s10_ss,df_s10_qt], [df_s11_zm,df_s11_ss,df_s11_qt], [df_sc48_zm,df_sc48_ss,df_sc48_qt], [df_sc98_zm,df_sc98_ss,df_sc98_qt]
#         , [df_sc00_zm,df_sc00_ss,df_sc00_qt], [df_fico_zm,df_fico_ss,df_fico_qt], [df_rule42_zm,df_rule42_ss,df_rule42_qt], [df_lybh_zm,df_lybh_ss,df_lybh_qt], [df_td212_zm,df_td212_ss,df_td212_qt]
#         , [df_sk_zm,df_sk_ss,df_sk_qt]]
# Open_Excel2(df_list, path, start_row=2, start_col=2, key='风控监控')

# 风控自有模型
# path_model = 'F:/租后表分析数据_ld/风控周报/3_风控监控_模型_auto - 副本.xlsx'
# df_model_list = [[df_model_2311_p, df_model_2311_m,df_model_2311_mt,df_model_2311_mc],[df_model_yl_p,df_model_yl_m,df_model_yl_mt,df_model_yl_mc],
#                 [df_model_2401_p,df_model_2401_m,df_model_2401_mt,df_model_2401_mc],[df_model_2501_p,df_model_2501_m,df_model_2501_mt,df_model_2501_mc],
#                 [df_model_2311,df_model_2311_zm,df_model_2311_ss,df_model_2311_qt],[df_model_yl,df_model_yl_zm,df_model_yl_ss,df_model_yl_qt],
#                 [df_model_2401,df_model_2401_zm,df_model_2401_ss,df_model_2401_qt], [df_model_2501,df_model_2501_zm,df_model_2501_ss,df_model_2501_qt]]
# Open_Excel2(df_model_list, path_model, start_row=2, start_col=2)

# PSI
# path_psi = 'F:/租后表分析数据_ld/风控周报/2_风控监控_PSI_auto - 副本.xlsx'
# df_psi_list = [[df_lydj, df_zhdj, df_ydf1, df_lybh, df_skfx], [df_xhdj], [df_hmf, df_qyf], [df_ysf, df_raf], [df_fico, df_td212], [df_yl134, df_yl007, df_yl074, df_yl082],
#             [df_qj008, df_qj035, df_qj036, df_qj001], [df_201048, df_202098, df_202100, df_ydf2], [df_grade]]
# Open_Excel2(df_psi_list, path_psi, start_row=2, start_col=2)

2

In [285]:
# # 将数据写入excel
# # 火眸分
# row_len = Open_Excel2(df_hmf_zm, path, sheet_name='21_火眸分', start_row=3, start_col=2)
# row_len1 = Open_Excel2(df_hmf_ss, path, sheet_name='21_火眸分', start_row=row_len+2, start_col=2)
# row_len2 = Open_Excel2(df_hmf_qt, path, sheet_name='21_火眸分', start_row=row_len1+2, start_col=2)
# # 履约历史等级
# row_len = Open_Excel2(df_ly_zm, path, sheet_name='31_履约历史等级', start_row=3, start_col=2)
# row_len1 = Open_Excel2(df_ly_ss, path, sheet_name='31_履约历史等级', start_row=row_len+2, start_col=2)
# row_len2 = Open_Excel2(df_ly_qt, path, sheet_name='31_履约历史等级', start_row=row_len1+2, start_col=2)
# # 综合风险等级
# row_len = Open_Excel2(df_zhfx_zm, path, sheet_name='32_综合风险等级', start_row=3, start_col=2)
# row_len1 = Open_Excel2(df_zhfx_ss, path, sheet_name='32_综合风险等级', start_row=row_len+2, start_col=2)
# row_len2 = Open_Excel2(df_zhfx_qt, path, sheet_name='32_综合风险等级', start_row=row_len1+2, start_col=2)
# # 细分综合风险等级
# row_len = Open_Excel2(df_xf_zm, path, sheet_name='33_细分综合风险等级', start_row=3, start_col=2)
# row_len1 = Open_Excel2(df_xf_ss, path, sheet_name='33_细分综合风险等级', start_row=row_len+2, start_col=2)
# row_len2 = Open_Excel2(df_xf_qt, path, sheet_name='33_细分综合风险等级', start_row=row_len1+2, start_col=2)
# # 蚁盾分1.0
# row_len = Open_Excel2(df_ydf1_zm, path, sheet_name='34_蚁盾分1.0', start_row=3, start_col=2)
# row_len1 = Open_Excel2(df_ydf1_ss, path, sheet_name='34_蚁盾分1.0', start_row=row_len+2, start_col=2)
# row_len2 = Open_Excel2(df_ydf1_qt, path, sheet_name='34_蚁盾分1.0', start_row=row_len1+2, start_col=2)
# # 蚁盾分2.0
# row_len = Open_Excel2(df_ydf2_zm, path, sheet_name='35_蚁盾分2.0', start_row=3, start_col=2)
# row_len1 = Open_Excel2(df_ydf2_ss, path, sheet_name='35_蚁盾分2.0', start_row=row_len+2, start_col=2)
# row_len2 = Open_Excel2(df_ydf2_qt, path, sheet_name='35_蚁盾分2.0', start_row=row_len1+2, start_col=2)
# # 云商分
# row_len = Open_Excel2(df_ysf_zm, path, sheet_name='36_云商分', start_row=3, start_col=2)
# row_len1 = Open_Excel2(df_ysf_ss, path, sheet_name='36_云商分', start_row=row_len+2, start_col=2)
# row_len2 = Open_Excel2(df_ysf_qt, path, sheet_name='36_云商分', start_row=row_len1+2, start_col=2)
# # 融安分
# row_len = Open_Excel2(df_raf_zm, path, sheet_name='37_融安分', start_row=3, start_col=2)
# row_len1 = Open_Excel2(df_raf_ss, path, sheet_name='37_融安分', start_row=row_len+2, start_col=2)
# row_len2 = Open_Excel2(df_raf_qt, path, sheet_name='37_融安分', start_row=row_len1+2, start_col=2)
# # 青云分
# row_len = Open_Excel2(df_qyf_zm, path, sheet_name='39_青云分', start_row=3, start_col=2)
# row_len1 = Open_Excel2(df_qyf_ss, path, sheet_name='39_青云分', start_row=row_len+2, start_col=2)
# row_len2 = Open_Excel2(df_qyf_qt, path, sheet_name='39_青云分', start_row=row_len1+2, start_col=2)
# # 评分等级
# row_len = Open_Excel2(df_pf_zm, path, sheet_name='310_评分等级', start_row=3, start_col=2)
# row_len1 = Open_Excel2(df_pf_ss, path, sheet_name='310_评分等级', start_row=row_len+2, start_col=2)
# row_len2 = Open_Excel2(df_pf_qt, path, sheet_name='310_评分等级', start_row=row_len1+2, start_col=2)
# # 策略240703_04
# row_len = Open_Excel2(df_cl4_zm, path, sheet_name='312_策略240703_4', start_row=3, start_col=2)
# row_len1 = Open_Excel2(df_cl4_ss, path, sheet_name='312_策略240703_4', start_row=row_len+2, start_col=2)
# row_len2 = Open_Excel2(df_cl4_qt, path, sheet_name='312_策略240703_4', start_row=row_len1+2, start_col=2)
# # 策略240703_10
# row_len = Open_Excel2(df_cl10_zm, path, sheet_name='313_策略240703_10', start_row=3, start_col=2)
# row_len1 = Open_Excel2(df_cl10_ss, path, sheet_name='313_策略240703_10', start_row=row_len+2, start_col=2)
# row_len2 = Open_Excel2(df_cl10_qt, path, sheet_name='313_策略240703_10', start_row=row_len1+2, start_col=2)
# # 策略240703_14
# row_len = Open_Excel2(df_cl14_zm, path, sheet_name='314_策略240703_14', start_row=3, start_col=2)
# row_len1 = Open_Excel2(df_cl14_ss, path, sheet_name='314_策略240703_14', start_row=row_len+2, start_col=2)
# row_len2 = Open_Excel2(df_cl14_qt, path, sheet_name='314_策略240703_14', start_row=row_len1+2, start_col=2)
# # strategy_240801_1
# row_len = Open_Excel2(df_s1_zm, path, sheet_name='315_策略240801_1', start_row=3, start_col=2)
# row_len1 = Open_Excel2(df_s1_ss, path, sheet_name='315_策略240801_1', start_row=row_len+2, start_col=2)
# row_len2 = Open_Excel2(df_s1_qt, path, sheet_name='315_策略240801_1', start_row=row_len1+2, start_col=2)
# # strategy_240801_2
# row_len = Open_Excel2(df_s2_zm, path, sheet_name='315_策略240801_2', start_row=3, start_col=2)
# row_len1 = Open_Excel2(df_s2_ss, path, sheet_name='315_策略240801_2', start_row=row_len+2, start_col=2)
# row_len2 = Open_Excel2(df_s2_qt, path, sheet_name='315_策略240801_2', start_row=row_len1+2, start_col=2)
# # strategy_240801_3
# row_len = Open_Excel2(df_s3_zm, path, sheet_name='315_策略240801_3', start_row=3, start_col=2)
# row_len1 = Open_Excel2(df_s3_ss, path, sheet_name='315_策略240801_3', start_row=row_len+2, start_col=2)
# row_len2 = Open_Excel2(df_s3_qt, path, sheet_name='315_策略240801_3', start_row=row_len1+2, start_col=2)
# # strategy_240801_4
# row_len = Open_Excel2(df_s4_zm, path, sheet_name='315_策略240801_4', start_row=3, start_col=2)
# row_len1 = Open_Excel2(df_s4_ss, path, sheet_name='315_策略240801_4', start_row=row_len+2, start_col=2)
# row_len2 = Open_Excel2(df_s4_qt, path, sheet_name='315_策略240801_4', start_row=row_len1+2, start_col=2)
# # strategy_240801_5
# row_len = Open_Excel2(df_s5_zm, path, sheet_name='315_策略240801_5', start_row=3, start_col=2)
# row_len1 = Open_Excel2(df_s5_ss, path, sheet_name='315_策略240801_5', start_row=row_len+2, start_col=2)
# row_len2 = Open_Excel2(df_s5_qt, path, sheet_name='315_策略240801_5', start_row=row_len1+2, start_col=2)
# # strategy_240801_7
# row_len = Open_Excel2(df_s7_zm, path, sheet_name='315_策略240801_7', start_row=3, start_col=2)
# row_len1 = Open_Excel2(df_s7_ss, path, sheet_name='315_策略240801_7', start_row=row_len+2, start_col=2)
# row_len2 = Open_Excel2(df_s7_qt, path, sheet_name='315_策略240801_7', start_row=row_len1+2, start_col=2)
# # strategy_240801_8
# row_len = Open_Excel2(df_s8_zm, path, sheet_name='315_策略240801_8', start_row=3, start_col=2)
# row_len1 = Open_Excel2(df_s8_ss, path, sheet_name='315_策略240801_8', start_row=row_len+2, start_col=2)
# row_len2 = Open_Excel2(df_s8_qt, path, sheet_name='315_策略240801_8', start_row=row_len1+2, start_col=2)
# # strategy_240801_9
# row_len = Open_Excel2(df_s9_zm, path, sheet_name='315_策略240801_9', start_row=3, start_col=2)
# row_len1 = Open_Excel2(df_s9_ss, path, sheet_name='315_策略240801_9', start_row=row_len+2, start_col=2)
# row_len2 = Open_Excel2(df_s9_qt, path, sheet_name='315_策略240801_9', start_row=row_len1+2, start_col=2)
# # strategy_240801_10
# row_len = Open_Excel2(df_s10_zm, path, sheet_name='315_策略240801_10', start_row=3, start_col=2)
# row_len1 = Open_Excel2(df_s10_ss, path, sheet_name='315_策略240801_10', start_row=row_len+2, start_col=2)
# row_len2 = Open_Excel2(df_s10_qt, path, sheet_name='315_策略240801_10', start_row=row_len1+2, start_col=2)
# # strategy_240801_11
# row_len = Open_Excel2(df_s11_zm, path, sheet_name='315_策略240801_11', start_row=3, start_col=2)
# row_len1 = Open_Excel2(df_s11_ss, path, sheet_name='315_策略240801_11', start_row=row_len+2, start_col=2)
# row_len2 = Open_Excel2(df_s11_qt, path, sheet_name='315_策略240801_11', start_row=row_len1+2, start_col=2)
# # score_201048
# row_len = Open_Excel2(df_sc48_zm, path, sheet_name='316_融360_201048', start_row=3, start_col=2)
# row_len1 = Open_Excel2(df_sc48_ss, path, sheet_name='316_融360_201048', start_row=row_len+2, start_col=2)
# row_len2 = Open_Excel2(df_sc48_qt, path, sheet_name='316_融360_201048', start_row=row_len1+2, start_col=2)
# # score_202098
# row_len = Open_Excel2(df_sc98_zm, path, sheet_name='316_融360_202098', start_row=3, start_col=2)
# row_len1 = Open_Excel2(df_sc98_ss, path, sheet_name='316_融360_202098', start_row=row_len+2, start_col=2)
# row_len2 = Open_Excel2(df_sc98_qt, path, sheet_name='316_融360_202098', start_row=row_len1+2, start_col=2)
# # score_202100
# row_len = Open_Excel2(df_sc00_zm, path, sheet_name='316_融360_202100', start_row=3, start_col=2)
# row_len1 = Open_Excel2(df_sc00_ss, path, sheet_name='316_融360_202100', start_row=row_len+2, start_col=2)
# row_len2 = Open_Excel2(df_sc00_qt, path, sheet_name='316_融360_202100', start_row=row_len1+2, start_col=2)
# # 317_极信FICO分
# row_len = Open_Excel2(df_fico_zm, path, sheet_name='317_极信FICO分', start_row=3, start_col=2)
# row_len1 = Open_Excel2(df_fico_ss, path, sheet_name='317_极信FICO分', start_row=row_len+2, start_col=2)
# row_len2 = Open_Excel2(df_fico_qt, path, sheet_name='317_极信FICO分', start_row=row_len1+2, start_col=2)
# # 规则42
# row_len = Open_Excel2(df_rule42_zm, path, sheet_name='318_规则42', start_row=3, start_col=2)
# row_len1 = Open_Excel2(df_rule42_ss, path, sheet_name='318_规则42', start_row=row_len+2, start_col=2)
# row_len2 = Open_Excel2(df_rule42_qt, path, sheet_name='318_规则42', start_row=row_len1+2, start_col=2)
# # 履约保护
# row_len = Open_Excel2(df_lybh_zm, path, sheet_name='319_履约保护', start_row=3, start_col=2)
# row_len1 = Open_Excel2(df_lybh_ss, path, sheet_name='319_履约保护', start_row=row_len+2, start_col=2)
# row_len2 = Open_Excel2(df_lybh_qt, path, sheet_name='319_履约保护', start_row=row_len1+2, start_col=2)
# # TD212
# row_len = Open_Excel2(df_td212_zm, path, sheet_name='320_TD212', start_row=3, start_col=2)
# row_len1 = Open_Excel2(df_td212_ss, path, sheet_name='320_TD212', start_row=row_len+2, start_col=2)
# row_len2 = Open_Excel2(df_td212_qt, path, sheet_name='320_TD212', start_row=row_len1+2, start_col=2)
# # 数控风险等级
# row_len = Open_Excel2(df_sk_zm, path, sheet_name='321_数控风险等级', start_row=3, start_col=2)
# row_len1 = Open_Excel2(df_sk_ss, path, sheet_name='321_数控风险等级', start_row=row_len+2, start_col=2)
# row_len2 = Open_Excel2(df_sk_qt, path, sheet_name='321_数控风险等级', start_row=row_len1+2, start_col=2)
# # 模型2311
# row_len = Open_Excel2(df_model_2311, path, sheet_name='21_效果_model_2311', start_row=3, start_col=2)
# # 模型yl
# row_len = Open_Excel2(df_model_yl, path, sheet_name='22_效果_model_yl', start_row=3, start_col=2)
# # 模型2401
# row_len = Open_Excel2(df_model_2401, path, sheet_name='23_效果_model_2401', start_row=3, start_col=2)
# # 模型2501
# row_len = Open_Excel2(df_model_2501, path, sheet_name='24_效果_model_2501', start_row=3, start_col=2)

In [ ]:
# path_psi = 'F:/租后表分析数据_ld/风控周报/2_风控监控_PSI_模板 - 副本.xlsx'
# # 履约历史等级
# row_len = Open_Excel2(df_lydj, path_psi, sheet_name='1_支付宝系', start_row=3, start_col=2)
# # 综合风险等级
# row_len1 = Open_Excel2(df_zhdj, path_psi, sheet_name='1_支付宝系', start_row=row_len+3, start_col=2)
# # 蚁盾分1.0
# row_len2 = Open_Excel2(df_ydf1, path_psi, sheet_name='1_支付宝系', start_row=row_len1+3, start_col=2)
# # 履约保护
# row_len3 = Open_Excel2(df_lybh, path_psi, sheet_name='1_支付宝系', start_row=row_len2+3, start_col=2)
# # 数控风险等级
# Open_Excel2(df_skfx, path_psi, sheet_name='1_支付宝系', start_row=row_len3+3, start_col=2)

# # 细化综合风险等级
# Open_Excel2(df_xhdj, path_psi, sheet_name='2_支付宝系', start_row=3, start_col=2)

# # 火眸分
# row_len = Open_Excel2(df_hmf, path_psi, sheet_name='2_冰鉴', start_row=3, start_col=2)
# # 青云分
# Open_Excel2(df_qyf, path_psi, sheet_name='2_冰鉴', start_row=row_len+3, start_col=2)

# # 云商分
# row_len = Open_Excel2(df_ysf, path_psi, sheet_name='3_百融', start_row=3, start_col=2)
# # 融安分
# Open_Excel2(df_raf, path_psi, sheet_name='3_百融', start_row=row_len+3, start_col=2)

# # fico分
# row_len = Open_Excel2(df_fico, path_psi, sheet_name='4_极信', start_row=3, start_col=2)
# # td212
# Open_Excel2(df_td212, path_psi, sheet_name='4_极信', start_row=row_len+3, start_col=2)

# # jx_yl_UPPC134_Bin
# row_len = Open_Excel2(df_yl134, path_psi, sheet_name='5_银联', start_row=3, start_col=2)
# # jx_yl_UPPD007_Bin
# row_len1 = Open_Excel2(df_yl007, path_psi, sheet_name='5_银联', start_row=row_len+3, start_col=2)
# # jx_yl_UPPE074_Bin
# row_len2 = Open_Excel2(df_yl074, path_psi, sheet_name='5_银联', start_row=row_len1+3, start_col=2)
# # jx_yl_UPPE082_Bin
# Open_Excel2(df_yl082, path_psi, sheet_name='5_银联', start_row=row_len2+3, start_col=2)

# # xy_qj_C22180008_Bin
# row_len = Open_Excel2(df_qj008, path_psi, sheet_name='6_全景雷达', start_row=3, start_col=2)
# # xy_qj_B22170035_Bin
# row_len1 = Open_Excel2(df_qj035, path_psi, sheet_name='6_全景雷达', start_row=row_len+3, start_col=2)
# # xy_qj_B22170036_Bin
# row_len2 = Open_Excel2(df_qj036, path_psi, sheet_name='6_全景雷达', start_row=row_len1+3, start_col=2)
# # xy_qj_A22160001_Bin
# Open_Excel2(df_qj001, path_psi, sheet_name='6_全景雷达', start_row=row_len2+3, start_col=2)

# # score_201048_Bin
# row_len = Open_Excel2(df_201048, path_psi, sheet_name='7_融360', start_row=3, start_col=2)
# # score_202098_Bin
# row_len1 = Open_Excel2(df_202098, path_psi, sheet_name='7_融360', start_row=row_len+3, start_col=2)
# # score_202100_Bin
# row_len2 = Open_Excel2(df_202100, path_psi, sheet_name='7_融360', start_row=row_len1+3, start_col=2)
# # 蚁盾分2.0
# Open_Excel2(df_ydf2, path_psi, sheet_name='7_融360', start_row=row_len2+3, start_col=2)

# # 评分等级
# Open_Excel2(df_grade, path_psi, sheet_name='8.策略', start_row=3, start_col=2)

# path_model = 'F:/租后表分析数据_ld/风控周报/3_风控监控_模型_模板.xlsx'
# # 模型2311
# row_len = Open_Excel2(df_model_2311, path_model, sheet_name='11_转化_model_2311', start_row=3, start_col=2)
# row_len1 = Open_Excel2(df_model_2311_m, path_model, sheet_name='11_转化_model_2311', start_row=row_len+3, start_col=2)
# row_len2 = Open_Excel2(df_model_2311_mt, path_model, sheet_name='11_转化_model_2311', start_row=row_len1+3, start_col=2)
# row_len3 = Open_Excel2(df_model_2311_mc, path_model, sheet_name='11_转化_model_2311', start_row=row_len2+3, start_col=2)
# # 模型yl
# row_len = Open_Excel2(df_model_yl, path_model, sheet_name='12_转化_model_yl', start_row=3, start_col=2)
# row_len1 = Open_Excel2(df_model_yl_m, path_model, sheet_name='12_转化_model_yl', start_row=row_len+3, start_col=2)
# row_len2 = Open_Excel2(df_model_yl_mt, path_model, sheet_name='12_转化_model_yl', start_row=row_len1+3, start_col=2)
# row_len3 = Open_Excel2(df_model_yl_mc, path_model, sheet_name='12_转化_model_yl', start_row=row_len2+3, start_col=2)
# # 模型2401
# row_len = Open_Excel2(df_model_2401, path_model, sheet_name='13_转化_model_2401', start_row=3, start_col=2)
# row_len1 = Open_Excel2(df_model_2401_m, path_model, sheet_name='13_转化_model_2401', start_row=row_len+3, start_col=2)
# row_len2 = Open_Excel2(df_model_2401_mt, path_model, sheet_name='13_转化_model_2401', start_row=row_len1+3, start_col=2)
# row_len3 = Open_Excel2(df_model_2401_mc, path_model, sheet_name='13_转化_model_2401', start_row=row_len2+3, start_col=2)
# # 模型2501
# row_len = Open_Excel2(df_model_2501, path_model, sheet_name='14_转化_model_2501', start_row=3, start_col=2)
# row_len1 = Open_Excel2(df_model_2501_m, path_model, sheet_name='14_转化_model_2501', start_row=row_len+3, start_col=2)
# row_len2 = Open_Excel2(df_model_2501_mt, path_model, sheet_name='14_转化_model_2501', start_row=row_len1+3, start_col=2)
# row_len3 = Open_Excel2(df_model_2501_mc, path_model, sheet_name='14_转化_model_2501', start_row=row_len2+3, start_col=2)
